In [192]:
import hashlib
import hmac
import csv
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed
from multiprocessing import  Pool
import multiprocessing
import itertools
import random

In [193]:
from operator import itemgetter
import logging, sys

In [194]:
import warnings
warnings.filterwarnings("ignore")

In [195]:
_DEBUG = False
KAGGLE = False

In [196]:
# logging.basicConfig( #stream=sys.stdout, 
#                     level=logging.DEBUG)

In [197]:
# logging.debug('A debug message!')

In [198]:
pd.options.display.max_rows=500
pd.options.display.max_columns=500
pd.set_option('display.max_colwidth', None)

In [199]:
l1=[0,1,2,3]
l2=['a','b','c','d']
l3=['hello','how','are','you']

ll=[l1,l2,l3]
# ll[0]=l1
# ll[1]=l2
# ll[2]=l3


list(zip(*ll))

[(0, 'a', 'hello'), (1, 'b', 'how'), (2, 'c', 'are'), (3, 'd', 'you')]

In [200]:
import time
from multiprocessing import Value
from ctypes import py_object

In [201]:
def init_pool_processes(shared_value):
    global p
    p = shared_value
def parallelize_dataframe(df, func, data_to_share=None,other_args=None, n_cores=4):
    start = time.time()
    df_split = np.array_split(df, n_cores)
    
    if data_to_share is not None:
        p = Value(py_object)
        p.value = data_to_share        
        pool = Pool(processes=n_cores,
                    initializer=init_pool_processes, initargs=(p,))
    else:
        pool = Pool(n_cores)
        
    arg_list=[]
    arg_list.append(df_split)
            
    if other_args is not None:
        for arg in other_args:
            arg_list.append(itertools.repeat(arg))
    
    parallel_args = zip(*arg_list)
    end = time.time()
    print(f'Parallel preparation time: {end-start}')
    df = pd.concat(pool.starmap(func, parallel_args))
    pool.close()
    pool.join()
    return df

In [202]:
def debugprint(msg):
    if _DEBUG:
        print(msg)

In [203]:
def get_bin(bins,val):
    matching_bin=-1
    for i in range(len(bins)-1):
        if bins[i] <= val < bins[i+1]:
            matching_bin= bins[i]
    if matching_bin==-1:
        matching_bin=bins[-1]
    return matching_bin

In [204]:
def calculate_roll(server_seed,client_seed,nonce):
    # Compute the HMAC-SHA512 hash of string1 using string2 as the key
    
    string1 = f"{nonce}:{server_seed}:{nonce}"
    hmac_key =   f"{nonce}:{client_seed}:{nonce}".encode()
    hmac_hash = hmac.new(key=hmac_key,
                         msg=string1.encode(),
                         digestmod=hashlib.sha512).hexdigest()

    # Convert the first 8 characters of the HMAC-SHA512 hash to an integer
    string3 = hmac_hash[:8]
    number = int(string3, 16)

    # Compute the roll value
    roll = round(number / 429496.7295)
    
    return roll

In [205]:
def compute_roll_hash_arrays(server_seed_array,client_seed_array,nonce_array):
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                           client_seed_array,
                                           nonce_array)
    
    # Compute roll for hash using the hash which is the next element in array
    # Vectorize the function
    vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
                                           client_seed_array[:chain_length-1],
                                           nonce_array[:chain_length-1])
    return roll_array,roll_array_hash

    

In [206]:
# trans_36 = bytes((x ^ 0x36) for x in range(256))
# trans_36

In [207]:
chain_length = 5110101 #1000001

filename = "sha256_hashchain_b9556671f785fe935bee087665b4047e421ea4491a5e2021a8152cab0b74c953.npy"

if KAGGLE:
    filepath_client ="/kaggle/input/hash-file-generation-client-seed/"
else:
    filepath_client="data/"
    
# filename_client=f'{filepath_client}sha256_hashchain_client_5M.npy'
filename_client=f'{filepath_client}sha256_hashchain_client1.npy'


# Define the number of CPU cores to use
num_cores = 8

In [208]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [209]:
# server_seed_array=np.load(filename)
# print(len(server_seed_array))
# server_seed_array[0:10]

In [210]:
client_seed_array=np.load(filename_client,
                         allow_pickle=True,fix_imports=True,encoding='latin1')
client_seed_array= client_seed_array[:chain_length]
print(len(client_seed_array))
client_seed_array[0:10]

1010102


array(['3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       '57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e',
       '3e49b34f2d6c7116d92ac8818c94f87d5616d7f2aea5024c2f38413605dc321c',
       'a0731d74c9501894d16ba144bc10dd49e8439c7f88304bfdbe4a755870c46da3',
       '665122519e5835c8e7ebfa0ab5b1949d75ac2c9e3dc1a6965a4b6e26bfc4ed76',
       '87fa02b24749184f0ce9183f21715ce556db72ecd1ad4f32b02227b40f3b24ab',
       'edee47aed4d9b598fa647145a17e44f9d155dc53ad231c913ff5c93fc04e56ee',
       'baf9d66be4136fe8acd30c042704b1798380fe0f3ee45d5800ae01db88e03ed3',
       '8eac9ebdcbfcf1720f6b0317fa66569c3129d278c0e2605069e7c5a0a2179c8d',
       'aeab33d31e0fb10105ce7ad75e3a40736db70a824c3ed6489957d6736243184b'],
      dtype='<U64')

In [211]:
# str_cat ='pppppp'
# # str_cat ='ppppppppppppppp'
# print(len(str_cat))
# client_seed_array_1=(str_cat + pd.Series(client_seed_array)).values
# client_seed_array_1[:2]

In [212]:
# str_cat ='ppppppppppppppp'
str_cat ='woxpwoxpwoxpwoxpwoxp'
print(len(str_cat))
client_seed_array_2=(str_cat + pd.Series(client_seed_array)).values
client_seed_array_2[:2]

20


array(['woxpwoxpwoxpwoxpwoxp3455dda4b3aecaa36d4687277766a079feebbb4ab01dc038bc8fb8a36ddad6aa',
       'woxpwoxpwoxpwoxpwoxp57eea50e4484ba102d776506baf4b1dc0dcb3fc74fe2ab56b10b0f427cb6409e'],
      dtype=object)

In [213]:
np.random.seed(5000)
low = 1500
# nonce_array = np.random.randint(low, high=low+chain_length, 
#                                 size=chain_length)

nonce_array = np.arange(low,low+chain_length,1)
np.random.shuffle(nonce_array)
print(len(nonce_array))
print(nonce_array[:10])
pd.Series(nonce_array).nunique()

5110101
[ 638661 1297273 1057802 1410000 2886373 4139846 2635511 4657935 2336816
  640280]


5110101

In [214]:
# roll_array,roll_array_hash=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)
# print(len(roll_array[0:10]))
# print(roll_array[0:100])
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

In [215]:
# roll_array_1,roll_array_hash_1=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_1,
#                                        nonce_array)
# print(len(roll_array_1[0:10]))
# print(roll_array_1[0:100])
# print(len(roll_array_hash_1[0:10]))
# print(roll_array_hash_1[0:100])

In [216]:
# roll_array_2,roll_array_hash_2=compute_roll_hash_arrays(server_seed_array,
#                                        client_seed_array_2,
#                                        nonce_array)
# print(len(roll_array_2[0:10]))
# print(roll_array_2[0:100])
# print(len(roll_array_hash_2[0:10]))
# print(roll_array_hash_2[0:100])

In [217]:
# # Vectorize the function
# vectorized_calculate_roll = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array = vectorized_calculate_roll(server_seed_array,
#                                        client_seed_array,
#                                        nonce_array)

# # Print the output
# print(len(roll_array[0:10]))
# print(roll_array[0:100])

In [218]:
# # Vectorize the function
# vectorized_calculate_roll_hash = np.vectorize(calculate_roll)

# # Compute the roll values for the input arrays
# roll_array_hash = vectorized_calculate_roll_hash(server_seed_array[1:],
#                                        client_seed_array[:chain_length-1],
#                                        nonce_array[:chain_length-1])

# # Print the output
# print(len(roll_array_hash[0:10]))
# print(roll_array_hash[0:100])

#### Top k hash verification of client and server

In [219]:
# # server_hash_orig = "d100f357cf1b05cc23ddc35c5f34a960911f9e67582a6d5ff27d9a210137e9c0"
# # nonce= 2079
# # client_seed_orig = "ppppppd3f19c18078cea0c25643b203c9a56dc87cc5e71fa46c2cf5520ee8477f5b43c"
# # chain_length=100

# # server_hash_orig = "ffed0cf8eb7020128c153bea86b3f29bc328af0451052576211138023bb300bd"
# # nonce= 2078
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# # chain_length=100

# server_hash_orig = "68f1ed13e2f17e3d38fd888e470eaae28720675dc7c5913a6b7fd9586df159b4"
# nonce= 2116
# # client_seed_orig = "ppppppc34b5f7748ee0255fc2c64af17e5cc78cd65a9c302b34652c0aec87081c91f3d"
# client_seed_orig = "woxpwoxpwoxpwoxpwoxp"+"c023a89e8027c289bc936fb9a6be20aa5d1dcba191e70d98004e860fb6c0d5db"
# chain_length=1000



In [220]:
# Define a function to generate a single hash in the hash chain
def generate_hash(message):
    # Hash the message using SHA-256
    hash_obj = hashlib.sha256(message)

    # Get the hexadecimal representation of the hash
    hash_hex = hash_obj.hexdigest()

    # Return the hash
    return hash_hex

In [221]:
def generate_hash_chain(seed,chain_length):
    hash_list=[seed]
    message = seed.encode()
    for i in range(chain_length):
        hash_hex= generate_hash(message)
        # Write the hash to the CSV file
        hash_list.append(hash_hex)
        # Update the message with the current hash
        message = hash_hex.encode()
    return hash_list

def compute_multirolls(hash_list,client_seed,nonce):
    rolls=[]
    for cur_hash in hash_list:
        roll = calculate_roll(cur_hash,client_seed,nonce)
        rolls.append(roll)
    return rolls
def compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce):
    rolls=[]
    for cur_hash_nonce in hash_list_nonce:
        roll = calculate_roll(server_hash,client_seed,cur_hash_nonce)
        rolls.append(roll)
    return rolls

Function description 

Input
--------
Server Hash
Nonce
Client Seed

Output
----------
Category that has more number of successes - GT or LT <br>
How many number of success ?<br>

Return state of each condition: (Pass, Fail, Tie) <br>
100 values
-----------

1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin

1000 values
----------------
1. Total number of low values on the given category is higher than that of other category
2. This category has the least value
3. This category has the most number of low values 
4. This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
5. This category has the most number of low values which are less than or equal to / greater than or equal to matching hash value bin


In [222]:
ABOVE="Above"
BELOW="Below"
INTER ="Intermediate"
def set_tristate(val1,val2):
    if val1 == val2:
        below_state=-1
        above_state =-1
    else:
        below_state = val1 > val2
        above_state = int(~below_state)
        below_state = int(below_state)
    return below_state,above_state

# def set_diffstate(val1,val2):
#     below_state = val1-val2
#     above_state = -1 * below_state
#     return below_state,above_state
def compute_least_condition(bin_counts):
    mask = bin_counts.index < 5
    val_below = bin_counts[mask].min()
    val_above = bin_counts[~mask].min()
    #NOTE: here val_above and val_below are swapped in arguments
    #since least value in below state should be lesser than above state
    below_state,above_state=set_tristate(val_above,val_below)
    return below_state,above_state
def compute_mostlow_condition(bin_counts,low_values_cutoff):
    mask = bin_counts < low_values_cutoff
    mask2 = bin_counts.index < 5
    val_below = bin_counts[mask & mask2].count()
    val_above = bin_counts[mask & ~mask2].count()
    
#     print(f'val_below:{val_below} val_above:{val_above}')
    below_state,above_state=set_tristate(val_below,val_above)
    return below_state,above_state

def compute_conditions(rolls_arr,size,initial_value=None):
    
    rolls_arr = rolls_arr[:size]
    low_values_cutoff = size * 0.1
    
    below_conditions = np.full(5,-100)
    above_conditions = np.full(5,-100)
    #Condition 1: 
    #Total number of low values on the given category is higher than that of other category 
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts_cond1 = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Sum bincounts')
    debugprint(bin_counts_cond1)
#     print(bin_counts.loc[0],bin_counts.loc[1])
    
    below_conditions[0],above_conditions[0]=set_tristate(bin_counts_cond1.loc[1],bin_counts_cond1.loc[0])
    
    bins = list(range(0,10001,1000))
    labels = list(range(0,10))
    bin_counts = pd.cut(rolls_arr, bins,labels=labels).value_counts()
    debugprint('Full bincounts')
    debugprint(bin_counts)
    #Condition 2: 
    #This category has the least value
    below_conditions[1],above_conditions[1]=compute_least_condition(bin_counts)
    
    #Condition 3: 
    #This category has the most number of low values
    below_conditions[2],above_conditions[2]=compute_mostlow_condition(bin_counts,low_values_cutoff)
    
#     print(f'INITIAL roll:  {rolls_arr[0]}')g
    if initial_value is None:
        initial_value=rolls_arr[0]
    initial_bin = get_bin(labels,int(initial_value/1000))
    
    if initial_bin<5:
        mask_filt = bin_counts.index > initial_bin
    else:
        mask_filt = bin_counts.index < initial_bin
    debugprint(f'filtered bincounts with initial bin:{initial_bin}')
    debugprint(bin_counts[mask_filt])
    #Condition 4: 
    #This category has the least value which are less than or equal to / greater than or equal to matching hash value bin
    below_conditions[3],above_conditions[3]=compute_least_condition(bin_counts[mask_filt])
    
    #Condition 5: 
    #This category has the most number of low values
    below_conditions[4],above_conditions[4]=compute_mostlow_condition(bin_counts[mask_filt],
                                                                      low_values_cutoff)
   
    return below_conditions,above_conditions,bin_counts_cond1,bin_counts
    
def predict_seed_output(server_hash,nonce,client_seed,
                        hash_list_server=None,hash_list_nonce=None,
                        initial_value=None):
    chain_length=2000
    if (hash_list_server is None) and (hash_list_nonce is None):
        hash_list_server = generate_hash_chain(server_hash,chain_length)
#     print('Hash chain completed')
    if hash_list_nonce is not None:
        rolls_list  = compute_multirolls_nonce(server_hash,client_seed,hash_list_nonce)
    else:
        rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
#     print('Roll computation completed')
    #100 values conditions
    rolls_arr=np.array(rolls_list)
    cond_100_below,cond_100_above,\
    bin_counts_cond1_100,bin_counts_100 = compute_conditions(rolls_arr,100,initial_value)
    cond_1000_below,cond_1000_above,\
    bin_counts_cond1_1000,bin_counts_1000 = compute_conditions(rolls_arr,1000,initial_value)
    cond_2000_below,cond_2000_above,\
    bin_counts_cond1_2000,bin_counts_2000 = compute_conditions(rolls_arr,2000,initial_value)
#     cond_4000_below,cond_4000_above = compute_conditions(rolls_arr,4000)
    
    cond_below_all = np.concatenate((cond_100_below,cond_1000_below,
                                     cond_2000_below,
#                                      cond_4000_below
                                    ))
    cond_above_all = np.concatenate((cond_100_above,cond_1000_above,
                                     cond_2000_above,
#                                      cond_4000_above
                                    ))
    
    debugprint(cond_below_all)
    debugprint(cond_above_all)
    
    below_passes = (cond_below_all == 1).sum()
    above_passes = (cond_above_all == 1).sum()
    
    if below_passes > above_passes:
        cond_all =  cond_below_all
        passed_state = BELOW
        total_passes = below_passes
    else:
        cond_all = cond_above_all
        passed_state = ABOVE
        total_passes = above_passes
  
    diff = abs(below_passes-above_passes)
    bin_counts_list = [bin_counts_cond1_100,bin_counts_100,
                      bin_counts_cond1_1000,bin_counts_1000,
                      bin_counts_cond1_2000,bin_counts_2000]
    
    if (bin_counts_cond1_100.loc[-1] < 5) & \
    (bin_counts_cond1_1000.loc[-1] < 50) & (bin_counts_cond1_2000.loc[-1] < 100):
        passed_state = INTER
    

    return cond_all,diff,total_passes,passed_state,bin_counts_list



In [223]:
def compare_prev(server_seed_array,
                client_seed,
                nonce_array,value_to_match=None):
#     print('compare_prev STARTED')
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(server_seed_array,
                                       client_seed,
                                       nonce_array)
    if value_to_match is None:
        value_to_match = roll_array[0]
    indices =np.where(roll_array==value_to_match)[0]
    indices_prev = indices -1 
#     print(len(indices_prev))
#     print(np.sort(roll_array[indices_prev]))
    prev_roll_match = roll_array[indices_prev]
    bins = [0,4750,5250,10000]
    labels = [0,-1,1]
    bin_counts = pd.cut(roll_array, bins,labels=labels).value_counts()
#     print('Sum bincounts')
#     print(bin_counts)
    
    return bin_counts

def validate_test_topk(test,client_seed_data,pass_exp1,pass_exp2,
                       pass_state_reqd=None,last_index=None,result_df=None,
                       client_char_count=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    bincountcols=[]
    bincountprevcols=[]
    sumbincols = ['lt4750','inter','gt5250']
    
    for col in sumbincols:
        bincountprevcols.append('prev_'+str(col))
    bincountprevcols_nonce =['nonce_'+col for col in bincountprevcols]
    for hash_size in [100,1000,2000]:
        for col in sumbincols:
            bincountcols.append(str(hash_size)+'_'+str(col))
        for col in range(10):
            bincountcols.append(str(hash_size)+'_'+str(col))
#     print(bincountcols)

    bincountcols_nonce = ['nonce_'+col for col in bincountcols]
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'total_passes','passed_state',
                                    'total_passes_1','total_passes_2',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] + bincountprevcols+bincountprevcols_nonce +\
                                             bincountcols+bincountcols_nonce)
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
#         if server_count % 10==0:
#             i = len(client_seed_data)-1
#             print(f'server_count:{server_count}')
        
#         if not result_success:
#             i= last_success_index
#             i = len(client_seed_data)-1
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        chain_length=2000
        cur_hash_list = generate_hash_chain(cur_hash,chain_length)
        cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
#         cur_hash_list_rev = generate_hash_chain(cur_hash[::-1],chain_length)
        
#         client_seed_data=np.array(cur_hash_list[1:])
#         client_seed_data=(str(cur_nonce) + pd.Series(client_seed_data)).values
        
##         cur_hash_list=list((cur_hash + pd.Series(np.array(cur_hash_list))).values)
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 1000==0:
                print(f'current scan : {i}')
            
            if client_char_count is None:
                client_seed = client_seed_data[i]
            else:
                client_seed = client_seed_data[i][:client_char_count]

#             client_seed= generate_hash((client_seed_data[i] + prev_match).encode())
#             client_seed= generate_hash((str(cur_nonce)+client_seed_data[i]).encode())
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            if (3000<=roll_hash<=7000) or ((i+1) in last_indices):
                total_passes=-1
                passed_state=ABOVE
                passed_state_2=BELOW
 
            else:
                cond_all,diff,total_passes_1,passed_state,bin_counts_list = predict_seed_output(cur_hash,
                                                                              cur_nonce,
                                                                              client_seed,
                                                                             hash_list_server=cur_hash_list)
                #This condition is to reduce time execution
                total_passes_2=0
                passed_state_2=passed_state
                if (pass_exp2>=total_passes_1):
                    cond_all_2,diff_2,total_passes_2,passed_state_2,bin_counts_list_nonce = predict_seed_output(cur_hash,
                                                                                  cur_nonce,
                                                                                  client_seed,
                                                                                 hash_list_nonce=cur_hash_list_nonce)
#                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_nonce_2 = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              hash_list_nonce=cur_hash_list_nonce)
# #                 cond_all_3,diff_3,total_passes_3,passed_state_3,bin_counts_list_rev = predict_seed_output(cur_hash,
#                                                                               cur_nonce,
#                                                                               client_seed,
#                                                                              cur_hash_list_rev)
                total_passes = total_passes_1 + total_passes_2  

            i-=1
#             if (total_passes_1>=pass_exp1) & (total_passes_2>=pass_exp2) & (passed_state==passed_state_2):
#             if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
#                 & (last_index!=i+1):
            if (pass_exp2>=total_passes>=pass_exp1) & (passed_state==passed_state_2) \
                    & (passed_state!=INTER) & ((pass_state_reqd==None) | (pass_state_reqd==passed_state) ):
#                 print(f'Matching client at {i+1} for seed:{cur_seed} nonce:{cur_nonce}')
                match=True
#                 rand = random.randint(1,2)
#                 print(f'random number:{rand}')
#                 if rand==1:
#                     match=True
            elif (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
#             print('Hash Chain starts')
            server_hash_list = generate_hash_chain(cur_hash,10)
            bincount_prev = compare_prev(server_hash_list,client_seed,cur_nonce,value_to_match=roll_hash)
#             print('Nonce Hash Chain starts')
            nonce_hash_list = generate_hash_chain(str(cur_nonce),10)
            bincount_prev_nonce = compare_prev(cur_hash,client_seed,nonce_hash_list,value_to_match=roll_hash)
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            bincount_row =[]
            for bincount_df in bin_counts_list:
                bincount_row.extend(list(bincount_df.values))
#             print(bincount_row)
            bincount_row_nonce =[]
            for bincount_df in bin_counts_list_nonce:
                bincount_row_nonce.extend(list(bincount_df.values))
#             print(bincount_row_nonce)
            result_success = ((passed_state==ABOVE) & (roll_seed_actual>5250))  \
                            | ((passed_state==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,total_passes,passed_state,
                                           total_passes_1,total_passes_2,
                                           roll_seed_actual,roll_hash,roll_blank_server] + \
                                            list(bincount_prev.values) + list(bincount_prev_nonce.values) +\
                                            bincount_row + bincount_row_nonce
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,total_passes,passed_state,roll_seed_actual,roll_hash}')
          
    return result_df
            
    

In [224]:
def predict_digit_pattern(server_hash,roll_hash,nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=None,mismatch_digit_indices=None):

#     rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
#     roll_first = rolls_list[0]
    factors = np.array([10000,1000,100,10])
    factors_filt = factors[match_digit_indices]
    for i,val in enumerate(factors_filt):
        match = match & (int((roll_hash % val) / (val / 10))==match_digit_arr[i])
        if not match: 
            break

    if mismatch_digit_arr is not None:
        factors_filt = factors[mismatch_digit_indices]
        for i,val in enumerate(factors_filt):
            match = match & (int((roll_hash % val) / (val / 10))!=mismatch_digit_arr[i])
            if not match: 
                break            

    return match
 

def predict_digit_output(server_hash,nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        hash_list_server,match_digit_indices=None):

    rolls_list  = compute_multirolls(hash_list_server,client_seed,nonce)
    match = True
    
    #eliminate zero from all roll_hash
#     for roll in rolls_list:
#         str_roll = f'{roll:04d}'
#         cur_count = str_roll.count('0')
#         if cur_count!=0:
#             match=False
            
    for roll,match_digit,match_count,match_indices \
            in zip(rolls_list,match_digit_arr,match_digit_count_arr,match_digit_indices):
        str_roll = f'{roll:04d}'
        
        if match_digit_indices is None:
            str_roll_indexed = str_roll
        else:
            str_roll_indexed = ''
            for index in match_indices:
                str_roll_indexed = str_roll_indexed + str_roll[int(index)]
                
        cur_count = str_roll_indexed.count(match_digit)
        if cur_count!=match_count:
            match=False
            
    return match, rolls_list

def validate_digits(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    feature_chain_length=20,
                    match_digit_indices=None,
                    last_index=None,result_df=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list',
                                         ])
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
      
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
        
            match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
                         match_digit_arr,match_digit_count_arr,
                        cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    if not match:
                        print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            if result_success:
                last_success_index= i+1
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,pass_state_reqd,
                                           roll_seed_actual,rolls_list]
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
          
    return result_df
  

In [225]:

def validate_test_topk_2(test,client_seed_data,hash_count_to_check,pass_state_reqd=BELOW,
                       last_index=None,result_df=None):
    i = len(client_seed_data)-1 #if last_index is None else last_index - 1
    last_success_index =i
    # client_seed_data = client_seed_data[::-1]
    client_size = len(client_seed_data)
    client_scan_fresh=True 

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index','passed_state',
                                    'roll_actual','roll_hash','roll_blank_server'
                                         ] )
    result_success=True
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
        cur_hash_list = generate_hash_chain(cur_hash,hash_count_to_check+1)
#         cur_hash_list_nonce = generate_hash_chain(str(cur_nonce),chain_length)
        
        fullclientscan=False
        match = False
        prev_match=""
        while not match and not fullclientscan:
            
            if i % 10000==0:
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i] 
            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hashes=[]
            for hash_chain_val in cur_hash_list:
                roll_hashes.append(calculate_roll(hash_chain_val,client_seed,cur_nonce))
            roll_hashes_arr = np.array(roll_hashes)
            
            roll_hashes_arr_2 = roll_hashes_arr[:len(roll_hashes_arr)-1]

            if pass_state_reqd==BELOW:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]<4750)
                mask = roll_hashes_arr_2<5250
#                 roll_hashes_arr_1stdigit = (roll_hashes_arr_2  / 1000).astype('int')
#                 flag_check_all_1000 = True
#                 for val1000 in range(6,10):
#                     flag_check_all_1000 = flag_check_all_1000 & len(roll_hashes_arr_1stdigit[roll_hashes_arr_1stdigit==val1000])>0
                    
            else:
                last_rev = (roll_hashes_arr[len(roll_hashes_arr)-1]>5250)
                mask = roll_hashes_arr_2>4750
            
            match = (len(roll_hashes_arr_2[mask])==0) #& last_rev # & flag_check_all_1000
            roll_hash = roll_hashes_arr[0]
            i-=1
            if (i<0):
                i = len(client_seed_data)-1
                if client_scan_fresh:
                    fullclientscan=True
                    print(f'No matching client for seed:{cur_seed} nonce:{cur_nonce}')
        
        if match:
            roll_blank_server=calculate_roll('',client_seed,cur_nonce)
            
            result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                            | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
            result = [result_success,cur_seed, cur_hash, cur_nonce,
                                           client_seed,i+1,BELOW,
                                           roll_seed_actual,roll_hash,roll_blank_server] 
            result_df.loc[len(result_df.index)] = result
            print(f'server_count: {server_count} current result: {result_success,cur_nonce,roll_seed_actual,roll_hashes_arr}')
          
    return result_df
            
    

In [226]:
# test_file_random ='/kaggle/input/hash-generate-random-seeds/test_data_random.csv'
# test_seeds_2=pd.read_csv(test_file_random)
# print(test_seeds_2.shape)
# test_seeds_2.head()

In [227]:
# test_file ='/kaggle/input/hashdata/server seed test data 4.csv'
# test_seeds_4=pd.read_csv(test_file)
# print(test_seeds_4.shape)
# test_seeds_4.head()

In [228]:
# test_file ='/kaggle/input/hashdata/server seed test data 5.csv'
# test_seeds_5=pd.read_csv(test_file)
# print(test_seeds_5.shape)
# test_seeds_5.head()

In [229]:
# #create special client seed pattern array

# def get_hexval(val):
#     hexdata='abcdef'
#     if val >=10:
#         return hexdata[val-10]
#     else:
#         return str(val)

# str_spl_client = 'woxpwoxpwoxpwoxpwoxp066103c1b2a6ebe01cf30afd49a6b931278793fc457dee84510f03e11779d5be'
# spl_client_list=[]
# for index in range(20,20+64):
#     for val in range(16):
#         spl_client_list.append(str_spl_client[:index]+get_hexval(val)+str_spl_client[index+1:])
# spl_client_array = np.array(spl_client_list)
# print(len(spl_client_array))
# spl_client_array[1020:]

##### Machine Learning

Generate Data

In [230]:
def generate_data(test,client_seed_data,pass_state_reqd,
                    match_digit_arr,match_digit_count_arr,
                    mismatch_digit_arr=None,mismatch_digit_indices=None,
                    feature_chain_length=20,
                    is_data_hash=False,
                    match_digit_indices=None,
                    match_count_exp=1,
                    test_limit = None,
                    last_index=None,result_df=None,
                    print_client_scan=False,
                    hash_list_nonce=None):
    i = len(client_seed_data)-1 if last_index is None else last_index - 1
    last_success_index =i
    client_size = len(client_seed_data)
    client_scan_fresh=True 
    

    if result_df is None:
        result_df = pd.DataFrame(columns=['match','seed','hash','nonce',
                                    'client_seed','client_index',
                                    'passed_state',
                                    'roll_actual','roll_hash_list','roll_hash_list_nonce',
                                         ])
    result_success=True
    dictionary_list=[]
    for server_count,(server_index,row) in enumerate(test.iterrows()):
        
        if (server_count % 5)==0:
            print(f'server_count:{server_count}')
        
        if (test_limit is not None) and (server_count >= test_limit):
            break
        
        #for large volume of data, it is recommended to set i to entire lengthof client seeds
        #for every server seed
#         i = len(client_seed_data)-1 if last_index is None else last_index - 1
        
        cur_seed = row['seed']
        cur_nonce = row['nonce']
        cur_hash= row['hash']
        
       
        if is_data_hash:
            cur_hash_list = list(test[server_count+1:server_count+1+feature_chain_length+1]['seed'])
        else:
            cur_hash_list = generate_hash_chain(cur_hash,feature_chain_length)
        
          
        fullclientscan=False
        match = False
        prev_match=""
        match_count=0
        
        client_start = i
        while match_count < match_count_exp and not fullclientscan:
            
            if print_client_scan  & (i % 1000==0) :
                print(f'current scan : {i}')
            
            client_seed = client_seed_data[i]
            client_index = i

            roll_seed_actual=calculate_roll(cur_seed,client_seed,cur_nonce)
            roll_hash=calculate_roll(cur_hash,client_seed,cur_nonce)
            
            match = predict_digit_pattern(cur_hash,roll_hash,cur_nonce,client_seed,
                        match_digit_arr,match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,
                        mismatch_digit_indices=mismatch_digit_indices)
        
            # match, rolls_list = predict_digit_output(cur_hash,cur_nonce,client_seed,
            #              match_digit_arr,match_digit_count_arr,
            #             cur_hash_list,match_digit_indices=match_digit_indices)
            i-=1
            
            if i==client_start and not match:
                print()
                print(i,client_start)
                print(f'No further match for seed:{cur_seed} nonce:{cur_nonce} match count: {match_count}')
                print()
                fullclientscan=True
            if (i<0):
                i = len(client_seed_data)-1
       
            if match:
                
#                 print(f'Matched for seed:{cur_seed} i:{i} match count: {match_count}')
                rolls_list  = compute_multirolls(cur_hash_list,client_seed,cur_nonce)
                if hash_list_nonce is None:
                    hash_list_nonce = generate_hash_chain(str(cur_nonce),feature_chain_length)

                rolls_list_nonce  = compute_multirolls_nonce(cur_hash,client_seed,hash_list_nonce)

                match_count += 1
                
                result_success = ((pass_state_reqd==ABOVE) & (roll_seed_actual>5250))  \
                                | ((pass_state_reqd==BELOW) & (roll_seed_actual<4750))
                if result_success:
                    last_success_index= i+1
#                 result = [result_success,cur_seed, cur_hash, cur_nonce,
#                                                client_seed,client_index,pass_state_reqd,
#                                                roll_seed_actual,rolls_list,rolls_list_nonce]
                

                dictionary_data = {'match': result_success,'seed':cur_seed, 
                                   'hash': cur_hash, 'nonce':cur_nonce,
                                    'client_seed':client_seed,'client_index':client_index,
                                      'passed_state':pass_state_reqd,
                                     'roll_actual':roll_seed_actual,
                                   'roll_hash_list':rolls_list,'roll_hash_list_nonce':rolls_list_nonce
                                  }
                dictionary_list.append(dictionary_data)
                
#                 result_df.loc[len(result_df.index)] = result
#                 print(f'server_count: {server_count} current result: {result_success, cur_nonce,pass_state_reqd,roll_seed_actual,roll_hash}')
    
    result_df = pd.DataFrame.from_dict(dictionary_list)

    return result_df


In [231]:
hashval = 'a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
seed = hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
nonce = 2677 #2390

feature_chain_length=60
train_hash_size = 2158#121
train_client_size = 2000 #20

In [232]:
analysis_df=None

In [233]:
cur_hash_list = generate_hash_chain(hashval,train_hash_size)

train_data = pd.DataFrame(columns=['seed','hash','nonce'])
start=24
train_data['seed']=np.array(cur_hash_list[start:-1])
train_data['hash']=np.array(cur_hash_list)[start+1:]
train_data['nonce'] = nonce

print(len(train_data))
train_data.head()

2134


,seed,hash,nonce
0,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677
1,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,78392e401b0996ce99afaf8ca4dc08457a11af594810880cee87866d863f7400,2677
2,78392e401b0996ce99afaf8ca4dc08457a11af594810880cee87866d863f7400,27a8f3d3af9f07dae5691046573a7bd39748297676a32915cbf396d964a9472a,2677
3,27a8f3d3af9f07dae5691046573a7bd39748297676a32915cbf396d964a9472a,e86606a181f4989df87783084f7e9e7c51d8a21fd4a3abea3300b4f0c4d0065c,2677
4,e86606a181f4989df87783084f7e9e7c51d8a21fd4a3abea3300b4f0c4d0065c,317a0ca95eb93e9233d8e2933dcc7db7a05a5aaf97fe8928f0f34ec42dddba1e,2677


In [234]:
PATTERN_99=False
HIGH_ANALYSIS=False

cur_digit = 9
cur_zero_digit = 0

In [235]:
READ_FROM_FILE=False

In [236]:
%%time
# match_digit_arr = np.array(['999',])
# match_digit_count_arr=np.array([1])
# match_digit_indices = np.array(['012'])

if PATTERN_99:
    match_digit_arr=np.array([9,9])
    mismatch_digit_arr=np.array([9,9])
else:
    match_digit_arr=np.array([0,0])
    mismatch_digit_arr=np.array([0,0])
    
match_digit_indices = np.array([1,2])
match_digit_count_arr=np.array([1])
mismatch_digit_indices = np.array([0,3])

CPU times: user 14 µs, sys: 0 ns, total: 14 µs
Wall time: 17.2 µs


In [237]:
%%time

if not READ_FROM_FILE:

    cur_client_seed_array = client_seed_array_2[:1000001]

    results_df = None

    last_index =  len(cur_client_seed_array)-1 #1024
    start = 0

    hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)

    results_df = generate_data(train_data,
                                cur_client_seed_array,
                                 ABOVE,
                        match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                        mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                        match_count_exp=train_client_size,
                        feature_chain_length=feature_chain_length,
                        is_data_hash = True,
                        test_limit=38,
                        last_index=last_index,result_df=results_df,
                        hash_list_nonce=hash_list_nonce)
    row = results_df.iloc[len(results_df)-1]
    last_index = row['client_index']

server_count:0
server_count:5
server_count:10
server_count:15
server_count:20
server_count:25
server_count:30
server_count:35
CPU times: user 4min 20s, sys: 218 ms, total: 4min 20s
Wall time: 4min 20s


In [238]:
# last_index
# results_df['client_index'].min()

In [239]:
# results_df.head()

In [240]:
# print(len(results_df))
# results_df['client_seed'].nunique()

In [241]:
%%time
if PATTERN_99:
    file_pattern_str = 'x99x'
else:
    file_pattern_str = 'x00x'
filename = f'data/results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
if READ_FROM_FILE:
    results_df = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                 'roll_hash_list_nonce': pd.eval})
else:
    results_df.to_csv(filename,index=False)

CPU times: user 2.2 s, sys: 27.9 ms, total: 2.23 s
Wall time: 2.23 s


In [242]:
def save_cleaned_results_df(results_df,is_test):
    nonce_suffix = "_nonce"
    rolls_df = results_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    rolls_df_nonce = results_df['roll_hash_list_nonce'].apply(pd.Series)
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df.shape[1])]
    results_df_cleaned = pd.concat([results_df,rolls_df,rolls_df_nonce],axis=1)
    results_df_cleaned=results_df_cleaned.drop(['roll_hash_list','roll_hash_list_nonce'],axis=1)
    if is_test:
        filename = f'data/cleaned_results_df_test_{nonce}_Large_{file_pattern_str}_pattern.csv'
    else:
        filename = f'data/cleaned_results_df_{nonce}_Large_{file_pattern_str}_pattern.csv'
    results_df_cleaned.to_csv(filename,index=False)
    return results_df_cleaned

In [243]:
results_df.head()

,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_hash_list,roll_hash_list_nonce
0,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxpbca576ff189fdff8ab60527c66636322cb708225d24eb13291839f6baecad37d,999914,Above,6540,"[8005, 7909, 6435, 1879, 7496, 2959, 7072, 8501, 9416, 5509, 6792, 3002, 1274, 1804, 3972, 5530, 2335, 7330, 6817, 8424, 4051, 4816, 5393, 5057, 3043, 8046, 4595, 8761, 3961, 6047, 6266, 9955, 1828, 6384, 2047, 3041, 7816, 9014, 6702, 4484, 9269, 736, 5576, 7424, 2796, 4840, 9935, 7051, 5045, 9859, 3059, 8334, 7562, 6472, 6848, 6054, 8076, 3263, 6917, 8389, 9570]","[8005, 59, 18, 9930, 3775, 3984, 9337, 8417, 8548, 3040, 9676, 7842, 6570, 4618, 2191, 5410, 2165, 4175, 2408, 781, 471, 2245, 6545, 9612, 6865, 2793, 4414, 4884, 9709, 7991, 8892, 3744, 6436, 9328, 104, 3940, 4118, 5327, 477, 3345, 5433, 7766, 2964, 6263, 8897, 6629, 8385, 6791, 5517, 4199, 9678, 1357, 1680, 4168, 7707, 2538, 826, 4294, 2511, 13, 3630]"
1,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxp7fc8b3c9f7e5b3f5a469afbf55fb255a1e806b88cbf8e74186a052763472bf93,999586,Above,9905,"[3002, 1623, 39, 8296, 6077, 1944, 1408, 4627, 9552, 2682, 4402, 2222, 8977, 8048, 5371, 1037, 5668, 5354, 9824, 9854, 1143, 7836, 7989, 1398, 4883, 1181, 1110, 2219, 2621, 9240, 7950, 2978, 4856, 1810, 9139, 2046, 5341, 4421, 1739, 717, 9753, 3220, 1825, 690, 2429, 7001, 7142, 4753, 1106, 1040, 7407, 8939, 8884, 9624, 6155, 6427, 4367, 4057, 5291, 5211, 7054]","[3002, 9131, 4378, 3999, 9832, 774, 903, 9112, 7300, 5924, 7506, 3085, 436, 9209, 6337, 8281, 2416, 6216, 478, 2152, 3049, 1268, 4038, 8513, 5287, 90, 2064, 6119, 4321, 1165, 6800, 4074, 2844, 1978, 4041, 7538, 6354, 1071, 9343, 5864, 3577, 2362, 4314, 8649, 4114, 6040, 343, 8063, 4566, 6137, 5227, 9348, 8164, 7574, 8124, 8734, 2589, 8353, 3509, 5347, 8770]"
2,False,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxp4126b2a01da436318a635a2828e5928f24146bd003ea1ffea08eeb6d3c66eed7,999383,Above,1785,"[4006, 8062, 8796, 2709, 2034, 5096, 2251, 5167, 5415, 4240, 9311, 7288, 5481, 324, 2872, 8530, 8724, 9806, 2043, 2585, 3758, 8332, 594, 51, 7006, 2077, 1750, 4736, 1243, 9823, 7835, 8410, 1849, 73, 9474, 4620, 5123, 9757, 1443, 5274, 6414, 9884, 5085, 4100, 7255, 4169, 4070, 4754, 621, 9590, 9264, 2727, 4191, 3788, 3887, 5051, 6683, 6701, 9300, 4034, 9776]","[4006, 1519, 9859, 6244, 3178, 1485, 5191, 5605, 4683, 5753, 1258, 7290, 7321, 1907, 9798, 3446, 4014, 1608, 2879, 2536, 3631, 5721, 5398, 9834, 5731, 1109, 6974, 5849, 2242, 1875, 3720, 530, 9622, 9986, 1575, 1268, 7938, 7942, 7930, 4036, 1770, 8690, 6460, 3749, 1, 6010, 8569, 2118, 12, 9690, 1799, 4881, 5800, 1214, 9304, 8018, 453, 8237, 6895, 3354, 8202]"
3,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxp04e7109cfe8eded6c71722e2d180917f18db03a931ef1e056942ae2f99838883,999222,Above,5339,"[8004, 9001, 3796, 5269, 3993, 7856, 2397, 9743, 9009, 300, 1572, 7535, 7243, 2326, 2461, 2389, 4359, 9759, 3181, 5491, 109, 1318, 7564, 9714, 2619, 2627, 9128, 4458, 1235, 3388, 2295, 4706, 4887, 7259, 1319, 7143, 1583, 8485, 4873, 2253, 8270, 1218, 3202, 6591, 9583, 8651, 8433, 8708, 2365, 246, 9924, 4083, 1430, 1239, 7644, 1065, 3528, 2343, 4880, 5550, 593]","[8004, 3676, 5028, 4747, 5589, 9587, 8641, 3263, 4988, 5840, 7711, 3964, 402, 1758, 9943, 3699, 3151, 2347, 6225, 3907, 1656, 3577, 6212, 7792, 3392, 1720, 1852, 9549, 7287, 2175, 7845, 9838, 8816, 2346, 141, 6867, 2693, 7080, 2654, 7785, 7463, 7454, 6470, 4442, 143, 2523, 8445, 8514, 6171, 1226, 7893, 3335, 8086, 6989, 2487, 1032, 2603, 6445, 6771, 24, 1637]"
4,Tr

In [244]:
%%time
results_df_cleaned=save_cleaned_results_df(results_df,False)
results_df_cleaned.head()

CPU times: user 22.9 s, sys: 485 ms, total: 23.4 s
Wall time: 23.2 s


,match,seed,hash,nonce,client_seed,client_index,passed_state,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce
0,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxpbca576ff189fdff8ab60527c66636322cb708225d24eb13291839f6baecad37d,999914,Above,6540,8005,7909,6435,1879,7496,2959,7072,8501,9416,5509,6792,3002,1274,1804,3972,5530,2335,7330,6817,8424,4051,4816,5393,5057,3043,8046,4595,8761,3961,6047,6266,9955,1828,6384,2047,3041,7816,9014,6702,4484,9269,736,5576,7424,2796,4840,9935,7051,5045,9859,3059,8334,7562,6472,6848,6054,8076,3263,6917,8389,9570,8005,59,18,9930,3775,3984,9337,8417,8548,3040,9676,7842,6570,4618,2191,5410,2165,4175,2408,781,471,2245,6545,9612,6865,2793,4414,4884,9709,7991,8892,3744,6436,9328,104,3940,4118,5327,477,3345,5433,7766,2964,6263,8897,6629,8385,6791,5517,4199,9678,1357,1680,4168,7707,2538,826,4294,2511,13,3630
1,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxp7fc8b3c9f7e5b3f5a469afbf55fb255a1e806b88cbf8e74186a052763472bf93,999586,Above,9905,3002,1623,39,8296,6077,1944,1408,4627,9552,2682,4402,2222,8977,8048,5371,1037,5668,5354,9824,9854,1143,7836,7989,1398,4883,1181,1110,2219,2621,9240,7950,2978,4856,1810,9139,2046,5341,4421,1739,717,9753,3220,1825,690,2429,7001,7142,4753,1106,1040,7407,8939,8884,9624,6155,6427,4367,4057,5291,5211,7054,3002,9131,4378,3999,9832,774,903,9112,7300,5924,7506,3085,436,9209,6337,8281,2416,6216,478,2152,3049,1268,4038,8513,5287,90,2064,6119,4321,1165,6800,4074,2844,1978,4041,7538,6354,1071,9343,5864,3577,2362,4314,8649,4114,6040,343,8063,4566,6137,5227,9348,8164,7574,8124,8734,2589,8353,3509,5347,8770
2,False,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be,2677,woxpwoxpwoxpwoxpwoxp4126b2a01da436318a635a2828e5928f24146bd003ea1ffea08eeb6d3c66eed7,999383,Above,1785,4006,8062,8796,2709,2034,5096,2251,5167,5415,4240,9311,7288,5481,324,2872,8530,8724,9806,2043,2585,3758,8332,594,51,7006,2077,1750,4736,1243,9823,7835,8410,1849,73,9474,4620,5123,9757,1443,5274,6414,9884,5085,4100,7255,4169,4070,4754,621,9590,9264,2727,4191,3788,3887,5051,6683,6701,9300,4034,9776,4006,1519,9859,6244,3178,1485,5191,5605,4683,5753,1258,7290,7321,1907,9798,3446,4014,1608,2879,2536,3631,5721,5398,9834,5731,1109,6974,5849,2242,1875,3720,530,9622,9986,1575,1268,7938,7942,7930,4036,1770,8690,6460,3749,1,6010,8569,2118,12,9690,1799,4881,5800,1214,9304,8018,453,8237,6895,3354,8202
3,True,3a2f8b8d70e8c21a2c7e77f36ac7211cec947e0df3ca69451d1ada23d092b271,d98d

In [245]:
HIGH_TARGET= 9000 #8944
LOW_TARGET = 1000 #1056
ZERO_COL_CUTOFF = 4

In [246]:
def create_target_mask(data,bHighCheck=HIGH_ANALYSIS):
    if bHighCheck:
        mask = data['roll_actual'] > HIGH_TARGET
    else:
        mask = data['roll_actual'] < LOW_TARGET   
    return mask

In [247]:
def generate_features(initial_df,istrain):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
#     for col in list(rolls_df.columns)[1:]:
#         rolls_df[f'{col}_dig2'] = ((rolls_df[col] % 1000) / 100).astype('int')
    rolls_df['count_lt_1000'] = (rolls_df < 1000).sum(axis=1)
    rolls_df['count_gt_5000'] = (rolls_df > 5000).sum(axis=1)-1
    
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual']],rolls_df],axis=1)
        mask = create_target_mask(features_df)
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
#         features_df=features_df.drop(['roll_actual'],axis=1)

        features_df['roll_hash_list_nonce']=initial_df['roll_hash_list_nonce']
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = rolls_df
    return features_df


In [248]:
train_manual = generate_features(results_df[25*train_client_size:50*train_client_size],True)
# train_manual = generate_features(results_df,True)
train_manual.tail()

0    23446
1     2554
Name: target, dtype: int64


,roll_actual,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_5000,target,roll_hash_list_nonce
75995,602,2004,7687,1914,8228,9822,6041,2287,6065,8675,7473,7455,2417,1225,8943,9973,215,219,3176,6059,4330,3588,6813,2637,8346,2100,7828,6199,8083,185,3062,357,67,40,3813,9911,72,2161,6470,32,7585,9680,9769,3507,2155,7571,8592,7715,8782,2343,2024,6685,7556,8931,3179,312,2938,2113,6695,3448,1363,9065,9,29,1,"[2004, 2177, 9166, 1421, 7900, 6359, 6981, 4959, 1063, 843, 5718, 3580, 3460, 7233, 6298, 3913, 521, 4251, 3071, 1052, 2976, 7211, 5442, 4097, 6319, 2156, 1988, 2520, 2664, 4772, 2043, 2765, 6379, 6774, 7334, 7472, 7971, 7727, 5691, 9040, 5876, 8508, 812, 2291, 3056, 3181, 7947, 6561, 8133, 6817, 1259, 356, 6880, 8816, 3082, 6497, 8084, 8084, 1923, 9828, 4071]"
75996,9263,5001,4150,178,3724,5413,3653,7202,2212,9377,4,7046,5690,6019,5480,5799,2560,6933,6107,4025,2556,1623,4544,8429,3919,7431,5918,7697,8012,2110,1917,5868,2973,713,4371,7359,9772,4869,4839,966,4242,4545,4204,6351,5228,461,6719,1410,2524,9436,594,6030,1569,145,1069,2169,6677,8482,33,151,507,6331,10,26,0,"[5001, 29, 5802, 746, 6151, 1709, 2593, 3094, 3111, 4555, 9318, 4226, 884, 3071, 5980, 6798, 3955, 6681, 6772, 8448, 7438, 868, 4761, 9652, 1193, 7956, 208, 6449, 4006, 2445, 5136, 4645, 5188, 7971, 4668, 4583, 6027, 105, 2250, 7767, 9715, 2934, 6669, 4981, 9033, 292, 407, 734, 170, 5869, 7612, 4274, 9503, 1791, 8788, 2990, 5400, 7848, 1603, 7152, 9812]"
75997,9186,7002,4112,6441,4456,1022,3399,2107,5006,4075,6156,4094,6554,8357,8787,6980,4362,6514,3755,3840,2151,2267,9272,4761,1782,2734,2272,7342,4837,5114,7583,7719,9209,3186,4440,4385,6478,5694,4654,7088,2029,6407,277,2908,4482,1341,2938,6668,6654,7206,8805,8732,7915,7945,2610,1325,5992,7546,2383,3766,7805,5817,1,29,0,"[7002, 8492, 6943, 724, 1827, 1089, 8851, 1074, 6807, 1315, 8636, 6107, 4053, 4996, 3112, 5138, 233, 2165, 7196, 984, 324, 7021, 8071, 2696, 3790, 8366, 94, 434, 4664, 9345, 2090, 1086, 6956, 5202, 8073, 1724, 6050, 2138, 3690, 8344, 2181, 6360, 2321, 3640, 6565, 8892, 5717, 4441, 5669, 2376, 1657, 8354, 7946, 4909, 8719, 7511, 2701, 6006, 680, 7317, 4814]"
75998,9027,3003,5673,9406,8273,6784,4597,8780,334,6129,9126,5447,5577,2872,4315,6914,6889,9949,7761,5121,8461,9490,9388,8725,9240,6713,1333,1483,8869,5047,5375,9885,239,8219,5311,3071,7930,9908,5192,7288,3030,9944,7163,8200,7063,1342,2402,5842,9046,414,1653,2602,764,1046,1003,6492,4466,1077,4314,9231,1580,1730,4,37,0,"[3003, 4266, 3472, 7781, 8561, 4277, 953, 6969, 2177, 976, 365, 9340, 714, 9790, 2509, 9386, 5429, 3865, 2691, 5319, 5948, 9174, 4437, 5994, 1211, 2897, 7770, 9581, 212, 2893, 7883, 920, 967, 4045, 4018, 1986, 7552, 9663, 9696, 4887, 9012, 9052, 2173, 3420, 5170, 4170, 5365, 1161, 8338, 3989, 7103, 115, 9802, 4616, 9828, 2038, 3906, 8597, 1294, 8652, 7745]"
75999,9827,2009,4884,6942,5364,8442,7627,2967,5106,9438,368,5486,6438,7955,3462,4942,4527,1053,8911,8092,986,7921,7168,3392,153,6106,224,8317,3677,4155,9793,5519,1044,751,7921,881,688,550,6267,5721,910,4538,3271,6698,4492,2691,2535,7779,1502,7148,2474,542,7256,1346,5419,3971,8035,8279,5760,5428,1604,6667,10,29,0,"[2009, 3281, 4837, 3501, 4772, 3283, 599, 9013, 3194, 257, 6035, 2091, 8309, 2243, 8061, 4879, 7446, 9958, 8652, 5929, 7167, 9104, 6785, 1508, 2964, 2803, 7903, 7811, 8436, 6534, 4272, 2420, 4055, 1814, 4781, 5711, 9163, 6810, 9014, 5479, 76, 2296, 9188, 3457, 2721, 8789, 5108, 6401, 6743, 8321, 4113, 5626, 6988, 493, 3208, 7366, 415, 4608, 7629, 7514, 2031]"


In [249]:
cols_roll = [col for col in train_manual.columns if col.startswith('roll_') and col not in ['roll_0','roll_actual','roll_hash_list_nonce']]
print(cols_roll)

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']


In [250]:
def digit_9_stats(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual = (data['roll_actual']>HIGH_TARGET)
        
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'
        cutoff_range = range(0,7)
#         mask_actual =(data['roll_actual']<LOW_TARGET)
        
    mask_actual = create_target_mask(data,bHighCheck)

        
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [251]:
def add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,ratio_min,
                aft_cur,tr_total,tr_success,
               aft_all,aft_0,aft_1,
               analysis_df):
    if aft_cur >=ratio_min: 
        result=[nonce,zero_col_cutoff,pattern,bHighCheck,digit,cutoff,afterval,aft_cur,np.nan,
               aft_all,aft_0,aft_1,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,
               tr_total,tr_success,
               np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan,np.nan
               ]
        analysis_df.loc[len(analysis_df.index)]=result  
    return analysis_df

def save_train_stats(data,nonce,zero_col_cutoff,pattern,bHighCheck,ratio_min,
                    analysis_df=None):
    
    digits = [9,8,7,6,5,4,3,2,1]
    for digit in digits:
#         print(f'****** digit : {digit}')
        if analysis_df is None:
            columns = ['nonce','zero_col_cutoff','pattern_99','target_high','digit','cutoff','afterval','tr_ratio','te_ratio',
                      'aft_all','aft_1','aft_2',
                       'pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5','pr_tr_cutoff','pr_val_cutoff',
                      'tr_total','tr_success','te_total','te_success',
                      'te_pr_1_count','te_pr_2_count','te_pr_3_count',
                       'te_pr_4_count','te_pr_5_count',
                      'te_mean_count','te_med_count','te_tr_count','te_val_count']
            analysis_df = pd.DataFrame(columns=columns)

        after_vals = [8,10]
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            cutoff_range = range(0,6)

        mask_actual=create_target_mask(data,bHighCheck)
        for cutoff in cutoff_range:

#             print(f'********** cutoff:{cutoff}')
            mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff) 
            total_all = len(data[mask])
            if total_all ==0:
                success_all=0
                aft_all=0
            else:
                success_all = len(data[mask & mask_actual])
                aft_all =round(100 * success_all/ total_all ,0)
#             print(f'before values - probability: {aft_all} total : {total_all}  success: {success_all}')
            aft = np.zeros(len(after_vals))
            success = np.zeros(len(after_vals))
            total = np.zeros(len(after_vals))

            for i,val in enumerate(after_vals):
                mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+data['count_lo']<=18)
                total[i] = len(data[mask2])
                if total[i] ==0:
                    success[i]=0
                    aft[i]=0
                else:
                    success[i] = len(data[mask2 & mask_actual])
                    aft[i] =round(100 * success[i]/ total[i] ,0)
#                 print(f'AFTER - factor:{val} - probability: {aft[i]} total : {total[i]}  success: {success[i]}')

            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,25,ratio_min,
                                    aft_all,total_all,success_all,
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[0],ratio_min,
                                    aft[0],total[0],success[0],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
            analysis_df= add_one_aft(nonce,zero_col_cutoff,pattern,bHighCheck,
                                     digit,cutoff,after_vals[1],ratio_min,
                                    aft[1],total[1],success[1],
                                   aft_all,aft[0],aft[1],
                                   analysis_df)
    return analysis_df


In [252]:
# train_manual.head()

In [253]:
# results_df.iloc[1072]

In [254]:
train_manual['roll_actual_dig_1'] = (train_manual['roll_actual'] / 1000).astype('int')
print(train_manual['roll_actual_dig_1'].value_counts(normalize=True))

train_manual['count_hi'] = (train_manual[cols_roll] > 5250).sum(axis=1)
train_manual['count_lo'] = (train_manual[cols_roll] < 4750).sum(axis=1)

train_manual['count_dig_1'] = ( (train_manual[cols_roll] / 1000).astype('int')==1).sum(axis=1)
train_manual['count_dig_2'] = ( (train_manual[cols_roll] / 1000).astype('int')==2).sum(axis=1)
train_manual['count_dig_3'] = ( (train_manual[cols_roll] / 1000).astype('int')==3).sum(axis=1)
train_manual['count_dig_4'] = ( (train_manual[cols_roll] / 1000).astype('int')==4).sum(axis=1)
train_manual['count_dig_5'] = ( (train_manual[cols_roll] / 1000).astype('int')==5).sum(axis=1)
train_manual['count_dig_6'] = ( (train_manual[cols_roll] / 1000).astype('int')==6).sum(axis=1)
train_manual['count_dig_9'] = ( (train_manual[cols_roll] / 1000).astype('int')==9).sum(axis=1)
train_manual['count_dig_8'] = ( (train_manual[cols_roll] / 1000).astype('int')==8).sum(axis=1)
train_manual['count_dig_7'] = ( (train_manual[cols_roll] / 1000).astype('int')==7).sum(axis=1)
train_manual['count_dig_0'] = ( (train_manual[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
cols = ['roll_actual','roll_actual_dig_1',
        'count_dig_9','count_dig_0',
        'count_lt_1000','count_gt_5000','roll_0'] + cols_roll

1    0.103462
2    0.103462
6    0.102192
9    0.100769
7    0.100077
8    0.099154
0    0.098231
5    0.097769
3    0.097500
4    0.097385
Name: roll_actual_dig_1, dtype: float64


In [255]:
# for cur_digit in [9,8,7,5]:
#     print()
#     print(f'************************** cur digit {cur_digit} ************************************************')
#     print('************************** HIGH ************************************************')
#     digit_9_stats(train_manual,True)
#     print()
#     print('************************** LOW ************************************************')
#     digit_9_stats(train_manual,False)

In [256]:
# %%time
# temp_df=None
# for zero_cutoff in range(0,6):
#     temp_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=temp_df)
# temp_df

Prediction

In [257]:
# # hashval = '393cc340566014a83e78d302f079c98f5cd4e4bc4c34321231cf479559556038'#'3e24f29a3ae70e54aedd86b0c68640050be7dace8ae679f1fb85d1325a56ddae'
# seed = 'dummy' # hashval #'c77b3e783f094e255b0693f89094bed95ad9738d86f44fa4c09c8b0d58e2c73e'
# # nonce = 2548 #2390

In [258]:
# test_data = pd.DataFrame(columns=['seed','hash','nonce'])
# test_data.loc[0,'seed']=seed
# test_data.loc[0,'hash']=hashval
# test_data.loc[0,'nonce'] = nonce

# print(len(test_data))
# test_data.head()

In [259]:
# if PATTERN_99:
#     match_digit_arr=np.array([9,9])
#     mismatch_digit_arr=np.array([9,9])
# else:
#     match_digit_arr=np.array([0,0])
#     mismatch_digit_arr=np.array([0,0])
    
# match_digit_indices = np.array([1,2])
# match_digit_count_arr=np.array([1])
# mismatch_digit_indices = np.array([0,3])


In [260]:
# %%time
# # match_digit_arr = np.array(['999',])
# # match_digit_count_arr=np.array([1])
# # match_digit_indices = np.array(['012'])

# # match_digit_arr=np.array([9,9])
# # match_digit_indices = np.array([1,2])
# # match_digit_count_arr=np.array([1])

# if not READ_FROM_FILE_TEST:

#     cur_client_seed_array = client_seed_array_2[:1000001]

#     results_df_test = None

#     last_index_test =  len(cur_client_seed_array)-1 #1024
#     start = 0

#     results_df_test = generate_data(test_data,
#                                 cur_client_seed_array,
#                                  ABOVE,
#                         match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
#                         mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
#                         match_count_exp=10000, #train_client_size,
#                         feature_chain_length=20,
#                         is_data_hash = False,
#                         test_limit=1,
#                         last_index=last_index_test,result_df=results_df_test,
#                         print_client_scan=False)
#     row = results_df_test.iloc[len(results_df_test)-1]
#     last_index_test = row['client_index']

In [261]:
# print(results_df_test['client_index'].min())
# len(results_df_test)

In [262]:
def generate_test_features(initial_df):
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]

    print(cols_roll)
    rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

    rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
    rolls_df['roll_actual'] = initial_df['roll_actual']
    rolls_df['client_seed'] = initial_df['client_seed']
    rolls_df['client_index'] = initial_df['client_index']

    return rolls_df


In [263]:
def gen_test_data(cur_nonce,cur_hash,cur_seed,file_suffix=""):
    test_data = pd.DataFrame(columns=['seed','hash','nonce'])
    test_data.loc[0,'seed']=cur_seed
    test_data.loc[0,'hash']=cur_hash
    test_data.loc[0,'nonce'] = cur_nonce

    print(len(test_data))
    print(test_data.head())
    
    if PATTERN_99:
        match_digit_arr=np.array([9,9])
        mismatch_digit_arr=np.array([9,9])
    else:
        match_digit_arr=np.array([0,0])
        mismatch_digit_arr=np.array([0,0])

    match_digit_indices = np.array([1,2])
    match_digit_count_arr=np.array([1])
    mismatch_digit_indices = np.array([0,3]) 
    
    if PATTERN_99:
        file_pattern_str = 'x99x'
    else:
        file_pattern_str = 'x00x'
        
    filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern{file_suffix}.csv'
    
    if  READ_FROM_FILE_TEST:
        results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
                                                        'roll_hash_list_nonce': pd.eval})
    else:

        cur_client_seed_array = client_seed_array_2[:1000001]
        results_df_test = None

        last_index_test =  len(cur_client_seed_array)-1 #1024
        start = 0

        results_df_test = generate_data(test_data,
                                    cur_client_seed_array,
                                     ABOVE,
                            match_digit_arr,match_digit_count_arr,match_digit_indices=match_digit_indices,
                            mismatch_digit_arr=mismatch_digit_arr,mismatch_digit_indices=mismatch_digit_indices,
                            match_count_exp=10000, #train_client_size,
                            feature_chain_length=feature_chain_length,
                            is_data_hash = False,
                            test_limit=1,
                            last_index=last_index_test,result_df=results_df_test,
                            print_client_scan=False)
        #Remove client seed duplicates from test data
        results_df_test= results_df_test.drop_duplicates(subset='client_seed')
        
        results_df_test.to_csv(filename,index=False)
        
    print('Test Data Generation Completed')    
    test_manual = generate_test_features(results_df_test)
    print('Test Features Completed')    
        
#         row = results_df_test.iloc[len(results_df_test)-1]
#         last_index_test = row['client_index']   

    return results_df_test,test_manual
 

In [264]:
%%time
READ_FROM_FILE_TEST=False
# seed='dummy'
seed = generate_hash_chain(hashval,26)[25]
print('seed:',seed)
results_df_test,test_manual=gen_test_data(nonce,hashval,seed,file_suffix="")
print()
print(results_df_test[['seed','hash','nonce']].head(1))
test_manual.head()

seed: d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be
1
                                                               seed  \
0  d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be   

                                                               hash nonce  
0  a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd  2677  
server_count:0

999999 999999
No further match for seed:d98d0a6ee9de1d1e39bc3f6c6e65d20f4cc269ddcb3e3702e4b1b1e30140a0be nonce:2677 match count: 8084

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_4

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,1002,3340,3115,3417,3284,8049,3616,6398,9457,7569,9702,1441,882,3991,1726,5726,3406,1272,7986,1751,9586,1763,8885,3035,4245,1321,9036,6808,4942,6172,9451,5599,294,8802,4977,2656,8026,1434,1070,1743,943,598,5457,2523,6401,9335,1974,7129,4666,2892,8604,4175,3990,7616,2771,6974,3967,9599,3402,4845,3480,24,33,10,4,12,6,3,5,7,5,4,4,1321,woxpwoxpwoxpwoxpwoxpa08956326a9bc6ba503848d744c6081b538343260af150d6899b370200d1ad69,999756
1,7006,5797,7669,9207,7522,5045,4073,2549,5703,9807,2120,5004,7202,2766,6100,5710,4041,2909,7721,7468,7718,281,8598,346,1993,6653,4343,5526,8411,5985,8712,6595,3890,9515,6175,2029,2503,2910,6799,9852,9871,5670,5468,814,7188,3240,322,6238,2455,636,8655,3450,7584,3345,9124,3079,751,9988,7272,433,3739,33,25,1,8,6,3,9,6,7,4,9,7,6653,woxpwoxpwoxpwoxpwoxpd07d7fe627c0fd49997f83df4ab8cbe5b88180688023c82fa53bb4dcfdfd39ff,999721
2,7002,3312,1313,2209,1278,8948,804,208,3159,7008,5558,9902,5444,3070,8815,3335,7235,9770,4620,9161,8519,1829,201,3394,6915,5087,4724,8151,8396,9677,4002,395,7751,3352,6369,9931,6495,5103,361,2057,3975,8393,8376,304,4280,5596,2180,9819,913,2744,8775,9785,928,4782,7527,9216,225,7860,637,829,9427,28,29,3,4,7,5,5,3,9,8,5,11,5087,woxpwoxpwoxpwoxpwoxp852304f664e13ed004e22514907d4734d4bc58386b4a5fe3e9edcf453f3c9c36,999659
3,9004,4601,6085,6183,4521,2385,1407,6309,9633,6390,491,8674,3440,1787,9485,2993,7630,4039,413,6203,8537,700,4844,4404,4525,7800,8543,4624,4767,9585,665,2095,3820,3704,6877,1373,7296,208,9012,7456,8136,460,5234,4262,2190,882,2738,2871,719,8393,4411,4213,9808,7847,2497,2533,8384,9679,9206,6961,8477,26,31,3,8,3,11,1,7,7,7,5,8,7800,woxpwoxpwoxpwoxpwoxpc16cedb818b15937cbc49da805e9ebd5ba2cca6de1b8e150f676939cb723f765,999647
4,2001,9973,6125,2423,4445,7589,7942,3689,7119,8078,9469,3393,6210,8910,4160,796,4293,8483,8130,415,4623,3025,3307,51,4258,2195,7311,5621,4575,4044,2451,1188,7259,7291,5928,6014,2856,3797,6078,34,9215,131,9938,3445,7421,7549,2989,1864,6481,9042,7271,8603,6564,9488,862,5980,1631,3314,6893,4420,5662,31,29,3,5,7,8,4,7,6,5,9,6,2195,woxpwoxpwoxpwoxpwoxp49c858164782bf5378b4d899606c33c675f660f57d6a1d2c3694b3c61061363b,999504


In [265]:
%%time
READ_FROM_FILE_TEST=False
#Generate hash test data
print(f'{hashval=}')
hash_l2=generate_hash(hashval.encode())
print(f'{hash_l2=}')
results_df_test_hash,test_manual_hash=gen_test_data(nonce,hash_l2,hashval,file_suffix="_hash")
print()
print(results_df_test_hash.head(1))
test_manual_hash.head()

hashval='a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd'
hash_l2='b465a3bc3240974c62115114d220a77452036cf55edb5bc079668fa89ff9d194'
1
                                                               seed  \
0  a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd   

                                                               hash nonce  
0  b465a3bc3240974c62115114d220a77452036cf55edb5bc079668fa89ff9d194  2677  
server_count:0

999999 999999
No further match for seed:a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd nonce:2677 match count: 8154

Test Data Generation Completed
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll

,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_hi,count_lo,count_dig_1,count_dig_2,count_dig_3,count_dig_4,count_dig_5,count_dig_6,count_dig_9,count_dig_8,count_dig_7,count_dig_0,roll_actual,client_seed,client_index
0,3001,8082,9065,863,4986,7821,3767,2476,9992,7797,9648,741,2738,1143,1270,5418,3120,9115,2148,5262,6047,409,9132,8760,6427,39,4202,9902,4629,3559,8403,9262,1711,689,3587,2961,8996,2007,8075,6595,9656,2418,695,2468,4344,3614,1150,5643,8017,3670,6355,3556,2866,781,6555,3707,7324,7801,9806,7696,305,28,31,4,8,8,4,3,5,9,6,5,8,9531,woxpwoxpwoxpwoxpwoxpe806b294fbe0673080f15068b1a30e4f02b5445e917fcd256dde2c7ed1599831,999953
1,2009,8258,3430,6968,3041,1215,6881,8496,3367,3758,2010,7481,1581,4342,4076,1937,8930,5648,7930,161,1390,6484,2364,1320,2017,8274,7647,3796,7704,2021,7466,888,2205,9226,7910,7600,2386,2844,6680,2093,8086,7162,7746,5333,5101,3233,905,6374,4752,2057,1983,9484,1110,5613,6648,2624,9650,152,1783,4689,8040,27,31,8,10,6,4,4,6,3,6,9,4,4521,woxpwoxpwoxpwoxpwoxp6f1ff2a5c9b28e97b31a54744ad17599eb854e20f0f56e620657bea702e3dc4c,999907
2,6003,582,2186,5692,2987,5180,1084,6706,3257,2554,8393,4008,4807,3018,320,8211,2053,2783,9420,5869,7412,2017,1145,9749,3408,3433,5486,7678,2279,7061,2777,7366,5197,9211,171,3075,4055,3087,5700,6018,6840,2503,9615,3508,3111,8399,546,5183,7933,9932,5096,8948,4858,4833,293,2234,8554,3728,6640,8309,5973,25,28,2,10,9,5,9,4,5,6,5,5,1316,woxpwoxpwoxpwoxpwoxp8f4a77d7ec687fd91cecb42f5b9436253b5687d865afc509ba3a64bbf6b287fb,999850
3,6001,5112,3167,9924,1095,5848,5751,6794,810,1077,3892,4917,8987,3527,620,2883,646,1813,3380,421,3219,1625,7009,3625,5413,5324,9514,7001,6524,6234,5361,3125,935,1943,8166,26,8290,7677,7457,8825,9527,1507,5733,1626,198,1845,1784,1902,4695,4377,5831,2530,3039,7341,1019,2435,3308,4297,7404,3349,215,23,35,11,3,10,4,8,3,3,4,6,8,4320,woxpwoxpwoxpwoxpwoxpbc6f3726faff712c5351944e3bb305f2c5b5b5e8c6a8a7d7e920b8bc083f231b,999746
4,4009,9088,5274,955,6747,8886,527,5483,3013,554,5312,5736,1039,7208,9780,8718,3770,2660,5074,4272,6074,2248,3889,6923,9859,5754,677,4066,7112,7789,3184,9665,5935,468,1412,555,1102,2773,4520,636,7307,5022,2133,2206,7347,9631,5516,9092,8251,9238,2778,6755,7238,565,7097,6577,6547,9013,8071,6521,2623,33,25,3,7,4,3,9,7,8,4,7,8,2226,woxpwoxpwoxpwoxpwoxpf8fbebeeceec0fbbbd87ca501af903dae8b9154eb9c44c8676b6c3ba44b63a13,999719


In [266]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[25]
# cur_hash = cur_hash_list[26]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash25,test_manual_hash25=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash25")
# print()
# print(results_df_test_hash25.head(1))
# test_manual_hash25.head()

In [267]:
# %%time
# READ_FROM_FILE_TEST=False
# cur_seed = cur_hash_list[24]
# cur_hash = cur_hash_list[25]
# print(f'{cur_seed=}')
# print(f'{cur_hash=}')

# results_df_test_hash24,test_manual_hash24=gen_test_data(nonce,cur_hash,cur_seed,file_suffix="_hash24")
# print()
# print(results_df_test_hash24.head(1))
# test_manual_hash24.head()

In [268]:
%%time
results_df_test_cleaned=save_cleaned_results_df(results_df_test,True)
results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash25,True)
# results_df_test_cleaned=save_cleaned_results_df(results_df_test_hash24,True)

CPU times: user 5.06 s, sys: 177 ms, total: 5.24 s
Wall time: 5.07 s


In [269]:
# %%time
# if PATTERN_99:
#     file_pattern_str = 'x99x'
# else:
#     file_pattern_str = 'x00x'
# filename = f'data/results_df_test_{nonce}_{file_pattern_str}_pattern.csv'

# if READ_FROM_FILE_TEST:
#     results_df_test = pd.read_csv(filename, converters={'roll_hash_list': pd.eval,
#                                                         'roll_hash_list_nonce': pd.eval})
# else:
#     results_df_test.to_csv(filename,index=False)

In [270]:
# s=70
# e=80
# l=90
# client_seed_min_1=results_df[s*train_client_size:e*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[s*train_client_size:e*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)
# results_1 = results_df[s*train_client_size:e*train_client_size]
# print(results_1['client_index'].describe(percentiles=[0.09,0.1,0.15,0.2,0.25]))

# client_seed_min=results_df[e*train_client_size:l*train_client_size]['client_index'].min()
# client_seed_max=results_df[e*train_client_size:l*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# results_2 = results_df[e*train_client_size:l*train_client_size]
# print(results_2['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [271]:
# client_seed_min_1=results_df[25*train_client_size:39*train_client_size]['client_index'].min()
# client_seed_max_1=results_df[25*train_client_size:39*train_client_size]['client_index'].max()
# print(client_seed_min_1,client_seed_max_1)

# client_seed_min=results_df[39*train_client_size:50*train_client_size]['client_index'].min()
# client_seed_max=results_df[39*train_client_size:50*train_client_size]['client_index'].max()
# print(client_seed_min,client_seed_max)
# print(results_df[39*train_client_size:50*train_client_size]['client_index'].describe(percentiles=[0.05,0.06,0.07,0.08,0.09,0.25]))
# results_2 = results_df[39*train_client_size:50*train_client_size]
# client_seed_min_2=results_2[results_2['client_index']>client_seed_max_1]['client_index'].min()
# client_seed_max_2=results_2['client_index'].max()
# print(client_seed_min_2,client_seed_max_2)


In [272]:
# mask = (results_df_test['client_index']<=client_seed_max_1) | (results_df_test['client_index']>=client_seed_min_2)
# results_df_test = results_df_test[mask]
# print(len(results_df_test))
# print(results_df_test['client_index'].describe(percentiles=[0.25,0.5,0.52,0.55,0.57,0.6,0.7,0.8,0.9]))

In [273]:
# test_manual = generate_test_features(results_df_test)

In [274]:
# # rand=random.randint(0,len(rolls_df)-1)
# rand=np.random.choice(range(0,len(rolls_df)))
# print('random location:',rand)
# rolls_df.iloc[rand]['client_seed']

In [275]:
# res = []
# for i in range(100):
#     res.append(np.random.choice(range(0,len(rolls_df))) )
    
# pd.Series(np.array(res)).value_counts()

In [276]:

def check_special_pattern_match(data,bHighCheck,cutoff,after_val):
     
#     rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
#     cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    
    if bHighCheck:
        cutoff_col = f'count_dig_{cur_digit}'
        zero_col = 'count_dig_0'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_dig_0'
        zero_col = f'count_dig_{cur_digit}'
        after_col = 'count_hi'

#     rolls_df['count_hi'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df['count_lo'] = (rolls_df[cols_roll] < 4750).sum(axis=1)

#     rolls_df['count_dig_1'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
#     rolls_df['count_dig_2'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df['count_dig_3'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df['count_dig_4'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df['count_dig_5'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df['count_dig_6'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df['count_dig_9'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df['count_dig_8'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df['count_dig_7'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df['count_dig_0'] = ( (rolls_df[cols_roll] / 1000).astype('int')==cur_zero_digit).sum(axis=1)
    
    
#     rolls_df['roll_actual'] = initial_df['roll_actual']
#     rolls_df['client_seed'] = initial_df['client_seed']
#     rolls_df['client_index'] = initial_df['client_index']

    
    mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
    return data[mask]

def compareactual(rolls_df,bHighCheck):
    
    mask_actual=create_target_mask(rolls_df,bHighCheck)
    rolls_filt = rolls_df[mask_actual]
    return len(rolls_filt)


def save_test_stats(analysis_df, data,pattern,bHighCheck,is_prod=False):
    
    mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
    analysis_df_filt = analysis_df[mask]
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff
        
        if bHighCheck:
            cutoff_col = f'count_dig_{digit}'
            zero_col = 'count_dig_0'
            after_col = 'count_hi'
        else:
            cutoff_col = 'count_dig_0'
            zero_col = f'count_dig_{digit}'
            after_col = 'count_hi'
            
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==zero_col_cutoff)   \
                            & (data[after_col]<=after_val)  \
                            #& (data['count_hi']+data['count_lo']<=18)
    
        data_filt = data[mask]
        total = len(data_filt)
        analysis_df.at[row.Index,'te_total']  = total  
        
        if not is_prod:
            success  = compareactual(data_filt,bHighCheck)
            if total ==0:
                prob=0
            else:
                prob = round(100 * success/total,0)

            analysis_df.at[row.Index,'te_ratio']  = prob  
            analysis_df.at[row.Index,'te_success']  = success  
        
    return analysis_df

        

In [277]:

# bHighCheck = False
# for cur_digit in [9,8,7,5]:
#     for bHighCheck in [True,False]:
#         if bHighCheck:
#             high_low_text = 'HIGH'
#         else:
#             high_low_text = 'LOW'
#         print()
#         print(f'****** cur_digit {cur_digit} ******  {high_low_text} ')

#         for cutoff in range(0,7):
#             rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,25)
#             len_match  = compareactual(rolls_df,bHighCheck)
#             if len(rolls_df) ==0:
#                 success_count=0
#             else:
#                 success_count = round(100 * len_match/len(rolls_df),0)
#             print()
#             print(f'cutoff:{cutoff} General --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')
#             for afterval in [8,10]:
#                 rolls_df = check_special_pattern_match(test_manual,bHighCheck,cutoff,afterval)
#                 len_match  = compareactual(rolls_df,bHighCheck)
#                 if len(rolls_df)==0:
#                     success_count=0
#                 else:
#                     success_count = round(100 * len_match/len(rolls_df),0)
#                 print(f'cutoff:{cutoff} afterval:{afterval} --- total:{len(rolls_df)} matched:{len_match} success:{success_count}')



In [278]:
# analysis_df_test = save_test_stats(temp_df, test_manual)
# analysis_df_test

In [279]:
# mask = (analysis_df_test['digit'].isin([1,2,3,4,6])) & (analysis_df_test['tr_ratio']>=13)
# mask2 = mask & (analysis_df_test['te_ratio']>=13)
# print(len(analysis_df_test[mask]))
# print(len(analysis_df_test[mask2]))
# analysis_df_test[mask2]

##### Feature Generation

In [280]:
def generate_roll_features_k(rolls_df,roll_start,roll_end,suffix="",stat_suffix=""):
    first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    rolls_df[f'count_gt_9000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    rolls_df[f'count_lt_1000_k{stat_suffix}'] = (rolls_df[first_k_roll_cols] < 1000).sum(axis=1)
    
    return rolls_df
    
def generate_roll_features(rolls_df,
                           roll_start=1, roll_end=20,k=4,
                          suffix="",stat_suffix=""):
    cols_roll = [f'roll_{i}{suffix}' for i in range(roll_start,roll_end+1)]
    print(cols_roll)
    
    if (roll_start==1) & (roll_end==20):
        stat_suffix = suffix
    else:
        stat_suffix = f'_{roll_start}_{roll_end}{suffix}'
    
    rolls_df[f'count_lt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
    rolls_df[f'count_gt_9000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
    rolls_df[f'count_gt_8000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
    rolls_df[f'count_gt_7000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
    rolls_df[f'count_gt_6000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
    rolls_df[f'count_gt_5000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
    rolls_df[f'count_gt_4000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
    rolls_df[f'count_gt_3000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
    rolls_df[f'count_gt_2000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
    rolls_df[f'count_gt_1000{stat_suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
    rolls_df[f'count_hi{stat_suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
    rolls_df[f'count_lo{stat_suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
    rolls_df[f'count_lo_hi{stat_suffix}'] = rolls_df[f'count_hi{stat_suffix}'] \
                                            + rolls_df[f'count_lo{stat_suffix}']   


    rolls_df=generate_roll_features_k(rolls_df,roll_start,roll_start+k,
                                      suffix=suffix,stat_suffix=stat_suffix)
    
    roll_list_strs = np.full(len(rolls_df),"")
    for col in cols_roll:
        roll_list_strs = rolls_df[col].astype('str') + roll_list_strs
    
    rolls_df[f'total_dig_9{stat_suffix}']= roll_list_strs.str.count('9')
    rolls_df[f'total_dig_0{stat_suffix}']= roll_list_strs.str.count('0')
    
    rolls_df[f'roll_mean{stat_suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
    rolls_df[f'roll_std{stat_suffix}'] = (rolls_df[cols_roll]).std(axis=1)
     
    print(f'Roll Features from {roll_start} to {roll_end} {suffix} completed' )
    
    return rolls_df

def generate_features_full(initial_df,istrain,feature_chain_length):
    
    rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]    
    rolls_df_nonce = initial_df['roll_hash_list_nonce'].apply(pd.Series)
    nonce_suffix = '_nonce'
    rolls_df_nonce.columns = [f'roll_{i}{nonce_suffix}' for i in range(rolls_df_nonce.shape[1])] 
    
    #generate top 20 hash features
    rolls_df=generate_roll_features(rolls_df,roll_start=1,
                                    roll_end=20, k=4)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix,roll_start=1,
                                    roll_end=20 , k=4)
    
    #generate 25's hash features
    roll_start =25
    roll_end = 50
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)  

    #generate 50's hash features
    roll_start =50
    roll_end = 60
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)      
    #generate for full hash features
    roll_start =1
    roll_end = feature_chain_length
    rolls_df=generate_roll_features(rolls_df,
                                    roll_start=roll_start,
                                    roll_end=roll_end, k=5)
    rolls_df_nonce=generate_roll_features(rolls_df_nonce,
                                    suffix=nonce_suffix ,roll_start=roll_start,
                                    roll_end=roll_end, k=5)        
    if istrain:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                 rolls_df,
                                 rolls_df_nonce],axis=1)
        mask = create_target_mask(features_df)
            
        features_df['target'] =0
        features_df.loc[mask,'target'] = 1
        features_df['roll_actual']=features_df['roll_actual'].astype('int')
        print(features_df['target'].value_counts())
    else:
        features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
                                             rolls_df,rolls_df_nonce],axis=1)
    return features_df


In [281]:
# def generate_roll_features(initial_df,colname,feature_chain_length,
#                            k=4,
#                           suffix=""):
#     rolls_df = initial_df[colname].apply(pd.Series)
#     rolls_df.columns = [f'roll_{i}{suffix}' for i in range(rolls_df.shape[1])]
    
#     # roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([str(i) for i in x]))
    
#     cols_roll = [f'roll_{i}{suffix}' for i in range(1,feature_chain_length+1)]
#     print(cols_roll)
    
# #     rolls_df[f'count_lt_1000{suffix}'] = (rolls_df < 1000).sum(axis=1)
# #     rolls_df[f'count_gt_9000{suffix}'] = (rolls_df >= 9000).sum(axis=1)
# #     rolls_df[f'count_gt_5000{suffix}'] = (rolls_df >= 5000).sum(axis=1)
# #     rolls_df[f'count_gt_7000{suffix}'] = (rolls_df >= 7000).sum(axis=1)
# #     rolls_df[f'count_gt_8000{suffix}'] = (rolls_df >= 8000).sum(axis=1)
    
#     rolls_df[f'count_lt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==0).sum(axis=1)
#     rolls_df[f'count_gt_9000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==9).sum(axis=1)
#     rolls_df[f'count_gt_8000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==8).sum(axis=1)
#     rolls_df[f'count_gt_7000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==7).sum(axis=1)
#     rolls_df[f'count_gt_6000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==6).sum(axis=1)
#     rolls_df[f'count_gt_5000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==5).sum(axis=1)
#     rolls_df[f'count_gt_4000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==4).sum(axis=1)
#     rolls_df[f'count_gt_3000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==3).sum(axis=1)
#     rolls_df[f'count_gt_2000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==2).sum(axis=1)
#     rolls_df[f'count_gt_1000{suffix}'] = ( (rolls_df[cols_roll] / 1000).astype('int')==1).sum(axis=1)
    
#     rolls_df[f'count_hi{suffix}'] = (rolls_df[cols_roll] > 5250).sum(axis=1)
#     rolls_df[f'count_lo{suffix}'] = (rolls_df[cols_roll] < 4750).sum(axis=1)    
#     rolls_df[f'count_lo_hi{suffix}'] = rolls_df[f'count_hi{suffix}'] \
#                                             + rolls_df[f'count_lo{suffix}']   

#     first_k_roll_cols = [f'roll_{i}{suffix}' for i in range(1,k+1)]
#     rolls_df[f'count_gt_9000_k{suffix}'] = (rolls_df[first_k_roll_cols] >= 9000).sum(axis=1)
    
#     roll_list_strs = initial_df[colname].apply(lambda x: ', '.join([f'{i:04d}' for i in x]))
    
#     rolls_df[f'total_dig_9{suffix}']= roll_list_strs.str.count('9')
#     rolls_df[f'total_dig_0{suffix}']= roll_list_strs.str.count('0')
    
#     rolls_df[f'roll_mean{suffix}'] = (rolls_df[cols_roll]).mean(axis=1)
#     rolls_df[f'roll_std{suffix}'] = (rolls_df[cols_roll]).std(axis=1)
   

# #     if (suffix==""):
# #         rolls_df[f'roll_0_1st_digit'] = (rolls_df['roll_0'] / 1000).astype('int')
# #         rolls_df[f'roll_0_last_digit'] = (rolls_df['roll_0'] % 10)
    
    
#     return rolls_df

# def generate_features_full(initial_df,istrain,feature_chain_length):
    
#     rolls_df=generate_roll_features(initial_df,'roll_hash_list',feature_chain_length,k=4)
#     rolls_df_nonce=generate_roll_features(initial_df,'roll_hash_list_nonce',
#                                     feature_chain_length,k=4,suffix='_nonce' )
    
#     if istrain:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                  rolls_df,
#                                  rolls_df_nonce],axis=1)
#         mask = create_target_mask(features_df)
            
#         features_df['target'] =0
#         features_df.loc[mask,'target'] = 1
#         features_df['roll_actual']=features_df['roll_actual'].astype('int')
#         print(features_df['target'].value_counts())
#     else:
#         features_df = pd.concat([initial_df[['roll_actual','seed','hash','client_seed']],
#                                              rolls_df,rolls_df_nonce],axis=1)
#     return features_df


In [282]:
# HIGH_ANALYSIS=True

In [283]:
%%time
# train = generate_features_full(results_df[train_client_size:],True,feature_chain_length)
train = generate_features_full(results_df,True,feature_chain_length)
print(len(train))
train.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [284]:
%%time
test = generate_features_full(results_df_test,False,feature_chain_length)
print(len(test))
test.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

In [285]:
%%time
test_hash = generate_features_full(results_df_test_hash,False,feature_chain_length)
print(len(test))
test_hash.head()

['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 'roll_3_nonce', 'roll_4_nonce', 'roll_5_nonce', 'roll_6_nonce', 'roll_7_nonce', 'roll_8_nonce', 'roll_9_nonce', 'roll_10_nonce', 'roll_11_nonce', 'roll_12_nonce', 'roll_13_nonce', 'roll_14_nonce', 'roll_15_nonce', 'roll_16_nonce', 'roll_17_nonce', 'roll_18_nonce', 'roll_19_nonce', 'roll_20_nonce']
Roll Features from 1 to 20 _nonce completed
['roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50']
Roll Features from 25 to 50  completed
['roll_25_nonce', 'roll_26_nonce', 'roll_27_nonc

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

#### Machine Learning

In [286]:
USE_STAT_FEATS = True

In [287]:

if USE_STAT_FEATS:
    exclude_stat_cols=[]
else:
#     exclude_stat_cols = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_col_starts = ['roll_mean','roll_std','roll_mean_nonce','roll_std_nonce']
    exclude_stat_cols=[]
    for stat_col in exclude_stat_col_starts:
        exclude_stat_cols += [col for col in train.columns if col.startswith(stat_col)]
    print(exclude_stat_cols)
exclude_cols = []#[f'roll_{i}' for i in range(11,feature_chain_length)]
exclude_cols_nonce =[]# [f'roll_{i}_nonce' for i in range(11,feature_chain_length+1)]
exclude_count_cols = [f'count_gt_{i}000' for i in [1,2,3,4,6]]
exclude_count_cols_nonce = [f'count_gt_{i}000_nonce' for i in [1,2,3,4,6]]
othercols = ['roll_actual','client_seed','seed','hash'] + exclude_stat_cols

all_exclude_cols = othercols+exclude_cols+exclude_cols_nonce+\
                    exclude_count_cols+exclude_count_cols_nonce
                  
features = [col for col in train.columns if col not in all_exclude_cols]
print(len(features))
print(features)

265
['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60', 'count_lt_1000', 'count_gt_9000', 'count_gt_8000', 'count_gt_7000', 'count_gt_5000', 'count_hi', 'count_lo', 'count_lo_hi', 'count_gt_9000_k', 'count_lt_1000_k', 'total_dig_9', 'total_dig_0', 'roll_mean', 'roll_std', 'count_lt_1000_25_50', 'count_gt_9000_25_50', 'count_gt_8000_25_50', 'count_gt_7000_25_50', 'count_gt_6000_25_50', 'c

In [288]:
X = train[features].drop('target',axis=1)
y = train['target']

In [289]:
X.columns

Index(['roll_0', 'roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6',
       'roll_7', 'roll_8', 'roll_9',
       ...
       'count_gt_1000_1_60_nonce', 'count_hi_1_60_nonce',
       'count_lo_1_60_nonce', 'count_lo_hi_1_60_nonce',
       'count_gt_9000_k_1_60_nonce', 'count_lt_1000_k_1_60_nonce',
       'total_dig_9_1_60_nonce', 'total_dig_0_1_60_nonce',
       'roll_mean_1_60_nonce', 'roll_std_1_60_nonce'],
      dtype='object', length=264)

In [290]:
# results_df#[['nonce','roll_actual','roll_hash_list','client_seed','client_index']]

In [291]:
import xgboost as xgb
import matplotlib.pyplot as plt # for plotting graphs
import seaborn as sns # for plotting graphs
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, f1_score,precision_recall_curve

Train Validation Split

In [292]:
n_folds=10
tr_indices_folds = []
val_indices_folds=[]
tr_indices_seeds=[]
for fold in range(n_folds):
#     tr_indices = train[(fold+2) * train_client_size:].index
#     val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_offset = 1#25
    tr_length = 25#25
    tr_start = (fold+tr_offset) * train_client_size
    tr_end = (fold+tr_offset+tr_length) * train_client_size
    tr_indices = train[tr_start:tr_end].index #fold + 3
#     tr_indices = list(tr_indices) + list(train[70:90].index) #fold + 3
    
#     tr_indices_seed=[]
#     for i in range(1,tr_offset):
#         tr_start_seed = (fold+i) * train_client_size
#         tr_end_seed = (fold+i+1) * train_client_size
#         tr_indices_seed.append(train[tr_start_seed:tr_end_seed].index) #fold + 3 
        
    # tr_start_seed = (fold+1) * train_client_size
    # tr_end_seed = (fold+tr_offset) * train_client_size
    # tr_indices_seed = train[tr_start_seed:tr_end_seed].index #fold + 3
    
    tr_indices_seed = [train[0:(fold+1) * train_client_size].index]
#     val_indices = train[(fold+1) * train_client_size:(fold+2) * train_client_size].index
    val_indices = train[(fold) * train_client_size:(fold+1) * train_client_size].index
    tr_indices_seeds.append(tr_indices_seed)
    tr_indices_folds.append(tr_indices)
    val_indices_folds.append(val_indices)
    

In [293]:
#This function is useful to convert the predicted probabilities into labels 
#so that F1 score is optimized
#It first determines the probability threshold using precision recall curve at which F1 score is optimized and 
#then generate the optimized labels based on the determined probability threshold
def get_opt_cutoff_prec(labels,preds):
    precision, recall, thresholds  = precision_recall_curve(labels, preds)
    f1_score= 2*((precision*recall)/(precision+recall))
    
    f1_score = f1_score[:len(thresholds)]
    f1_score[np.isnan(f1_score)]=0
#     optimal_idx = np.nanargmax(precision[:len(precision)-1])
#     print(f'precision borders:{precision[0],precision[len(precision)-2]}')
#     print(f'recall borders:{recall[0],recall[len(recall)-2]}')
#     print(f'f1_score size:{len(f1_score)} recall size:{len(recall)}')
#     print(f'{precision=}')
#     print(f'{recall=}')
#     print(f'{f1_score=}')
#     print(f'{thresholds=}')
#     print(f'optimal_idx:{optimal_idx} precision size:{len(precision)} thresholds size:{len(thresholds)}')
    optimal_idx = np.nanargmax(f1_score)
    optimal_threshold = thresholds[optimal_idx]
#     return optimal_threshold, precision[optimal_idx]
    return optimal_threshold, f1_score[optimal_idx]

def convert_probtolabels(preds,cutoff=0.5):
    y_bin= preds.copy()
    y_bin[preds>cutoff] = 1
    y_bin[preds<=cutoff] = 0
    y_bin=y_bin.astype(int)

    return y_bin

In [294]:
ZERO_COL_CUTOFF=4

In [295]:
def train_status(data,bHighCheck):
    
    after_vals = [8,10]
    if bHighCheck:
        cutoff_col = 'count_gt_9000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
        cutoff_range = range(2+2,7+2)
        
    else:
        cutoff_col = 'total_dig_0'
        zero_col = 'total_dig_9'
        after_col = 'count_hi'
        cutoff_range = range(2,7)

    mask_actual=create_target_mask(data,bHighCheck)    
    for cutoff in cutoff_range:
        
        print(f'********** cutoff:{cutoff}')
        mask = (data[cutoff_col]==cutoff) & (data[zero_col]==ZERO_COL_CUTOFF) 
        total = len(data[mask])
        # print(data[mask])
        if total ==0:
            success_count=0
            prob=0
        else:
            success_count = len(data[mask & mask_actual])
            prob =round(100 * success_count/ total ,0)
        print(f'before values - probability: {prob} total : {total}  success: {success_count}')
        for val in after_vals:
            mask2 = mask   & (data[after_col]<=val) #& (data['count_hi']+train['count_lo']<=18)
            total = len(data[mask2])
            if total ==0:
                success_count=0
                prob=0
            else:
                success_count = len(data[mask2 & mask_actual])
                prob =round(100 * success_count/ total ,0)
            print(f'AFTER - factor:{val} - probability: {prob} total : {total}  success: {success_count}')


In [296]:
def validate_pattern(initial_df,bHighCheck,zero_col_cutoff,digit,cutoff,after_val):
     
    # rolls_df = initial_df['roll_hash_list'].apply(pd.Series)
    # rolls_df.columns = [f'roll_{i}' for i in range(rolls_df.shape[1])]
    
    # cols_roll = [col for col in rolls_df.columns if col.startswith('roll_') and col not in ['roll_0']]
    rolls_df= initial_df
    
    if bHighCheck:
        cutoff_col = f'count_gt_{digit}000'
        zero_col = 'count_lt_1000'
        after_col = 'count_hi'
    else:
        cutoff_col = 'count_lt_1000'
        zero_col = f'count_gt_{digit}000'
        after_col = 'count_hi'

    
    # rolls_df['client_seed'] = initial_df['client_seed']
    # rolls_df['client_index'] = initial_df['client_index']

    
    mask = (rolls_df[cutoff_col]==cutoff) \
                            & (rolls_df[after_col]<=after_val)  \
                            & (rolls_df[zero_col]==zero_col_cutoff)   \
                            #& (rolls_df['count_hi']+rolls_df['count_lo']<=18)
    
    return rolls_df[mask]
    

In [297]:
# val_res_df = validate_pattern(X_val,True,4,10)
# print(y_val[val_res_df.index])
# print(len(val_res_df))
# val_res_df

In [298]:
# probs= xgb_model.predict_proba(val_res_df)[:,1]
# print(probs)
# np.where(probs>0.3)

In [299]:
# # roll_actual_df = pd.DataFrame(columns=['roll_actual'])
# # roll_actual_df['roll_actual']=train['roll_actual'].iloc[tr_index]
# # print(len(X_tr))
# # print(len(roll_actual_df))

# oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# # fit and apply the transform
# roll_actual_df, y_temp = oversample.fit_resample(train[features+['roll_actual']].iloc[tr_index], 
#                                                  train['target'].iloc[tr_index])
# print(len(roll_actual_df))

# # tr_df = pd.concat([X_tr,roll_actual_df],axis=1)
# # print(len(tr_df))
# train_status(roll_actual_df,True)

In [300]:
%%time
READ_MODEL_FILE = False

if READ_MODEL_FILE:
    xgb_models = joblib.load(f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
else:

    params = { 'n_estimators':100,
              'max_leaves':25,
                'subsample':0.8,
              'random_state':145,
              # 'scale_pos_weight': 5,
    #           'max_depth':6,
            'learning_rate':0.05,
             'colsample_bytree':0.6,#0.85,
             'lambda':0.05,
             'alpha':0.1}

    # xgb_model = xgb.XGBClassifier(**params)
    xgb_models = []
    scores  = []
    ratios =[]
    tr_cutoffs=[]
    val_cutoffs=[]
    tr_last_cutoffs=[]

    for fold,(tr_index, val_index, tr_index_seed) in enumerate(zip(tr_indices_folds,val_indices_folds,tr_indices_seeds)):
        X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
        X_val,y_val = X.iloc[val_index],y.iloc[val_index]

        oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
        # fit and apply the transform
        X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
        print(pd.Series(y_tr).value_counts())


        print(f'\n ******************* fold: {fold} ********')
        print(f'tr size: { len(X_tr)}  val size:  {len(X_val)}')
#         print(f'{tr_index_seed}')
#         print(f'{val_index}')
#         print(f'{tr_index}')

        xgb_model = xgb.XGBClassifier(**params)
        xgb_model.fit(X_tr,y_tr)
        xgb_models.append(xgb_model)

        mean_opt_cutoff_tr = 0
        for tr_index_seed_cur in tr_index_seed:
            X_tr_seed,y_tr_seed = X.iloc[tr_index_seed_cur],y.iloc[tr_index_seed_cur]

            tr_probs = xgb_model.predict_proba(X_tr_seed)[:,1]
            opt_cutoff_tr, f1score = get_opt_cutoff_prec(y_tr_seed,tr_probs)
            # print('tr cutoff:',opt_cutoff_tr)
            tr_labels = convert_probtolabels(tr_probs,cutoff=opt_cutoff_tr) 
            mask1 = (tr_labels==1)
            mask2 = (y_tr_seed==1)
            total = len(tr_labels[mask1])
            success = len(tr_labels[mask1 & mask2])
            ratio = success/total if total!=0 else 0
            # print(f'tr ratio: {ratio} total: {total} success: {success}')
            mean_opt_cutoff_tr += opt_cutoff_tr / len(tr_index_seed)

        print(f'last cutoff :{opt_cutoff_tr}')
        print(f'mean tr cutoff :{mean_opt_cutoff_tr}')
        tr_last_cutoffs.append(opt_cutoff_tr)
        tr_cutoffs.append(mean_opt_cutoff_tr)
        val_probs = xgb_model.predict_proba(X_val)[:,1]
        print(val_probs[0:5])
        labels = convert_probtolabels(val_probs)
        score_init = accuracy_score(y_val,labels)

        opt_cutoff, f1score = get_opt_cutoff_prec(y_val,val_probs)
        print('valid cutoff:',opt_cutoff)
        val_cutoffs.append(opt_cutoff)
        # print(pd.Series(val_probs).describe(percentiles=[0.6,0.7,0.75,0.8]))
        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff_tr) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio: {ratio} total: {total} success: {success}')
        ratios.append(ratio)
        cur_f1 = f1_score(y_val, val_labels,average='macro')
        cur_acc = accuracy_score(y_val,val_labels)
        print(f'val accuracy score:{cur_acc} f1 score:{cur_f1:.4f} initial accuracy score:{score_init}')
        scores.append(cur_acc)

        val_labels = convert_probtolabels(val_probs,cutoff=opt_cutoff) 
        mask1 = (val_labels==1)
        mask2 = (y_val==1)
        total = len(val_labels[mask1])
        success = len(val_labels[mask1 & mask2])
        ratio = success/total if total!=0 else 0
        print(f'ratio @val_cutoff: {ratio} total: {total} success: {success}')

    #     train_status(X_tr,True)

    #     top_prob = np.sort(val_probs)[::-1][:1]
    #     top_label = y_val[val_probs==top_prob ]
    #     print(f'top_prob:{top_prob} top_label:{top_label}')

    print(f'mean score:{np.mean(scores)}')    
    print(f'mean ratio:{np.mean(ratios)}')   
    
    print(np.mean(np.array(val_cutoffs)))
    print(np.mean(np.array(tr_cutoffs)))
    print(np.mean(np.array(tr_last_cutoffs)))
    print(ratios)
    
    joblib.dump(xgb_models,f'data/models/models_{nonce}_{file_pattern_str}_pattern.dump')
    print('Models Save completed')   

0    44919
1    44919
Name: target, dtype: int64

 ******************* fold: 0 ********
tr size: 89838  val size:  2000
[16:44:45] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
last cutoff :0.4016263782978058
mean tr cutoff :0.4016263782978058
[0.49210185 0.38640755 0.45167065 0.5157468  0.36672077]
valid cutoff: 0.40162638
ratio: 0.104 total: 1625 success: 169
val accuracy score:0.258 f1 score:0.2521 initial accuracy score:0.7225
ratio @val_cutoff: 0.104 total: 1625 success: 169
0    44933
1    44933
Name: target, dtype: int64

 ******************* fold: 1 ********
tr size: 89866  val size:  2000
[16:45:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1645117766796/work/src/learner.cc:1115: Starting i

##### Prediction of Test 

In [301]:
# results_df[:train_client_size].head()

In [302]:
# results_df_test.head()

In [303]:
# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     for row in analysis_df.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for model_ind in model:
#                 probs+= model_ind.predict_proba(test_filt_df[features_test])[:,1] / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#         prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]         
#     return analysis_df


In [304]:
# def save_test_ml_probs_internal(analysis_df_filt,
# #                                 features_test,
# #                                 test,model,
#                                 prob_tr_cutoff,prob_val_cutoff,
#                                 pattern,bHighCheck,multi_models):
    
#     start = time.time()
#     #get data from shared memory
#     features_test = p.value[0]
#     test = p.value[1]
#     model = p.value[2]
# #     print(len(test))
# #     print(test.head())
#     end = time.time()
#     print(f'Read Shared memory time:{end-start}')
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df_filt.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df_filt.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df_filt.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df_filt.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df_filt.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df_filt.at[row.Index,'pr_val_cutoff']  = ratios[8]       

#     end = time.time()
#     print(f'Internal function time:{end-start}')
#     return analysis_df_filt  

# def save_test_ml_probs(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
#     other_args=[#features_test,test,model,
#                 prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,multi_models]
#     analysis_df_filt=parallelize_dataframe(analysis_df_filt, save_test_ml_probs_internal, 
#                                            data_to_share=[features_test,test,model],
#                                            other_args=other_args, n_cores= multiprocessing.cpu_count())
#     if len(analysis_df_filt)==len(analysis_df):
#         analysis_df=analysis_df_filt
#     else:
#         analysis_df=pd.concat([analysis_df[~mask],analysis_filt],axis=0)
#     return analysis_df

In [305]:
# temp_df = pd.DataFrame()
# temp_df['one'] = np.array([3,42,20,8,19,54,72,15])
# temp_df['two'] = np.array([0,1,3,1,1,2,2,3])

# temp_df_filt = temp_df[temp_df['one']>=42]
# for row in temp_df_filt.itertuples():
#     print(row.Index)
# #     row.loc[row.Index,'two']=5
#     temp_df.loc[row.Index,'two']=5
# print(temp_df_filt.head())    
# temp_df

In [306]:
# def save_test_ml_probs_old(analysis_df,test,model,prob_tr_cutoff,prob_val_cutoff,pattern,bHighCheck,
#                       multi_models=False):
#     features_test = [col for col in features if col!='target']
#     mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
#     analysis_df_filt = analysis_df[mask]
    
#     start = time.time()
    
#     for row in analysis_df_filt.itertuples():
#         digit = row.digit
#         cutoff = row.cutoff
#         after_val = row.afterval
#         bHighCheck = row.target_high
#         zero_col_cutoff = row.zero_col_cutoff

#         test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
#                                         digit,cutoff,after_val)
#         filt_size = len(test_filt_df)
#         if filt_size==0:
#             print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
#             continue
# #         print(f'validate pattern return size: {filt_size}')
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #                 probs+= convert_probtolabels(cur_prob,cutoff=val_cutoffs[i]) / len(model)
# #             prob_cutoffs = [np.mean(probs),np.median(probs),0.4,0.5,0.6,0.7,0.8,prob_tr_cutoff,prob_val_cutoff]
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]
#             prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4,0.5,prob_tr_cutoff,prob_val_cutoff]
# #         prob_cutoffs = [np.mean(probs),np.median(probs),0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4, 0.45,0.5]
#         ratios = np.zeros(len(prob_cutoffs))
#         total = np.zeros(len(prob_cutoffs))
#         for i,prob_cutoff in enumerate(prob_cutoffs):
#             indices= np.where(probs>prob_cutoff)[0]
#             roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#             total[i] = len(roll_filt)
#             if HIGH_ANALYSIS:
#                 mask = roll_filt>=HIGH_TARGET
#                 mask_arr =roll_filt.values>=HIGH_TARGET
#             else:
#                 mask=roll_filt<LOW_TARGET
#                 mask_arr =roll_filt.values<LOW_TARGET
#             success = len(roll_filt[mask])
#             if total[i]==0:
#                 ratios[i]=0
#             else:
#                 ratios[i] = 100 * success / total[i]
            
#             if (i >= 2) & (i<=6):
#                 analysis_df.at[row.Index,f'pr_{i-1}']  = ratios[i] 
#                 analysis_df.at[row.Index,f'te_pr_{i-1}_count']  = total[i] 
                
# #             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')
            
#         analysis_df.at[row.Index,'pr_mean']  = ratios[0]  
#         analysis_df.at[row.Index,'pr_med']  = ratios[1] 
#         analysis_df.at[row.Index,'pr_tr_cutoff']  = ratios[7]  
#         analysis_df.at[row.Index,'pr_val_cutoff']  = ratios[8]  
        
#     end = time.time()
#     print(f'Function time:{end-start}')   

#     return analysis_df

In [307]:
MEAN_CUTOFF=-1
MEDIAN_CUTOFF=-2
TR_CUTOFF=-3
VAL_CUTOFF=-4


def save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                      analysis_df,row_index,
                      bHighCheck):
    probs = test_filt_df['probs']
    prob_cutoffs = [np.mean(probs),
#                     np.percentile(probs,0.75),
                    np.median(probs),
                    0.1,0.2,0.3,0.4,0.5,
                    prob_tr_cutoff,prob_val_cutoff]
    ratios = np.zeros(len(prob_cutoffs))
    total = np.zeros(len(prob_cutoffs))
    for i,prob_cutoff in enumerate(prob_cutoffs):
        test_filt_prob=test_filt_df.loc[probs>prob_cutoff]
#         indices= np.where(probs>prob_cutoff)[0]
#         roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
        total[i] = len(test_filt_prob)
        mask=create_target_mask(test_filt_prob,bHighCheck)
        success = len(test_filt_prob[mask])
        if total[i]==0:
            ratios[i]=0
        else:
            ratios[i] = 100 * success / total[i]

        if (i >= 2) & (i<=6):
            analysis_df.at[row_index,f'pr_{i-1}']  = ratios[i] 
            analysis_df.at[row_index,f'te_pr_{i-1}_count']  = total[i] 

#             print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')

    analysis_df.at[row_index,'pr_mean']  = ratios[0]  
    analysis_df.at[row_index,'pr_med']  = ratios[1] 
    analysis_df.at[row_index,'pr_tr_cutoff']  = ratios[7]  
    analysis_df.at[row_index,'pr_val_cutoff']  = ratios[8]  
    
    analysis_df.at[row_index,'te_mean_count']  = total[0]  
    analysis_df.at[row_index,'te_med_count']  = total[1] 
    analysis_df.at[row_index,'te_tr_count']  = total[7]  
    analysis_df.at[row_index,'te_val_count']  = total[8]
    
    return analysis_df


def map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff):
    
    if prob_cutoff==MEAN_CUTOFF:
        prob_cutoff=np.mean(probs)
    elif prob_cutoff==MEDIAN_CUTOFF:
        prob_cutoff=np.median(probs)
    elif prob_cutoff==TR_CUTOFF:
        prob_cutoff=prob_tr_cutoff
    elif prob_cutoff==VAL_CUTOFF:
        prob_cutoff=prob_val_cutoff
        
    return prob_cutoff

def generate_probs(multi_models,model,data,features):
    if multi_models:
        probs= np.zeros(len(data))
        for i,model_ind in enumerate(model):
            cur_prob= model_ind.predict_proba(data[features])[:,1] 
            probs+= cur_prob / len(model)
    else: 
        probs= model.predict_proba(test_filt_df[features_test])[:,1]
    data['probs']=probs
    return data

def save_test_ml_probs(analysis_df,test,model,
                       prob_tr_cutoff,prob_val_cutoff,
                       pattern,bHighCheck,
                       gen_probs=True,
                       is_prod=False,
                       prob_cutoff=-1,
                      multi_models=False):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
#         print(f'{filt_size=}')
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue
        if gen_probs:
            test_filt_df = generate_probs(multi_models,model,test_filt_df,features_test)
        probs = test_filt_df['probs'].values
#         if multi_models:
#             probs= np.zeros(len(test_filt_df))
#             for i,model_ind in enumerate(model):
#                 cur_prob= model_ind.predict_proba(test_filt_df[features_test])[:,1] 
#                 probs+= cur_prob / len(model)
#         else: 
#             probs= model.predict_proba(test_filt_df[features_test])[:,1]

        if is_prod:
            prob_cutoff=map_prob_cutoff(prob_cutoff,probs,prob_tr_cutoff,prob_val_cutoff)
            mask = (test_filt_df['probs'] > prob_cutoff)
            test_filt_df=test_filt_df[mask]
        
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
        if not is_prod:
            analysis_df = save_prob_success(test_filt_df,prob_tr_cutoff,prob_val_cutoff,
                              analysis_df,row.Index,bHighCheck)
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all,analysis_df

In [308]:
# features_test = [col for col in features if col!='target']
# test_filt_df = validate_pattern(test[features_test],True,4,9,0,8)

# roll_filt = test['roll_actual'].loc[test_filt_df.index]
# total = len(roll_filt)
# if HIGH_ANALYSIS:
#     success = len(roll_filt[roll_filt>=9000])
# else:
#     success = len(roll_filt[roll_filt<1000])
# if total==0:
#     ratio=0
# else:
#     ratio = success / total
# print(f'ratio:{ratio} total:{total} success:{success}')
# # test_filt_df

In [309]:
# probs= xgb_model.predict_proba(test_filt_df)[:,1]
# # print(probs)
# print(np.mean(probs))
# print(np.median(probs))
# prob_cutoffs = [np.mean(probs),np.median(probs),0.1,0.2,0.3,0.4, 0.5]

# for prob_cutoff in prob_cutoffs:
#     indices= np.where(probs>prob_cutoff)[0]
# #     print(len(indices))
#     # print(indices)
#     # print(test_filt_df.iloc[indices].index)
#     roll_filt = test['roll_actual'].loc[test_filt_df.iloc[indices].index]
#     total = len(roll_filt)
#     if HIGH_ANALYSIS:
#         mask = roll_filt>=9000
#         mask_arr =roll_filt.values>=9000
#     else:
#         mask=roll_filt<1000
#         mask_arr =roll_filt.values<1000
# #     print(roll_filt.tail())
#     success = len(roll_filt[mask])
# #     print(np.where(mask_arr))
#     if total==0:
#         ratio=0
#     else:
#         ratio = success / total
#     print(f'prob_cutoff: {prob_cutoff} ratio:{ratio} total:{total} success:{success}')


In [310]:
CONT_ANALYSIS=False
IS_PROD=False

In [379]:
if not IS_PROD:
    
    actual_seed="e658f73f9831527c16614b72cf87f0a9718961a6099024b6301b10b54dc166d0"
#     actual_seed = cur_hash_list[25]
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    roll_array = vectorized_calculate_roll(actual_seed,
                                           results_df_test['client_seed'],
                                           nonce)

    results_df_test['roll_actual']=roll_array
    test_manual = generate_test_features(results_df_test)
    test = generate_features_full(results_df_test,False,feature_chain_length)

e658f73f9831527c16614b72cf87f0a9718961a6099024b6301b10b54dc166d0
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20', 'roll_21', 'roll_22', 'roll_23', 'roll_24', 'roll_25', 'roll_26', 'roll_27', 'roll_28', 'roll_29', 'roll_30', 'roll_31', 'roll_32', 'roll_33', 'roll_34', 'roll_35', 'roll_36', 'roll_37', 'roll_38', 'roll_39', 'roll_40', 'roll_41', 'roll_42', 'roll_43', 'roll_44', 'roll_45', 'roll_46', 'roll_47', 'roll_48', 'roll_49', 'roll_50', 'roll_51', 'roll_52', 'roll_53', 'roll_54', 'roll_55', 'roll_56', 'roll_57', 'roll_58', 'roll_59', 'roll_60']
['roll_1', 'roll_2', 'roll_3', 'roll_4', 'roll_5', 'roll_6', 'roll_7', 'roll_8', 'roll_9', 'roll_10', 'roll_11', 'roll_12', 'roll_13', 'roll_14', 'roll_15', 'roll_16', 'roll_17', 'roll_18', 'roll_19', 'roll_20']
Roll Features from 1 to 20  completed
['roll_1_nonce', 'roll_2_nonce', 

In [380]:
l1=list(range(1,261))
# l2=list(range(50,80))
for hash_idx in [-1]+l1:
    
    if hash_idx==-1:
        actual_seed="d0e068a90b3c836bda9220c2d0135028f66e80d325a75935baee007c85b73005"
        suffix = 'orig'
    else:    
        actual_seed = cur_hash_list[hash_idx]
        suffix = hash_idx
    print(actual_seed)
    # Vectorize the function
    vectorized_calculate_roll = np.vectorize(calculate_roll)

    # Compute the roll values for the input arrays
    test[f'roll_actual_{suffix}']=vectorized_calculate_roll(actual_seed,
                                           test['client_seed'],
                                           nonce)

d0e068a90b3c836bda9220c2d0135028f66e80d325a75935baee007c85b73005
b465a3bc3240974c62115114d220a77452036cf55edb5bc079668fa89ff9d194
d1b2b6107949538843c017e0ee7d32472bb9f91f78c0f1199beb456e73eea212
55bedd06ab7182b61f6ffceb7d6767e14e5d607dfdde7274a0868a26ef997fb9
93fabba30c6c2c242de21e57ab8fc40ca78b5750ef22f0855be665e93629d113
98c3c4771024c95ff516acd3e1788846438e29f17ca70f0e16e0446911f535f6
3f063a1c8c93610001f819f1d2f1b37ed014d8f0c21277cf263d2a929023b043
633b7f3f2a7ac4e2dca8ba2a9d4b631ba266a0cc7fc9da13a8db4d7cdb2978c1
276857047231ed6ecbdb2be05b70a93ced4a0f6668548a3b5ac7bca3bf422552
7b3ab7a3bcf59ff75b20b8f5e7f0c8040217b000cc8de6c9f774a7cb002e60be
38cb821ec009161b3236d30d3e08ba54fef4b930f66305e4c4b199bab3376af4
ac516663bc01d18d562d68c8c0db9d0bebe76f1f8b2d35bf0406cb94832eef09
9807998051eb0ff83472204b1b1c7ccf5b7254e61e966da9c1a1cac5886b38eb
a5add94f5f939b03ec1b34bfe4f349f422313d0867e399a823d36819523359da
e3fa0c99343ac64f36be1832cee0ac29b42041d4d71aecec59ef186c95423fa5
df8248ac8ad6d6eefd88d9d76

In [313]:
# analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')

In [314]:
mean_imp = np.zeros(len(features)-1)
imp_df = pd.DataFrame()
imp_df['feature'] = [col for col in features if col!='target']
for i,model in enumerate(xgb_models):
    cur_imp = model.feature_importances_
    imp_df[f'importance_m{i}'] = cur_imp
#     print(cur_imp)
    mean_imp += cur_imp / len(xgb_models)

# print(mean_imp)
imp_df['importance'] = mean_imp
imp_df = imp_df.sort_values('importance',ascending=False).reset_index(drop=True)
imp_df

,feature,importance_m0,importance_m1,importance_m2,importance_m3,importance_m4,importance_m5,importance_m6,importance_m7,importance_m8,importance_m9,importance
0,count_lt_1000_nonce,0.005656,0.005921,0.005533,0.004922,0.004409,0.004711,0.005019,0.005339,0.004941,0.004778,0.005123
1,count_gt_6000_25_50,0.003977,0.006341,0.005085,0.003484,0.005127,0.004502,0.005546,0.003174,0.003893,0.004348,0.004548
2,roll_16_nonce,0.003980,0.004904,0.003756,0.004563,0.005208,0.004856,0.004661,0.004036,0.004787,0.004565,0.004532
3,count_lo_hi_nonce,0.003511,0.003980,0.004890,0.004154,0.005168,0.003568,0.004875,0.006252,0.004586,0.003942,0.004493
4,roll_18_nonce,0.004021,0.004289,0.004717,0.004275,0.004155,0.004450,0.005142,0.004119,0.004881,0.004523,0.004457
5,roll_53_nonce,0.004705,0.004492,0.004831,0.005131,0.004212,0.004220,0.004495,0.004145,0.003920,0.004081,0.004423
6,roll_mean_1_60_nonce,0.004650,0.003792,0.004603,0.004577,0.003965,0.003867,0.004646,0.004832,0.004011,0.005260,0.004420
7,roll_57_nonce,0.004518,0.004304,0.004625,0.005118,0.004321,0.004505,0.004100,0.004170,0.004006,0.004311,0.004398
8,roll_45_nonce,0.004479,0.004389,0.004293,0.004877,0.004363,0.004248,0.004123,0.004605,0.004129,0.004470,0.004398
9,roll_mean_1_60,0.004199,0.004076,0.004501,0.004465,0.004133,0.004548,0.004415,0.004879,0.004315,0.004416,0.004395


In [315]:
# %%time
# test_hash25 = generate_features_full(results_df_test_hash25,False,feature_chain_length)
# test_hash25.head()

In [316]:
# %%time
# test_hash24 = generate_features_full(results_df_test_hash24,False,feature_chain_length)
# test_hash24.head()

In [317]:
hash_imp_name='roll_54'
nonce_suffix = '_nonce'
if hash_imp_name.find(nonce_suffix)!=-1:
    is_nonce=True
    hash_imp_name=hash_imp_name.replace(nonce_suffix,'')

hash_no = int(hash_imp_name.replace('roll_',''))
hash_no

54

In [318]:
hash_imp_count=5
roll_imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))
                                and ('std' not in col ) and ('mean' not in col )][:hash_imp_count] 
roll_imp_feats


['roll_16_nonce',
 'roll_18_nonce',
 'roll_53_nonce',
 'roll_57_nonce',
 'roll_45_nonce']

In [319]:
%%time
READ_FROM_FILE_TEST=False
hash_list_nonce = generate_hash_chain(str(nonce),feature_chain_length)
hash_imp_count=3
test_hash_imp = np.zeros((hash_imp_count))
hash_imp_names = [col for col in imp_df['feature'].values if (col.startswith('roll_'))
                                and ('std' not in col ) and ('mean' not in col )][:hash_imp_count] 
nonce_suffix = '_nonce'
test_hash_imp = []
for i in range(hash_imp_count):

    hash_imp_name = hash_imp_names[i]
    
    print(f'********* Processing Test Hash {hash_imp_name} ******************')

    if hash_imp_name.find(nonce_suffix)!=-1:
        is_nonce=True
        hash_imp_name=hash_imp_name.replace(nonce_suffix,'')
        nonce_no = int(hash_imp_name.replace('roll_',''))
        cur_nonce = hash_list_nonce[nonce_no]
        hash_no = 0
        file_suffix =f"_nonce{nonce_no}"
        print(f'{cur_nonce=}')
    else:
        hash_no = int(hash_imp_name.replace('roll_',''))
        cur_nonce= nonce
        file_suffix =f"_hash{hash_no}"

    cur_seed = cur_hash_list[hash_no]
    cur_hash = cur_hash_list[hash_no+1]
    print(f'{file_suffix=}')
    print(f'{hash_no=}')
    print(f'{cur_seed=}')
    print(f'{cur_hash=}')

    results_df_test_hash,test_manual_hash=gen_test_data(cur_nonce,cur_hash,cur_seed,
                                                            file_suffix=file_suffix)
    print()
    print(results_df_test_hash.head(1))
    cur_test_hash_imp = generate_features_full(results_df_test_hash,False,feature_chain_length)
    test_hash_imp.append(cur_test_hash_imp)

********* Processing Test Hash roll_16_nonce ******************
cur_nonce='d7b841d25a0533929ce8a1ded261db55340d72b7104a85b986291132cf07f29d'
file_suffix='_nonce16'
hash_no=0
cur_seed='a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd'
cur_hash='b465a3bc3240974c62115114d220a77452036cf55edb5bc079668fa89ff9d194'
1
                                                               seed  \
0  a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd   

                                                               hash  \
0  b465a3bc3240974c62115114d220a77452036cf55edb5bc079668fa89ff9d194   

                                                              nonce  
0  d7b841d25a0533929ce8a1ded261db55340d72b7104a85b986291132cf07f29d  
server_count:0

999999 999999
No further match for seed:a08fa060e49d0755163928e2445a6d5eb715a9827239fff6aa75a07f7dec36cd nonce:d7b841d25a0533929ce8a1ded261db55340d72b7104a85b986291132cf07f29d match count: 8105

Test Data Generation Completed
['ro

In [320]:
test_hash1 = test_hash.copy()
# test_hash = test_hash24.copy()
# test_hash = test_hash1.copy()

In [321]:
test_filt=test.copy()
test_filt_hash=test_hash.copy()
features_test = [col for col in features if col not in ['target','probs']]

In [322]:
def gen_hash_cutoff_df(models,feature_test,test_hash,k_bin_size=600,quant=0.95):
    test_hash_probs = generate_probs(True,models,test_hash,features_test)
    cutoffs = []
    ratios =[]
    totals =[]
    print(test_hash_probs['probs'].describe())
    cutoffs = np.linspace(test_hash_probs['probs'].min(),
                          test_hash_probs['probs'].quantile(quant),k_bin_size)
    for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):

        mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
        mask2=create_target_mask(test_hash_probs)

        total = len(test_hash_probs[mask])
        if total==0:
            ratio=0
            success=0
        else:
            success = len(test_hash_probs[mask & mask2])
            ratio = success / total
        ratios.append(ratio)
        totals.append(total)
    #     print(ratio,success,total)

    print(len(ratios),len(cutoffs))
    df=pd.DataFrame()
    df['cutoff']=cutoffs[:len(cutoffs)-1]
    df['cutoff_2']=cutoffs[1:len(cutoffs)]
    df['ratio'] =ratios
    df['total'] =totals

    df = df.sort_values(['ratio','cutoff'],ascending=[False,False]).reset_index(drop=True)
#     print(df[:5].mean())
#     print(df[:10].mean())
    return df

In [323]:
# test_hash_probs = generate_probs(True,xgb_models,test_hash,features_test)
# cutoffs = []
# ratios =[]
# totals =[]
# print(test_hash_probs['probs'].describe())
# cutoffs = np.linspace(test_hash_probs['probs'].min(),
#                       test_hash_probs['probs'].quantile(0.95),600)
# for i,cutoff in enumerate(cutoffs[:len(cutoffs)-1]):
# #     cutoff=0.4
# #     diff = 0.01
# #     print(cutoff,cutoffs[i+1])
#     mask = (test_hash_probs['probs']>=cutoff) & (test_hash_probs['probs']<=cutoffs[i+1])
#     mask2=create_target_mask(test_hash_probs)
    
# #     mask2 = test_hash_probs['roll_actual']>=9000
#     total = len(test_hash_probs[mask])
#     if total==0:
#         ratio=0
#         success=0
#     else:
#         success = len(test_hash_probs[mask & mask2])
#         ratio = success / total
#     ratios.append(ratio)
#     totals.append(total)
# #     print(ratio,success,total)

# print(len(ratios),len(cutoffs))
# df=pd.DataFrame()
# df['cutoff']=cutoffs[:len(cutoffs)-1]
# df['cutoff_2']=cutoffs[1:len(cutoffs)]
# df['ratio'] =ratios
# df['total'] =totals

# df = df.sort_values('ratio',ascending=False).reset_index(drop=True)
# print(df[:5].mean())
# print(df[:10].mean())
# df.head(10)

In [324]:
# df

In [325]:
# df.describe()

In [326]:
# print(len(df[df['total']==1]))
# print(len(df[(df['total']==1) & (df['ratio']>0)]))
# df[df['total']==1]

In [327]:
# df[df['ratio']<0.12].index[0]

In [328]:
# test['roll_actual']=test[f'roll_actual_orig'].copy()

In [329]:
def gen_all_k(models,features_test,data,test_hash_cutoff_df,target_total=1,is_k_data_req=True):
    test_probs = generate_probs(True,models,data,features_test)
    top_k=[]
    all_k=[]
    test_filt_all=None
    for k in range(len(test_hash_cutoff_df)):
        row = test_hash_cutoff_df.iloc[k]
    #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        mask2 = create_target_mask(test_probs)    
        test_filt = test_probs[mask]
        

        total = len(test_filt)
        if total==0:
            success=0
            ratio=0
        else:
            success = len(test_probs[mask & mask2])
            ratio = success / total
        if total==target_total:
            all_k.append(k) 
            if is_k_data_req:
#                 cur_test = test_filt[['roll_actual','client_seed','probs']]
                cur_test = test_filt
                cur_test['k']=k
                if test_filt_all is None:
                    test_filt_all = cur_test
                else:
                    test_filt_all = pd.concat([test_filt_all,cur_test],axis=0)
        if ratio>=0.12:
            if total==target_total:
                top_k.append((k,ratio,total))
    #     print(ratio,success,total,row['cutoff'],k)
    print(f'{len(all_k)=}')
    print(all_k)
    print(f'{len(top_k)=}')
    print(top_k)
#     mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
#     test_filt_hash  = test_hash_probs[mask]   
    
    return all_k,top_k,test_filt_all

In [381]:
%%time

k_bin_size=600

print(f'***************** Bin {k_bin_size} ***************** ')
df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
                            k_bin_size=k_bin_size,quant=0.95)
print('Test Hash Cutoff Df Stats')
print('Total size:',len(df[df['total']==1]))
print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

all_k,top_k,test_all_k = gen_all_k(xgb_models,features_test,
                                test,df,target_total=1)

top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
print('Test Size:',len(test_all_k))
test_all_k.head()

***************** Bin 600 ***************** 
count    8154.000000
mean        0.456720
std         0.037570
min         0.278201
25%         0.434736
50%         0.461185
75%         0.483939
max         0.550515
Name: probs, dtype: float64
599 600
Test Hash Cutoff Df Stats
Total size: 53
Success size: 3
len(all_k)=48
[1, 2, 18, 36, 371, 378, 393, 394, 396, 402, 405, 406, 411, 415, 420, 423, 424, 425, 426, 430, 431, 432, 439, 440, 444, 446, 451, 452, 453, 458, 464, 465, 466, 468, 472, 475, 479, 485, 487, 488, 490, 496, 502, 506, 507, 509, 527, 587]
len(top_k)=9
[(2, 1.0, 1), (393, 1.0, 1), (411, 1.0, 1), (415, 1.0, 1), (424, 1.0, 1), (431, 1.0, 1), (490, 1.0, 1), (506, 1.0, 1), (507, 1.0, 1)]
[2, 393, 411, 415, 424, 431, 490, 506, 507]
Test Size: 48
CPU times: user 7.6 s, sys: 412 ms, total: 8.01 s
Wall time: 2.17 s


,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll

In [331]:
# test_sel=test_all_k[:5]
test_sel=test_all_k.sort_values('probs',ascending=False)[:5]
test_sel

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll

In [332]:
top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
matches = set(list(test_sel.k)).intersection(set(top_k_ele))
print(f'{len(matches)}')
print(matches)

[36, 378, 393, 426, 440, 452]
3
{393, 378, 36}


Important Feats Top K Elements

In [333]:
%%time

k_bin_size=600

all_k_imp =[-1]*hash_imp_count
top_k_imp =[-1]*hash_imp_count
test_all_k_imp =[-1]*hash_imp_count
top_k_ele_imp =[-1]*hash_imp_count

df= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
                            k_bin_size=k_bin_size,quant=0.95)
print('Test Hash Cutoff Df Stats')
print('Total size:',len(df[df['total']==1]))
print('Success size:',len(df[(df['total']==1) & (df['ratio']>0)]))

for hash_idx in range(hash_imp_count):

    print(f'***************** Test Hash {hash_imp_names[hash_idx]} ***************** ')

    all_k_imp[hash_idx],top_k_imp[hash_idx],test_all_k_imp[hash_idx] = gen_all_k(xgb_models,features_test,
                                                                        test_hash_imp[hash_idx],
                                                                        df,target_total=1)

    top_k_ele_imp[hash_idx] = [x[0] for x in top_k_imp[hash_idx]]


count    8154.000000
mean        0.456720
std         0.037570
min         0.278201
25%         0.434736
50%         0.461185
75%         0.483939
max         0.550515
Name: probs, dtype: float64
599 600
Test Hash Cutoff Df Stats
Total size: 53
Success size: 3
***************** Test Hash roll_16_nonce ***************** 
len(all_k)=58
[1, 6, 7, 10, 11, 12, 15, 350, 369, 385, 388, 390, 396, 401, 406, 410, 413, 416, 417, 418, 422, 423, 435, 440, 441, 442, 444, 447, 448, 449, 452, 454, 457, 460, 461, 462, 463, 464, 465, 466, 471, 475, 480, 487, 494, 499, 506, 512, 514, 518, 519, 522, 526, 543, 548, 549, 574, 596]
len(top_k)=2
[(442, 1.0, 1), (464, 1.0, 1)]
***************** Test Hash roll_18_nonce ***************** 
len(all_k)=53
[0, 12, 13, 14, 25, 356, 364, 373, 380, 381, 385, 394, 401, 402, 403, 406, 427, 429, 430, 431, 432, 435, 436, 439, 440, 441, 442, 446, 448, 451, 452, 454, 457, 460, 467, 469, 478, 479, 482, 487, 493, 498, 501, 508, 521, 523, 524, 526, 538, 544, 545, 559, 573]
len(

In [334]:
test_all_k_imp[0].head()

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,count_hi_50_60_nonce,count_lo_50_60_nonce,count_lo_hi_50_60_nonce,count_gt_9000_k_50_60_nonce,count_lt_1000_k_50_60_nonce,total_dig_9_50_60_nonce,total_dig_0_50_60_nonce,roll_mean_50_60_nonce,roll_std_50_60_nonce,count_lt_1000_1_60_nonce,count_gt_9

END

In [335]:
%%time
test_sel_all = None
# bin_list = list(range(6054,7800,100))
bin_list = list(range(600,1001,100))
# bin_list = list(range(1000,2001,100))
# bin_list = list(range(2100,3000,100))
# bin_list = list(range(1057,2001,100))
# bin_list = list(range(6000,6500,100))
total_matches=0
start =0
for i,k_bin_size in enumerate(bin_list):
    print(f'***************** Bin {k_bin_size} ***************** ')
    df_temp= gen_hash_cutoff_df(xgb_models,features_test,test_hash,
                                k_bin_size=k_bin_size,quant=0.95)
    end = int(len(test) * (i+1) / len(bin_list) )
    print(f'{end=}')
    all_k_temp,top_k_temp,test_sel_temp = gen_all_k(xgb_models,features_test,
                                                    test,
                                                    df_temp,
                                                    target_total=1)
    start = end
    
    cur_test_sel = test_sel_temp[:5]
    print(test_sel_temp[:10][['k','probs','roll_actual']])
    
#     cur_test_sel=test_sel_temp.sort_values('probs',ascending=False)[:5]
#     cur_test_sel=cur_test_sel[cur_test_sel['k'].isin(all_k_temp[:10])]
    
#     cur_test_sel=test_sel_temp[:100]
# #     print(cur_test_sel[['k','probs','roll_actual']])
#     cur_test_sel=cur_test_sel.sort_values('probs',ascending=False)[:10]
#     print(cur_test_sel[['k','probs','roll_actual']])

    top_k_ele = [x[0] for x in top_k_temp]
    print(top_k_ele)
    matches = set(list(cur_test_sel.k)).intersection(set(top_k_ele))
    print(f'{len(matches)}')
    print(matches)
    
    cur_test_sel['bin']=k_bin_size
    
    
    
    if test_sel_all is None:
        test_sel_all = cur_test_sel
        total_matches +=len(matches)
    else:
        mask = cur_test_sel.index.isin(test_sel_all.index)
        if len(cur_test_sel[mask])==0:
            total_matches +=len(matches)
            test_sel_all = pd.concat([test_sel_all,cur_test_sel],axis=0)
            print(f'total matches: {total_matches} size:{len(test_sel_all)}')
print('Test Size:',len(test_sel_all))
print(f'{total_matches=}')
# print(all_k_temp)
# print(top_k_temp)
test_sel_all.head()

***************** Bin 600 ***************** 
count    8154.000000
mean        0.456720
std         0.037570
min         0.278201
25%         0.434736
50%         0.461185
75%         0.483939
max         0.550515
Name: probs, dtype: float64
599 600
end=1616
len(all_k)=48
[1, 2, 18, 36, 371, 378, 393, 394, 396, 402, 405, 406, 411, 415, 420, 423, 424, 425, 426, 430, 431, 432, 439, 440, 444, 446, 451, 452, 453, 458, 464, 465, 466, 468, 472, 475, 479, 485, 487, 488, 490, 496, 502, 506, 507, 509, 527, 587]
len(top_k)=6
[(36, 1.0, 1), (378, 1.0, 1), (393, 1.0, 1), (426, 1.0, 1), (440, 1.0, 1), (452, 1.0, 1)]
        k     probs  roll_actual
3388    1  0.343630         2644
343     2  0.325485         8109
106    18  0.359054         1540
4804   36  0.375586          893
7794  371  0.372971         1798
1585  378  0.369807          521
1928  393  0.362685          735
7938  394  0.361893         6316
5104  396  0.360471         7025
234   402  0.358097         7718
[36, 378, 393, 426, 440, 45

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [336]:
temp = df[df.index.isin(all_k)].sort_values('cutoff',ascending=False)
temp_filt = temp[:10]
print(temp_filt.index)
top_k_ele = [x[0] for x in top_k]
print(top_k_ele)
set(list(temp_filt.index)).intersection(set(top_k_ele))

Int64Index([36, 371, 378, 393, 394, 396, 18, 402, 405, 406], dtype='int64')
[36, 378, 393, 426, 440, 452]


{36, 378, 393}

In [337]:
# test_probs = generate_probs(True,xgb_models,test,features_test)
# top_k=[]
# all_k=[]
# for k in range(len(df)):
#     row = df.iloc[k]
# #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
#     mask2 = create_target_mask(test_probs)    
#     test_filt = test_probs[mask]

#     total = len(test_filt)
#     if total==0:
#         success=0
#         ratio=0
#     else:
#         success = len(test_probs[mask & mask2])
#         ratio = success / total
#     if total==1:
#         all_k.append(k) 
#     if ratio>=0.12:
#         if total==1:
#             top_k.append((k,ratio,total))
# #     print(ratio,success,total,row['cutoff'],k)
# print(len(all_k))
# print(all_k)
# print(len(top_k))
# print(top_k)
# mask = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
# test_filt_hash  = test_hash_probs[mask]

In [338]:
# all_k_arr = np.array(all_k)
# diff = np.diff(all_k_arr)
# print(diff)
# ind = np.where(diff>=20)
# print(ind[0]+1)
# trans_k = np.sort(np.concatenate((all_k_arr[ind[0]],all_k_arr[ind[0]+1])))
# print(len(trans_k))
# trans_k

In [339]:
# def get_sim_cutoff_data(test,xgb_models,features_test,test_hash_cutoff_df,
#                        is_imp_feats,imp_feats_count,gen_probs=True):
#     if gen_probs:
#         test_probs = generate_probs(True,xgb_models,test,features_test)
#     else:
#         test_probs = test
#     print('Generate probs completed')
#     test_probs['roll_actual_init']=test_probs['roll_actual'].copy()

#     if is_imp_feats:
#         hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
# #         hash_groups = [['roll_mean_nonce']]
#     else:
#     #     hash_groups = [['roll_21_nonce', 'roll_37_nonce', 'roll_mean_nonce','roll_36_nonce','roll_48',
#     #                    'roll_53_nonce','roll_32_nonce','roll_44_nonce','roll_38_nonce','roll_17']]
#         hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
# #         hash_groups = [list(range(25,36)) ]
#     hash_groups_flat = [item for sublist in hash_groups for item in sublist]
#     print(hash_groups)
#     print('list creation complete')
    

#     top_k=[]
#     sim_cutoff_df = pd.DataFrame(columns=['k','ratio','total','hash_no','hash_group','roll_mean'])
#     df_size = len(test_hash_cutoff_df)
#     for i,row in enumerate(test_hash_cutoff_df.itertuples()):
#         k=row.Index
# #         if (k%10)==0:
# #             print(f'loop for k = {k}')
# #         row = test_hash_cutoff_df.iloc[k]
#     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
#         mask = (test_probs['probs']>=row.cutoff) & (test_probs['probs']<=row.cutoff_2)
#         test_filt = test_probs[mask]
#         total = len(test_filt)
#         ratios=[]
        
#         # test_filt[hash_groups_flat] <
# #     if HIGH_ANALYSIS:
# #         success = (test_filt[hash_groups_flat] > HIGH_TARGET).sum(axis=1)
# #     else:
# #         success = (test_filt[hash_groups_flat] < LOW_TARGET).sum(axis=1)
    
# #     ratio_mean = success / len(hash_groups_flat)
# #     roll_mean_mean = test_filt[hash_groups_flat].mean(axis=1)

#         for idx in hash_groups_flat:
# #             print(f'k:{k} idx:{idx}')
#             if is_imp_feats:
#                 test_filt['roll_actual']=test_filt[idx].copy()    
#             else:
#                 test_filt['roll_actual']=test_filt[f'roll_actual_{idx}'].copy()
#             mask2 = create_target_mask(test_filt)
#             if total==0:
#                 success=0
#                 ratio=0
#             else:
#                 success = len(test_filt[mask2])
#                 ratio = success / total
# #             print(ratio,success,total,row['cutoff'],k)
#             roll_mean=test_filt['roll_actual'].mean()
#             sim_cutoff_df.loc[len(sim_cutoff_df.index)] = [k,ratio, total, idx, np.nan,roll_mean]

#     for group_no,group in enumerate(hash_groups):
#         start =group[0]
#         end = group[-1]
#         mask = (sim_cutoff_df['hash_no']>=start) & (sim_cutoff_df['hash_no']<=end) 
#         sim_cutoff_df.loc[mask,'hash_group']=group_no
        
#     test_probs['roll_actual']=test_probs['roll_actual_init'].copy()
    
#     return sim_cutoff_df,test_probs
        



In [340]:
# def func(data):
#     return data['x']**2


# temp_df = pd.DataFrame(columns=['x','y','z'])
# temp_df['x']=np.random.randint(5,12,size=10)
# temp_df['y']=np.random.randint(65,82,size=10)
# temp_df['z']=np.random.randint(1000,2000,size=10)
# res=temp_df.apply(func)
# res

In [341]:
def get_sim_summary(test_filt_all_k,
                       test_hash_cutoff_df,
                       is_imp_feats,imp_feats_count):

    if is_imp_feats:
        print('imp_feats_count:',imp_feats_count)
        hash_groups =[ [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:imp_feats_count] ]
    else:
        hash_groups = [list(range(1,11)),list(range(25,36)),list(range(50,60)),list(range(100,111)) ]
    hash_groups_flat = [item for sublist in hash_groups for item in sublist]
    print(hash_groups)
    
    
    if HIGH_ANALYSIS:
        success = (test_filt_all_k[hash_groups_flat] > HIGH_TARGET).sum(axis=1)
    else:
        success = (test_filt_all_k[hash_groups_flat] < LOW_TARGET).sum(axis=1)
    
    ratio_mean = success / len(hash_groups_flat)
    roll_mean = test_filt_all_k[hash_groups_flat].mean(axis=1)
    
    sim_summary = pd.DataFrame()
    sim_summary['k']=test_filt_all_k['k']
    sim_summary['ratio_mean']=ratio_mean
    sim_summary['roll_mean']=roll_mean
    
    sim_summary=sim_summary.reset_index(drop=True)
    
    return sim_summary
    

In [342]:
# def get_hash_prob_top_data(test,xgb_models,features_test,
#                            test_hash_cutoff_df,is_imp_feats,imp_feats_count,
#                            test_filt_all_k,
#                            sim_cutoff_df = None,is_min_total=True,
#                            total_cri=12,top_n=10,
#                           ratio_mean_asc=False,roll_mean_asc=True):

# #     if sim_cutoff_df is None:
# #         sim_cutoff_df,test_probs= get_sim_cutoff_data(test,xgb_models,features_test,
# #                                                       test_hash_cutoff_df,
# #                                                      is_imp_feats,imp_feats_count)
# #     else:
# #         test_probs = generate_probs(True,xgb_models,test,features_test)
        
# #     print(len(sim_cutoff_df))
# # #     print(sim_cutoff_df.head())
# #     #filter only high total records
# #     if is_min_total:
# #         sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']>=total_cri]
# #     else:
# #         sim_cutoff_df_filt = sim_cutoff_df[sim_cutoff_df['total']<=total_cri]
# #         if total_cri!=1:
# #             sim_cutoff_df_filt = sim_cutoff_df_filt[sim_cutoff_df_filt['total']>1]
# #     print(len(sim_cutoff_df_filt))
# # #     print(sim_cutoff_df_filt.dtypes)
# #     sim_cutoff_df_filt['total']=  sim_cutoff_df_filt['total'].astype('float')
# #     sim_summary  = sim_cutoff_df_filt.groupby(['k']).agg( total =('total','mean'),
# #                                                         roll_mean=('roll_mean','mean'),
# #                                                          ratio_mean=('ratio', 'mean'), 
# #                                                           ratio_std=('ratio', 'std')).reset_index()
    
#     sim_summary= get_sim_summary(test_filt_all_k,test_hash_cutoff_df,is_imp_feats,imp_feats_count)
# #     print(sim_summary.head(20))
# #     print(sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=False))
    
#     top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[ratio_mean_asc,roll_mean_asc])[:top_n].reset_index(drop=True)
#     print('--- Top 5 data ---')
#     print(top_mean_data[:5])
# #     print('--- Top 6 to 10 data ---')
# #     print(top_mean_data[5:20])
# #     top_std_row = top_mean_data.sort_values('ratio_std',ascending=True).iloc[0]
# #     print('--- Selected K ---')
# #     print(top_std_row)
# #     k_sel = int(top_std_row['k'])

# #     k_sel = int(top_mean_data[0:1]['k'])
# #     row = test_hash_cutoff_df.iloc[k_sel]
# #     #     print(row['ratio'],row['cutoff'],row['cutoff_2'])
# #     mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# #     test_sel = test_probs[mask]
# #     print('Test Size:',len(test_sel))
    
#     return top_mean_data


In [343]:
# k_sel,test_sel = get_test_sel_hash_prob(test,xgb_models,features_test,df,
#                           min_total=12, top_n=10)
# print(k_sel)

In [344]:
def create_cutoff_pos_df(test,models,features_test,test_hash_cutoff_df,
                         imp_feats_count_list,test_filt_all_k,
                         gen_probs=True,is_compute_matches=True,
                   test_ratio_limit=0.12,sim_cutoff_dfs=None,
                         low_high_record_condns=[(True,12,10),(False,10,10),(False,2,10)],
                        ratio_mean_asc=False,roll_mean_asc=True,
                        top_k_ele=None):

    cutoff_pos_df =pd.DataFrame(columns = ['imp_feats_count','condn_max','condn_val',
                                           'k_top_1','k_top_2','k_top_3','k_top_4','k_top_5',
                                          'mean_top_1', 'mean_top_2','mean_top_3','mean_top_4',
                                          'mean_top_5','match_pos_1',
                                          'match_total_top_5','match_total_top_10','match_ks_top_5'])

    
    max_good_k= test_hash_cutoff_df[test_hash_cutoff_df['ratio']<test_ratio_limit].index[0]-1
    print(f'{max_good_k=}')
    is_imp_feats = True 
    gen_sim_cutoff=False
    if sim_cutoff_dfs is None:
        gen_sim_cutoff=True
        sim_cutoff_dfs=[]
    elif gen_probs:
        test_probs = generate_probs(True,xgb_models,test,features_test) 
    else:
        test_probs = test
    top_mean_data_list=[]
    for i,imp_feats_count in enumerate(imp_feats_count_list):
        print()
        print(f'********************** Feature count {imp_feats_count} *****************')
#         if gen_sim_cutoff:
#             sim_cutoff_df,test_probs= get_sim_cutoff_data(test,models,features_test,test_hash_cutoff_df,
#                                                      is_imp_feats,imp_feats_count,gen_probs)
#             sim_cutoff_dfs.append(sim_cutoff_df)
#         else:
#             sim_cutoff_df=sim_cutoff_dfs[i]
        
        for condn in low_high_record_condns:
            print(f'******* Condition {condn} for feature count {imp_feats_count} *****************')
#             top_mean_data = get_hash_prob_top_data(test,models,features_test,
#                                        test_hash_cutoff_df,is_imp_feats,imp_feats_count,
#                                         test_filt_all_k,
#                                        sim_cutoff_df = None,
#                                         is_min_total=condn[0], total_cri=condn[1],
#                                         top_n=condn[2],
#                                         ratio_mean_asc=ratio_mean_asc,roll_mean_asc=roll_mean_asc)
            
            sim_summary= get_sim_summary(test_filt_all_k,test_hash_cutoff_df,
                                         is_imp_feats,imp_feats_count)
            top_mean_data = sim_summary.sort_values(['ratio_mean','roll_mean'],ascending=[ratio_mean_asc,roll_mean_asc])[:top_n].reset_index(drop=True)
            print('--- Top 5 data ---')
            print(top_mean_data[:5])

            top_mean_data_list.append(top_mean_data)
            
            if top_k_ele is not None:
                print(f'\n******success data *****')
                mask= top_mean_data['k'].isin(top_k_ele)
                print(top_mean_data[mask])
                print(f'\n******success positions *****')
                cur_positions = list(top_mean_data[mask].index+1)
                print(cur_positions)
            
#             print(top_mean_data)
            top_5_mean_data = top_mean_data[:5]
            k_tops=list(top_5_mean_data['k'].values)
            mean_tops=list(top_5_mean_data['ratio_mean'].values)
            
            match_pos_1=np.nan
            match_total_top_5=np.nan
            match_total_top_10=np.nan
            match_ks_top_5=np.nan
            
            if is_compute_matches:
                
                top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='average',
                                                                  ascending=False)
                top_5_mean_data = top_mean_data[:5]
               
                matches_top_5=top_5_mean_data[top_5_mean_data['k']<=max_good_k]
                matches_top_5=matches_top_5[matches_top_5['ratio_mean']>0]
                match_total_top_5=len(matches_top_5)

                matches_top=top_mean_data[top_mean_data['k']<=max_good_k]
                matches_top=matches_top[matches_top['ratio_mean']>0]
                match_total_top_10 = len(matches_top)

                if match_total_top_5==0:
                    match_ks_top_5=[]
                    if match_total_top_10==0:
                        match_pos_1=11
                    else:
                        match_pos_1=matches_top.iloc[0]['rank']
                else:
    #                 match_pos_1=matches_top_5.index[0]+1
                    match_pos_1=matches_top_5.iloc[0]['rank']
                    match_ks_top_5 = list(matches_top_5['k'].values)
 
            
            result = [imp_feats_count,condn[0],condn[1]]+k_tops+mean_tops \
                    +[match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5]
        
#             print(cutoff_pos_df.columns)
#             print(f'{len(cutoff_pos_df.columns),len(result),result}')
            next_idx = len(cutoff_pos_df.index)
            cutoff_pos_df.loc[next_idx] = result
        
            if is_compute_matches:
                match_cols = [col for col in cutoff_pos_df if col.startswith('match')]
                print(cutoff_pos_df.loc[next_idx][match_cols])
        
    return cutoff_pos_df,sim_cutoff_dfs,top_mean_data_list


In [345]:
import math

def get_test_sel_cutoff_pos(test_probs,models,features_test,test_hash_cutoff_df,
                            cutoff_pos_df_hash,
                            is_min_total=False,total_cri=2,top_n=10,
                           sim_cutoff_dfs=None,imp_feats_count_list=None):
    cutoff_pos_stat = cutoff_pos_df_hash.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Overall Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_df_hash_filt=cutoff_pos_df_hash[cutoff_pos_df_hash['condn_max']==False]
    cutoff_pos_stat = cutoff_pos_df_hash_filt.groupby(['imp_feats_count'])['match_pos_1',
                                               'match_total_top_5',
                                               'match_total_top_10'].mean().reset_index()
    print(' ........... Low Positions of Features .........')
    print(cutoff_pos_stat)
    cutoff_pos_best= cutoff_pos_stat.sort_values(['match_pos_1','match_total_top_5'],
                                                ascending=[True,False])\
                                    .reset_index().iloc[0]
    print('*** Best cutoff position in hash **** ')
    print(cutoff_pos_best)
    predicted_raw_pos=cutoff_pos_best['match_pos_1']
    predicted_max_pos = math.ceil(predicted_raw_pos)
    
    if predicted_raw_pos==predicted_max_pos:
        predicted_max_pos+=1
    
    is_imp_feats=True
    imp_feats_count=cutoff_pos_best['imp_feats_count']
    is_min_total=False
    
    if sim_cutoff_dfs is None:
        sim_cutoff_df,test_probs= get_sim_cutoff_data(test_probs,models,features_test,test_hash_cutoff_df,
                                                 is_imp_feats,imp_feats_count,gen_probs=False)
    else:
        #to use existing buffer, read the matching sim cutoff df from the list of buffer
        idx = imp_feats_count_list.index(imp_feats_count)
        sim_cutoff_df=sim_cutoff_dfs[idx]

    top_mean_data = get_hash_prob_top_data(test_probs,models,features_test,
                           test_hash_cutoff_df,is_imp_feats,imp_feats_count,
                           sim_cutoff_df = sim_cutoff_df,
                            is_min_total=is_min_total, total_cri=total_cri,
                            top_n=top_n)
    
    top_mean_data['rank']=top_mean_data['ratio_mean'].rank(method='min',ascending=False)
    mask = (top_mean_data['rank']<=predicted_max_pos) & (top_mean_data['ratio_mean']>0)
    top_mean_sel = top_mean_data[mask]
    k_sel_list = list(top_mean_sel['k'].values)
    print(top_mean_data)
    print(top_mean_sel)
    print(f'{k_sel_list=}')
    

#     mask = cutoff_pos_df_actual['imp_feats_count']==cutoff_pos_best['cutoff_pos_best']
#     mask2 = cutoff_pos_df_actual['match_pos_1']<=predicted_max_pos
#     k_sel = cutoff_pos_df_actual.loc[mask & mask2, 'k' ]
    
    test_sel  = None
    for k_sel in k_sel_list:
        row = test_hash_cutoff_df.loc[k_sel]
    #     print(row)
        mask = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
        cur_test_sel = test_probs[mask]
        cur_test_sel['k']=k_sel
        if test_sel is None:
            test_sel = cur_test_sel
        else:
            test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
        print('Test Size:',len(test_sel))

    return k_sel_list,test_sel,top_mean_sel



In [346]:
# k_sel_list=[218,198,201,190,184]
# print(df[df.index.isin(k_sel_list)])
# test_temp=None
# for k_sel in k_sel_list:
#     row = df.loc[k_sel]
# #     print(row)
#     mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
#     cur_test_sel = test[mask]
#     cur_test_sel['k']=k_sel
#     if test_temp is None:
#         test_temp = cur_test_sel
#     else:
#         test_temp = pd.concat([test_temp,cur_test_sel],axis=0)

# cols = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:20]        
# # test_temp
# test_temp['feats_mean']=test_temp[cols].mean(axis=1)
# test_temp['feats_std']=test_temp[cols].std(axis=1)
# test_temp['feats_count']=(test_temp[cols]<1000).sum(axis=1)
# test_temp[['k','roll_actual','feats_mean','feats_std','feats_count']+cols]

In [347]:
# test_orig= test.copy()
# test_hash_orig = test_hash.copy()

In [348]:
count_all_k = len(all_k)
if count_all_k<10:
    print(f'PROJECT WARNING: number of target records is {count_all_k} which is less than 10 ')
count_top_k = len(top_k)    
if count_top_k<2:
    print(f'PROJECT WARNING: number of probable success is {count_top_k} which is less than 2')  
hash_success = len(df[(df['total']==2) & (df['ratio']>0)])    
if hash_success<2:
    print(f'PROJECT WARNING: number of hash success is {hash_success} which is less than 2')

In [349]:
print(len(test))
mask = test.duplicated(subset='client_seed')
print(len(test[mask]))
print(test[mask]['client_seed'].nunique())

print(len(test_hash))
mask = test_hash.duplicated(subset='client_seed')
print(len(test_hash[mask]))
print(test_hash[mask]['client_seed'].nunique())

8084
0
0
8154
0
0


In [350]:
# test=test.drop_duplicates(subset='client_seed')
# test_hash=test_hash.drop_duplicates(subset='client_seed')

In [351]:
# %%time
# sim_cutoff_dfs_temp=None
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# cutoff_pos_df_temp,sim_cutoff_dfs_temp,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin([2,22,70,100,178,200,486,
#                                                     488,525,604,672,681,747,
#                                                    812,1212,1392,1536])],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_temp,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_temp

In [352]:
# %%time
# imp_feats_count_list = [30,40]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[df['total']<=10],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,10,20),(False,2,20)])
# cutoff_pos_df_hash

In [353]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [354]:
# print(len(df))
# df_filt = df[(df['total']>=50)]
# print(len(df_filt))
# # df_filt[df_filt['ratio']==0.12]

In [355]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs=None
# min_normal_count=50
# max_low_count = 5
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']<=max_low_count) | (df['total']>=min_normal_count)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(True,min_normal_count,10),
#                                             (False,max_low_count,20),(False,2,20)])
# cutoff_pos_df_hash

In [356]:
# cutoff_pos_df_hash[cutoff_pos_df_hash['condn_val']==2]

In [367]:
print(all_k)
# print(trans_k)

[1, 2, 18, 36, 371, 378, 393, 394, 396, 402, 405, 406, 411, 415, 420, 423, 424, 425, 426, 430, 431, 432, 439, 440, 444, 446, 451, 452, 453, 458, 464, 465, 466, 468, 472, 475, 479, 485, 487, 488, 490, 496, 502, 506, 507, 509, 527, 587]


In [358]:
# %%time
# imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
# sim_cutoff_dfs=None
# cutoff_pos_df_hash,sim_cutoff_dfs,test_probs_hash=create_cutoff_pos_df(test_hash,xgb_models,
#                     features_test,df[(df['total']==1)],
#                     imp_feats_count_list,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs,
#                     low_high_record_condns=[(False,1,20)])
# cutoff_pos_df_hash

In [359]:
# temp=df.loc[[243,181,220,226,245,10,4]]
# temp['diff']=(df.loc[214,'cutoff']-temp['cutoff']).abs()
# temp.sort_values('diff')

In [360]:
# # set(top_mean_data_actual[0]['k']).intersection(set(top_k_ele))

# for i in range(13):
#     cur_top_data = top_mean_actual[i]
#     print(f'******{imp_feats_count_list[i]=}*****')
#     print(cur_top_data[cur_top_data['k'].isin(top_k_ele)])
#     print(cur_top_data[cur_top_data['k'].isin(top_k_ele)].index+1)

Check top important hash test data for ascending or descending ratio mean performance

In [368]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
# imp_feats_count_list = [2,5]
top_n = 62
imp_feats_size = len(imp_feats_count_list)
first_match_pos_arr= np.zeros((hash_imp_count,2,imp_feats_size))
dictionary_list=[]
for hash_idx in range(hash_imp_count):
    top_n = len(all_k_imp[hash_idx])
    print(f'\ntop_n:{top_n}')
    for ratio_mean_order in [False,True]:
        print(f'********************** Test Hash {hash_imp_names[hash_idx]} ratio_mean order {ratio_mean_order}***********************')
        sim_cutoff_dfs_actual=None
        cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test_hash_imp[hash_idx],xgb_models,
                            features_test,df[df.index.isin(all_k_imp[hash_idx])],
                            imp_feats_count_list,test_all_k_imp[hash_idx],is_compute_matches=False,
                           test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                            low_high_record_condns=[(False,1,top_n)],
                            ratio_mean_asc=ratio_mean_order,roll_mean_asc=not(HIGH_ANALYSIS))

        first_match_pos_list=[]
        match_pos_list=[]
        for feats_idx in range(imp_feats_size):
            cur_top_data = top_mean_actual[feats_idx]
            print(f'\n******{imp_feats_count_list[feats_idx]} feats success data *****')
            mask= cur_top_data['k'].isin(top_k_ele_imp[hash_idx])
            print(cur_top_data[mask])
            cur_positions = list(cur_top_data[mask].index+1)
            
            dictionary_data = {'imp_feats_count': imp_feats_count_list[feats_idx],
                               'hash_index':hash_idx, 
                               'hash_feat_name': hash_imp_names[hash_idx], 
                                'ratio_mean_asc':ratio_mean_order,
                                'first_match_pos':cur_positions[0],
                                'other_match_pos':cur_positions[1:]
                  }
            dictionary_list.append(dictionary_data)
            print(f'\n {imp_feats_count_list[feats_idx]} feats success positions')
            print(cur_positions)
        
        
match_pos_df = pd.DataFrame.from_dict(dictionary_list)
match_pos_df        


top_n:58
********************** Test Hash roll_16_nonce ratio_mean order False***********************
max_good_k=349

********************** Feature count 2 *****************
******* Condition (False, 1, 58) for feature count 2 *****************
imp_feats_count: 2
[['roll_16_nonce', 'roll_18_nonce']]
--- Top 5 data ---
     k  ratio_mean  roll_mean
0  464         1.0      376.0
1  506         1.0      698.0
2  418         0.5     1443.0
3  549         0.5     3318.5
4  410         0.5     4748.0

********************** Feature count 5 *****************
******* Condition (False, 1, 58) for feature count 5 *****************
imp_feats_count: 5
[['roll_16_nonce', 'roll_18_nonce', 'roll_53_nonce', 'roll_mean_1_60_nonce', 'roll_57_nonce']]
--- Top 5 data ---
     k  ratio_mean    roll_mean
0  418         0.4  2166.570000
1  369         0.4  2569.163333
2  506         0.4  2973.493333
3  464         0.4  3608.876667
4  413         0.2  2985.613333

********************** Feature count 10 ***

,imp_feats_count,hash_index,hash_feat_name,ratio_mean_asc,first_match_pos,other_match_pos
0,2,0,roll_16_nonce,False,1,[55]
1,5,0,roll_16_nonce,False,4,[44]
2,10,0,roll_16_nonce,False,2,[56]
3,15,0,roll_16_nonce,False,3,[51]
4,20,0,roll_16_nonce,False,1,[50]
5,25,0,roll_16_nonce,False,2,[52]
6,30,0,roll_16_nonce,False,2,[54]
7,35,0,roll_16_nonce,False,2,[57]
8,40,0,roll_16_nonce,False,4,[52]
9,45,0,roll_16_nonce,False,5,[53]


In [369]:
        
match_summary =match_pos_df.groupby(['hash_index','ratio_mean_asc'])\
                            .agg(first_match_pos_mean =('first_match_pos','mean'))
                                                                        
print(match_summary)
match_top_pos = match_summary['first_match_pos_mean'].groupby('hash_index', group_keys=False).nsmallest(1)
match_top_pos = match_top_pos.reset_index()
print(match_top_pos)
match_value_counts = match_top_pos['ratio_mean_asc'].value_counts()
top_ratio_mean_asc = match_value_counts.index[0]  
if len(match_value_counts)==2:
    ratio_mean_order_equal = match_top_pos['ratio_mean_asc'].value_counts().iloc[0]==match_top_pos['ratio_mean_asc'].value_counts().iloc[1]
    if ratio_mean_order_equal:
        print('PROJECT WARNING: Both Ratio mean order ascending and descending are equal')
top_ratio_mean_asc

                           first_match_pos_mean
hash_index ratio_mean_asc                      
0          False                       4.230769
           True                       11.076923
1          False                       6.461538
           True                        7.384615
2          False                      10.538462
           True                       28.000000
   hash_index  ratio_mean_asc  first_match_pos_mean
0           0           False              4.230769
1           1           False              6.461538
2           2           False             10.538462


False

In [363]:
len(match_value_counts)

1

In [370]:
# match_pos_df.groupby('hash_index')
mask = (match_pos_df['ratio_mean_asc']==top_ratio_mean_asc)
# mask = (match_pos_df['ratio_mean_asc']==False)
pos_top_df= match_pos_df[mask].sort_values(['hash_index','first_match_pos'],ascending=True)\
            .groupby(['hash_index'])\
            .head(5)
print(pos_top_df['imp_feats_count'].value_counts())
pos_top_df
# match_pos_df.sort_values(['first_match_pos'],ascending=True)\
#             .groupby(['hash_index','ratio_mean_asc'])\
#             .head(5)\
#             .sort_values(['hash_index','ratio_mean_asc'],ascending=True)
    

2     3
40    2
20    1
10    1
25    1
30    1
45    1
15    1
35    1
55    1
60    1
50    1
Name: imp_feats_count, dtype: int64


,imp_feats_count,hash_index,hash_feat_name,ratio_mean_asc,first_match_pos,other_match_pos
0,2,0,roll_16_nonce,False,1,[55]
4,20,0,roll_16_nonce,False,1,[50]
2,10,0,roll_16_nonce,False,2,[56]
5,25,0,roll_16_nonce,False,2,[52]
6,30,0,roll_16_nonce,False,2,[54]
34,40,1,roll_18_nonce,False,1,"[9, 23, 48, 50, 51]"
35,45,1,roll_18_nonce,False,2,"[10, 25, 39, 43, 51]"
29,15,1,roll_18_nonce,False,3,"[13, 37, 38, 45, 48]"
33,35,1,roll_18_nonce,False,4,"[11, 26, 46, 48, 49]"
26,2,1,roll_18_nonce,False,5,"[28, 40, 42, 44, 45]"


In [384]:
%%time
imp_feats_count_list = [2,5,10,15,20,25,30,35,40,45,50,55,60]
sim_cutoff_dfs_actual=None
top_n = len(all_k)
cutoff_pos_df_actual,sim_cutoff_dfs_actual,top_mean_actual=create_cutoff_pos_df(test,xgb_models,
                    features_test,df[df.index.isin(all_k)],
                    imp_feats_count_list,test_all_k,is_compute_matches=False,
                   test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
                    low_high_record_condns=[(False,1,top_n)],
                    ratio_mean_asc=top_ratio_mean_asc,roll_mean_asc=not(HIGH_ANALYSIS),
                    top_k_ele=top_k_ele)
cutoff_pos_df_actual

max_good_k=370

********************** Feature count 2 *****************
******* Condition (False, 1, 48) for feature count 2 *****************
imp_feats_count: 2
[['roll_16_nonce', 'roll_18_nonce']]
--- Top 5 data ---
     k  ratio_mean  roll_mean
0  423         1.0       82.0
1  475         1.0      591.5
2  424         0.5      837.5
3  506         0.5     1463.5
4  502         0.5     2116.0

******success data *****
      k  ratio_mean  roll_mean
2   424         0.5      837.5
3   506         0.5     1463.5
5     2         0.5     2775.0
8   507         0.5     3575.5
12  490         0.5     4526.0
20  411         0.0     3568.5
28  431         0.0     5052.5
31  393         0.0     5252.5
35  415         0.0     5765.5

******success positions *****
[3, 4, 6, 9, 13, 21, 29, 32, 36]

********************** Feature count 5 *****************
******* Condition (False, 1, 48) for feature count 5 *****************
imp_feats_count: 5
[['roll_16_nonce', 'roll_18_nonce', 'roll_53_nonce', 

,imp_feats_count,condn_max,condn_val,k_top_1,k_top_2,k_top_3,k_top_4,k_top_5,mean_top_1,mean_top_2,mean_top_3,mean_top_4,mean_top_5,match_pos_1,match_total_top_5,match_total_top_10,match_ks_top_5
0,2,False,1,423,475,424,506,502,1.000000,1.000000,0.500000,0.500000,0.500000,NaN,NaN,NaN,NaN
1,5,False,1,424,423,485,475,405,0.400000,0.400000,0.400000,0.400000,0.400000,NaN,NaN,NaN,NaN
2,10,False,1,507,18,2,468,424,0.400000,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
3,15,False,1,507,468,423,424,452,0.266667,0.200000,0.200000,0.200000,0.200000,NaN,NaN,NaN,NaN
4,20,False,1,488,415,507,18,420,0.250000,0.200000,0.200000,0.150000,0.150000,NaN,NaN,NaN,NaN
5,25,False,1,488,502,415,18,468,0.200000,0.200000,0.200000,0.160000,0.160000,NaN,NaN,NaN,NaN
6,30,False,1,472,488,502,415,18,0.200000,0.166667,0.166667,0.166667,0.133333,NaN,NaN,NaN,NaN
7,35,False,1,488,415,440,472,18,0.171429,0.171429,0.171429,0.171429,0.142857,NaN,NaN,NaN,NaN
8,40,False,1,488,464,440,490,502,0.175000,0.175000,0.175000,0.150000,0.150000,NaN,NaN,NaN,NaN
9,45,False,1,488,464,440,502,490,0.177778,0.177778,0.155556,0.133333,0.133333,NaN,NaN,NaN,NaN


In [ ]:
# %%time
# # imp_feats_count_list = [2,5,10,15,20,25,30,40]
# sim_cutoff_dfs_actual=None
# cutoff_pos_df_actual,sim_cutoff_dfs_actual,test_probs=create_cutoff_pos_df(test,xgb_models,
#                     features_test,df[df.index.isin(trans_k)],
#                     imp_feats_count_list,is_compute_matches=False,
#                    test_ratio_limit=0.12,sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                     low_high_record_condns=[(False,2,20)])
# cutoff_pos_df_actual

In [ ]:
# # k_sel,test_sel,cutoff_pos_best =get_test_sel_cutoff_pos(test_probs,cutoff_pos_df_hash,cutoff_pos_df_actual)
# # imp_feats_count_list = [2,5,10,15,20,25]

# k_sel,test_sel,top_mean_best =get_test_sel_cutoff_pos(test_probs,xgb_models,features_test,
#                                                       df,
#                             cutoff_pos_df_hash,
#                             is_min_total=False,total_cri=2,top_n=10,
#                             sim_cutoff_dfs=sim_cutoff_dfs_actual,
#                             imp_feats_count_list=imp_feats_count_list)
# print(f'{k_sel=}')
# print('cutoff best')
# print(top_mean_best)
# print(' ** selected test **')
# test_sel[['k','client_seed','probs']]

In [372]:
test_sel=None
#array([ 10, 536, 371, 333, 518])
k_list = top_mean_actual[8][:5]['k'].values
print(k_list)
for k_sel in k_list:
    row = df.iloc[k_sel]
#     print(row)
    mask = (test['probs']>=row['cutoff']) & (test['probs']<=row['cutoff_2'])
    cur_test_sel = test[mask]
    cur_test_sel['k']=k_sel
    if test_sel is None:
        test_sel = cur_test_sel
    else:
        test_sel = pd.concat([test_sel,cur_test_sel],axis=0)
    print('Test Size:',len(test_sel))
test_sel[['k','roll_actual','client_seed']]

[488 464 440 490 502]
Test Size: 1
Test Size: 2
Test Size: 3
Test Size: 4
Test Size: 5


,k,roll_actual,client_seed
5881,488,5332,woxpwoxpwoxpwoxpwoxp32f9243dc110324c13bda0256adc254243ac50a9c29e9b4c5183e40f71473e5b
5854,464,9871,woxpwoxpwoxpwoxpwoxpf598f694dca32659bd691ad98b4bd6954c92d880706c55c4a018722771531723
7838,440,440,woxpwoxpwoxpwoxpwoxpb88ce29497592453da87cc89b117f0c0a6c8b957201f17f03f55b9fa429dc222
1957,490,7858,woxpwoxpwoxpwoxpwoxp54e0eae91d1d51ecb3b43ee662d0b91f29e47a1a371cbe606c7dd78ba40c8066
5931,502,9265,woxpwoxpwoxpwoxpwoxp2b657e42345ef6fdae007a5b953e0115b79cb88370b776230667abb90b0be544


In [ ]:
imp_feats = [col for col in imp_df['feature'].values if (col.startswith('roll_'))][:10] 
test_sel[['k']+imp_feats]

In [ ]:
sim_hash_group[sim_hash_group['k']==24]

In [ ]:
sim_cutoff_df[sim_cutoff_df['k']==24]

In [ ]:
def compute_total(data,cutoffs,cols):
    mask = None
    for cutoff,col in zip(cutoffs,cols):
        cur_mask =(data[col]>=cutoff) 
        if mask is None:
            mask = cur_mask
        else:
            mask = mask & cur_mask
    data_filt = data[mask]
    total = len(data_filt)
    return total,data_filt

In [ ]:
cols=['roll_20_nonce','roll_14_nonce','roll_3_nonce','roll_16_nonce','roll_11_nonce']
cols=['roll_43_nonce','roll_56_nonce','roll_31_nonce','roll_20_nonce','roll_45_nonce']
cols=['roll_31_nonce','roll_52_nonce','roll_15_nonce','roll_45_nonce','roll_2_nonce']
cols=['roll_31_nonce','roll_52_nonce']
cols=['roll_52_nonce','roll_31_nonce','roll_24_nonce','roll_49_nonce','roll_41_nonce']
cols=['roll_mean_25_50_nonce','roll_52_nonce','roll_31_nonce']
cols=['roll_52_nonce','roll_31_nonce']
cols=['roll_46_nonce','roll_25_nonce']
cols=['roll_42_nonce','roll_37_nonce']

# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce','roll_12_nonce','roll_18_nonce']
# cols=['roll_20_nonce','roll_15_nonce','roll_13_nonce']
# cols=['roll_6_nonce','roll_11_nonce','roll_12_nonce','roll_10_nonce',]




In [ ]:
def get_cutoff(data,data_hash,col,quantile):
    mask=create_target_mask(data_hash)
#     cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].mean()
#     cutoff = data_hash[mask][col].quantile(0.5)
    cutoff = data[col].mean()-data_hash[col].mean()+data_hash[mask][col].quantile(quantile)
    return cutoff



best_ratio =0
best_success = 0
best_total = 0
best_cutoffs = []
best_quantile = 0
best_actual = test_filt
# for quantile in np.linspace(0.01,0.95,50):
for quantile in np.linspace(0.01,0.95,50):
    cutoffs=[]
    for col in cols:
        #use entire hash to determine cutoff
        cutoff = get_cutoff(test,test_hash,col,quantile)
        cutoffs.append(cutoff)
    
#     print(cutoffs)
    total,filt_inter = compute_total(test_filt_hash,cutoffs,cols)
    mask2=create_target_mask(filt_inter)
#     mask2 = (filt_inter['roll_actual']>=9000)
    success = len(filt_inter[mask2])
    
    total_actual,filt_inter_actual = compute_total(test_filt,cutoffs,cols)
    mask2=create_target_mask(filt_inter_actual)

#     mask2 = (filt_inter_actual['roll_actual']>=9000)
    
    if total_actual==0:
        success_actual=0
        ratio_actual=0
    else:
        success_actual = len(filt_inter_actual[mask2])
        ratio_actual = success_actual / total_actual
    print('\nquantile:',quantile)
    print('Actuals:',ratio_actual,success_actual,total_actual)
    
    if total==0:
        ratio=0
    else:
        ratio = success / total
    print('Hash:',ratio,success,total)
#     print(quantile,cutoffs)
    print(total,total_actual)
#     print(total,success)
    if (total==0) or (success==0) : #or (total_actual<1):
        break

#     print(ratio,success,total)
    if ratio >= best_ratio:
        best_ratio = ratio
        best_success = success
        best_total = total
        best_cutoffs = cutoffs
        best_quantile = quantile
        best_actual = filt_inter_actual
print(best_quantile)        
print(best_ratio,best_success,best_total)        
print(best_cutoffs)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
mask2=create_target_mask(best_actual)
success = len(best_actual[mask2])
total = len(best_actual)
ratio = success / total
print(ratio,success,total)

In [ ]:
test_sel = best_actual.copy()

In [ ]:


def get_actual_based_data(data,data_hash,data_filt,cols,total_cutoff=3):

    best_filt_inter = data
    best_total= 0
    best_quantile = 0
    for quantile in np.linspace(0.01,0.95,50):
        cutoffs=[]
        for col in cols:
            cutoff = get_cutoff(data,data_hash,col,quantile)
        #     cutoff = get_cutoff(test,test_hash,col)
            cutoffs.append(cutoff)
#         print(quantile,cutoffs)

        total,filt_inter=compute_total(data_filt,cutoffs,cols)
        if total<total_cutoff:
            break
        best_filt_inter = filt_inter
        best_total = total
        best_quantile = quantile
        
    return best_filt_inter,best_total,best_quantile

In [ ]:
for cutoff in range(0,11):

    best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,test_filt,cols,
                                                                        total_cutoff=cutoff)
#     print('quantile:',best_quantile)
    mask2=create_target_mask(best_filt_inter)
    # mask2 = (best_filt_inter['roll_actual']>=9000)
    success = len(best_filt_inter[mask2])
    if best_total==0:
        ratio=0
        success=0
    else:
        success = len(best_filt_inter[mask2])
        ratio = success / best_total
    print(cutoff, ratio,success,best_total)
    # print(test_filt[mask]['roll_actual'].describe())

In [ ]:
test_sel =best_filt_inter.copy()

In [ ]:
best_filt_inter,best_total,best_quantile=get_actual_based_data(test_filt,test_filt_hash,test_filt,cols,
                                                              total_cutoff=4)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
success = len(best_filt_inter[mask2])
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
# mask_actual = (test_probs['probs']>=row['cutoff']) & (test_probs['probs']<=row['cutoff_2'])
# idx= test_probs[mask_actual].index
# mask_hash = test_hash_probs.index.isin(idx)
# test_filt_hash_pos = test_hash_probs.loc[mask_hash]

mask_hash = (test_hash_probs['probs']>=row['cutoff']) & (test_hash_probs['probs']<=row['cutoff_2'])
idx= test_hash_probs[mask_hash].index
mask_actual = test_probs.index.isin(idx)
test_filt_pos = test_probs.loc[mask_actual]

best_filt_inter,best_total,best_quantile=get_actual_based_data(test,test_hash,
                                                               test_filt_pos,cols,
                                                              total_cutoff=37)
print('quantile:',best_quantile)
mask2=create_target_mask(best_filt_inter)
# mask2 = (best_filt_inter['roll_actual']>=9000)
if best_total==0:
    ratio=0
    success=0
else:
    success = len(best_filt_inter[mask2])
    ratio = success / best_total
print(ratio,success,best_total)
# print(test_filt[mask]['roll_actual'].describe())

In [ ]:
best_filt_inter

In [ ]:
test_sel=best_filt_inter.copy()

In [ ]:
# analysis_df_copy  = analysis_df.copy()

In [ ]:
def gen_analysis(nonce,train_manual,test_manual,test,models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",save_files=True,gen_probs=True,
                analysis_folder="analysis"):
    if CONT_ANALYSIS:
        analysis_df=pd.read_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv')
    else:
        analysis_df=None
    print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
    for zero_cutoff in range(0,6):
        analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
                            analysis_df=analysis_df)
    print('Train completed')
    analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
    print('Test completed')   
    
    if save_files:
        joblib.dump(xgb_models,f'data/models/models_{nonce}{file_suffix}.dump')
        print('Models Save completed')   

    features_test = [col for col in features if col not in ['target','probs']]
    
    if gen_probs:
        test=generate_probs(True,models,test,features_test)
    else:
        test['probs'] = np.load(f'data/probs/probs_{nonce}{file_suffix}.npy')

    if save_files:
        np.save(f'data/probs/probs_{nonce}{file_suffix}.npy',test['probs'].values)
    print('Probability Prediction completed')   

    test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
                                                models,
                                               np.mean(np.array(tr_last_cutoffs)),
                                                 np.mean(np.array(val_cutoffs)), 
                                                PATTERN_99,HIGH_ANALYSIS,multi_models=multi_models,
                                                  gen_probs=False)
    print(len(analysis_df))
    analysis_df.to_csv(f'data/{analysis_folder}/analysis_df_{nonce}{file_suffix}.csv',index=False)  
    print('Analysis Generation completed')  
    return analysis_df,test

In [ ]:
test['roll_actual'].head()

In [ ]:
%%time
analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="",
                save_files=True,analysis_folder="analysis_25_to_50_seeds"
#                 gen_probs=False
                    )
analysis_df.head()

In [ ]:
%%time
analysis_df_hash,test_hash = gen_analysis(nonce,train_manual,test_manual_hash,test_hash,
                                xgb_models,tr_last_cutoffs,val_cutoffs,
                                multi_models=True,file_suffix="_hash",
                                save_files=True,
                                         )
analysis_df_hash.head()

In [ ]:
# %%time
# if CONT_ANALYSIS:
#     analysis_df=pd.read_csv(f'data/analysis_df_{nonce}.csv')
# else:
#     analysis_df=None
# print(f'Analysis for PATTERN_99:{PATTERN_99} HIGH_ANALYSIS:{HIGH_ANALYSIS}')
# for zero_cutoff in range(0,6):
#     analysis_df=save_train_stats(train_manual,nonce,zero_cutoff,PATTERN_99,HIGH_ANALYSIS,0,
#                         analysis_df=analysis_df)
# print('Train completed')
# analysis_df = save_test_stats(analysis_df, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=True)
# print('Test completed')

In [ ]:
# %%time

# # model_no=5
# # analysis_df = save_test_ml_probs(analysis_df,test,
# #                                 xgb_models[model_no],np.mean(np.array(tr_last_cutoffs)),
# #                                 np.mean(np.array(val_cutoffs)))


# #Save the Models
# joblib.dump(xgb_models,f'models_{nonce}.dump')

# features_test = [col for col in features if col not in ['target','probs']
# test=generate_probs(True,xgb_models,test,features_test)
# np.save(f'probs/probs_{nonce}.npy',test['probs'].values)
                 
# test_filt_all,analysis_df = save_test_ml_probs(analysis_df,test,
#                                             xgb_models,
#                                            np.mean(np.array(tr_last_cutoffs)),
#                                              np.mean(np.array(val_cutoffs)), 
#                                             PATTERN_99,HIGH_ANALYSIS,multi_models=True,
#                                               gen_probs=False)
# print(len(analysis_df))
# analysis_df.to_csv(f'data/analysis/analysis_df_{nonce}.csv',index=False)

In [ ]:
# xgb_models= joblib.load(f'data/models/models_{nonce}.dump')
# for model in xgb_models:
#     temp_probs = model.predict_proba(test[0:5][features_test])
#     print(temp_probs)

In [ ]:
# len(analysis_df[(analysis_df['pattern_99']==True) & (analysis_df['target_high']==False)])

In [ ]:
# analysis_df=pd.read_csv(f'data/analysis/analysis_df_2582.csv')

In [ ]:
def report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     pattern,bHighCheck,
                    zerocolcutoffs,is_ratio_cri_lessthan=False,
                               is_aftval_opp=False,is_aftval_sum=False):
    mask1 = analysis_df['digit'].isin(digitlist)
    if is_ratio_cri_lessthan:
        mask2= (analysis_df['tr_ratio']<ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']>=ratio_cutoff) | (analysis_df['aft_1']>=ratio_cutoff) | (analysis_df['aft_2']>=ratio_cutoff)) 
    else:
        mask2=  (analysis_df['tr_ratio']>=ratio_cutoff) 
        if is_aftval_opp:
            mask2= mask2 & ((analysis_df['aft_all']<ratio_cutoff) | (analysis_df['aft_1']<ratio_cutoff) | (analysis_df['aft_2']<ratio_cutoff)) \
    
        
    mask= mask1 & mask2 \
             & ((analysis_df['aft_all']!=0) & (analysis_df['aft_1']!=0) & (analysis_df['aft_2']!=0)) \
                & (analysis_df['te_total']>=count_cutoff_min) \
                & (analysis_df['te_total']<=count_cutoff_max) \
                & (analysis_df['pattern_99']==pattern) \
                & (analysis_df['target_high']==bHighCheck) \
                & (analysis_df['zero_col_cutoff'].isin(zerocolcutoffs))
        
    if is_aftval_sum:
        mask= mask & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

    filtered = analysis_df[mask]
    
    records  = len(filtered)
    cols = ['tr_ratio','tr_total','te_total']
    
    return filtered

In [ ]:
# %%time

# suffix='_hash'
# digit_list=[1,2,3,4]
# # digit_list=[9,8,7,5]
# zero_col_cutoff_list=[0,1]

# cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#        'pr_tr_cutoff','pr_val_cutoff']
# analysis_list_hash=[]
# mean_list=[]
# model_means=[]
# mask = analysis_df['digit'].isin(digit_list)
# mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
# cutoffs = [(13,20),(13,50),(13,100),(14,20),
#            (14,100),(14,50),(15,20),(15,50),(15,100)]
# cutoffs =[(18,3)]

# # if CONT_ANALYSIS:
# #     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# # else:
# #     analysis_mean_df = None

# analysis_mean_df_hash =None    
# mean_df_hash=None

# for (ratio_cutoff,count_cutoff) in cutoffs:

#     filtered = report_exceptional_teratio(analysis_df_hash,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                          )
#     print(len(filtered))
# #     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
# #                                     xgb_models,
# #                                    np.mean(np.array(tr_last_cutoffs)),
# #                                      np.mean(np.array(val_cutoffs)), 
# #                                   PATTERN_99,HIGH_ANALYSIS,
# #                                   gen_probs=False,
# #                                   multi_models=True )
#     analysis_list_hash.append(filtered)
#     records  = len(filtered)
# #     print('Records:',records)
# #     cur_mean = filtered[cols].mean(axis=0)
#     cur_mean = pd.Series()
#     pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                        'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                       'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                      'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                      'te_ratio':'te_total'}
    
#     for pr_col,pr_count_col in pr_count_dict.items():
#         cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
#     model_mean = cur_mean.mean()
#     te_total_sum = filtered['te_total'].sum(axis=0)
# #     model_mean = filtered[cols].mean(axis=None)
# #     print('Overall mean:',model_mean)
# #     print(cur_mean)

#     cur_mean.loc['overall_mean']=model_mean
#     cur_mean.loc['ratio_cutoff']=ratio_cutoff
#     cur_mean.loc['total_cutoff']=count_cutoff
#     cur_mean.loc['records']=records
#     cur_mean.loc['te_total_sum']=te_total_sum

#     if mean_df_hash is None:
#         mean_df_hash = pd.DataFrame({'temp':cur_mean})    
#     else:
#         mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

# mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
# mean_df_hash['nonce']=nonce
# mean_df_hash['pattern_99']=PATTERN_99
# mean_df_hash['target_high']=HIGH_ANALYSIS
# key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#             'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
#             'overall_mean']
# cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

# mean_df_hash=mean_df_hash[cols_reorder]
# # mean_df
# # temp_df
# if analysis_mean_df_hash is None:
#     analysis_mean_df_hash=mean_df_hash 
# else:
#     analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
# analysis_mean_df_hash

In [ ]:
# %%time

# if not IS_PROD:

#     cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
#            'pr_tr_cutoff','pr_val_cutoff']
#     analysis_list=[]
#     mean_list=[]
#     model_means=[]
#     mask = analysis_df['digit'].isin(digit_list)
#     mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
#     #            (14,100),(14,50),(15,20),(15,50),(15,100)]
#     # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
#     #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
#     # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(18,3)]

# #     if CONT_ANALYSIS:
# #         analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
# #     else:
# #         analysis_mean_df = None

#     analysis_mean_df =None    
#     mean_df=None

#     for (ratio_cutoff,count_cutoff) in cutoffs:

#         filtered = report_exceptional_teratio(analysis_df,digit_list,
#                      ratio_cutoff,count_cutoff,10000,
#                      PATTERN_99,HIGH_ANALYSIS,zero_col_cutoff_list,is_aftval_sum=True,
#                                           is_aftval_opp=True,
#                                              )
   
    
# #         filtered = analysis_df[mask1]
#         print(len(filtered))
# #         test_filt_all,filtered = save_test_ml_probs(filtered,test,
# #                                         xgb_models,
# #                                        np.mean(np.array(tr_last_cutoffs)),
# #                                          np.mean(np.array(val_cutoffs)), 
# #                                       PATTERN_99,HIGH_ANALYSIS,
# #                                       gen_probs=False,
# #                                       multi_models=True )
#         analysis_list.append(filtered)
#         records  = len(filtered)
#     #     print('Records:',records)
# #         cur_mean = filtered[cols].mean(axis=0)
#         cur_mean = pd.Series()
#         pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
#                            'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
#                           'pr_mean':'te_mean_count','pr_med':'te_med_count',
#                          'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
#                          'te_ratio':'te_total'}

#         for pr_col,pr_count_col in pr_count_dict.items():
#             cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
#         model_mean = cur_mean.mean()
#         te_total_sum = filtered['te_total'].sum(axis=0)
        
        
#     #     model_mean = filtered[cols].mean(axis=None)
#     #     print('Overall mean:',model_mean)
#     #     print(cur_mean)

#         cur_mean.loc['te_total_sum']=te_total_sum
#         cur_mean.loc['overall_mean']=model_mean
#         cur_mean.loc['ratio_cutoff']=ratio_cutoff
#         cur_mean.loc['total_cutoff']=count_cutoff
#         cur_mean.loc['records']=records

#         if mean_df is None:
#             mean_df = pd.DataFrame({'temp':cur_mean})    
#         else:
#             mean_df=pd.concat([mean_df,cur_mean],axis=1)

#     mean_df=mean_df.transpose().reset_index(drop=True)
#     mean_df['nonce']=nonce
#     mean_df['pattern_99']=PATTERN_99
#     mean_df['target_high']=HIGH_ANALYSIS
#     key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
#                 'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
#                 'pr_mean','pr_med','overall_mean']
#     cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

#     mean_df=mean_df[cols_reorder]
#     # mean_df
#     # temp_df
#     if analysis_mean_df is None:
#         analysis_mean_df=mean_df 
#     else:
#         analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
# analysis_mean_df

In [ ]:
%%time

suffix='_hash'
digit_list=[1,2,3,4]
digit_list=[9,8,7,5]
# digit_list=[6]
zero_col_cutoff_list=[0,1]
only_better_probs = False
better_prob_cutoff= 11
cutoff_col = 'te_ratio'


cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
       'pr_tr_cutoff','pr_val_cutoff']
analysis_list_hash=[]
mean_list=[]
model_means=[]
mask = analysis_df['digit'].isin(digit_list)
mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
cutoffs = [(13,15),(13,20),(13,50),(13,100),(14,15),(14,20),
           (14,100),(14,50),(15,15),(15,20),(15,50),(15,100),(18,50),(20,50),(25,50),]
# cutoffs =[(14,20)]

# if CONT_ANALYSIS:
#     analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}{suffix}.csv')
# else:
#     analysis_mean_df = None

analysis_mean_df_hash =None    
mean_df_hash=None

for (ratio_cutoff,count_cutoff) in cutoffs:

    mask1= mask & (analysis_df_hash['tr_ratio']>=ratio_cutoff) \
                & (analysis_df_hash['te_total']>=count_cutoff) \
                & (analysis_df_hash['pattern_99']==PATTERN_99) \
                & (analysis_df_hash['target_high']==HIGH_ANALYSIS) \
    
    if only_better_probs:
        mask1 = mask1 & (analysis_df_hash[cutoff_col]>=better_prob_cutoff)

    filtered = analysis_df_hash[mask1]
    print(len(filtered))
#     test_filt_all,filtered = save_test_ml_probs(filtered,test_hash,
#                                     xgb_models,
#                                    np.mean(np.array(tr_last_cutoffs)),
#                                      np.mean(np.array(val_cutoffs)), 
#                                   PATTERN_99,HIGH_ANALYSIS,
#                                   gen_probs=False,
#                                   multi_models=True )
    analysis_list_hash.append(filtered)
    records  = len(filtered)
#     print('Records:',records)
#     cur_mean = filtered[cols].mean(axis=0)
    cur_mean = pd.Series()
    pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                       'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                      'pr_mean':'te_mean_count','pr_med':'te_med_count',
                     'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                     'te_ratio':'te_total'}
    
    for pr_col,pr_count_col in pr_count_dict.items():
        cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()
    
    model_mean = cur_mean.mean()
    te_total_sum = filtered['te_total'].sum(axis=0)
#     model_mean = filtered[cols].mean(axis=None)
#     print('Overall mean:',model_mean)
#     print(cur_mean)

    cur_mean.loc['overall_mean']=model_mean
    cur_mean.loc['ratio_cutoff']=ratio_cutoff
    cur_mean.loc['total_cutoff']=count_cutoff
    cur_mean.loc['records']=records
    cur_mean.loc['te_total_sum']=te_total_sum

    if mean_df_hash is None:
        mean_df_hash = pd.DataFrame({'temp':cur_mean})    
    else:
        mean_df_hash=pd.concat([mean_df_hash,cur_mean],axis=1)

mean_df_hash=mean_df_hash.transpose().reset_index(drop=True)
mean_df_hash['nonce']=nonce
mean_df_hash['pattern_99']=PATTERN_99
mean_df_hash['target_high']=HIGH_ANALYSIS
key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
            'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff','pr_mean','pr_med',
            'overall_mean']
cols_reorder = key_cols+ [col for col in mean_df_hash.columns if col not in key_cols]

mean_df_hash=mean_df_hash[cols_reorder]
# mean_df
# temp_df
if analysis_mean_df_hash is None:
    analysis_mean_df_hash=mean_df_hash 
else:
    analysis_mean_df_hash = pd.concat([analysis_mean_df,mean_df_hash],axis=0,ignore_index=True)
analysis_mean_df_hash

In [ ]:
analysis_list_hash[0]

In [ ]:
print(len(analysis_df_hash))
len(analysis_df_hash[analysis_df_hash['te_ratio']!=0])

In [ ]:
print(len(analysis_df))
len(analysis_df[analysis_df['te_ratio']!=0])

In [ ]:
%%time

if not False: #IS_PROD:

    cols = ['te_ratio','pr_mean','pr_med','pr_1','pr_2','pr_3','pr_4','pr_5',
           'pr_tr_cutoff','pr_val_cutoff']
    analysis_list=[]
    mean_list=[]
    model_means=[]
    mask = analysis_df['digit'].isin(digit_list)
    mask = mask & analysis_df['zero_col_cutoff'].isin(zero_col_cutoff_list)
#     cutoffs = [(13,20),(13,50),(13,100),(14,20),
#                (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(9,20),(9,50),(9,100),(10,20),(10,50),(10,100),(11,20),(11,50),(11,100),(12,20),(12,50),(13,20),(13,50),(13,100),(14,20),
    #            (14,100),(14,50),(15,20),(15,50),(15,100)]
    # cutoffs = [(11,20),(11,50),(11,100),(12,20),(12,50),(12,100),
    #            (13,20),(13,50),(13,100),(14,20),(14,50),(14,100),(15,20),(15,50),]
    # cutoffs = [(12,20),(12,50)]
#     cutoffs =[(14,20)]

    if CONT_ANALYSIS:
        analysis_mean_df=pd.read_csv(f'data/analysis/analysis_mean_df_{nonce}.csv')
    else:
        analysis_mean_df = None

    analysis_mean_df =None    
    mean_df=None

    for i,(ratio_cutoff,count_cutoff) in enumerate(cutoffs):

    #     mask1= mask & (analysis_df['cutoff']==5) \
    #                 & (analysis_df['te_total']>=count_cutoff) \
    #                 & (analysis_df['tr_ratio']>=ratio_cutoff) \
    #                 & (analysis_df['zero_col_cutoff']==3) \
        mask1= mask & (analysis_df['tr_ratio']>=ratio_cutoff) \
                    & (analysis_df['te_total']<=count_cutoff) \
                    & (analysis_df['pattern_99']==PATTERN_99) \
                    & (analysis_df['target_high']==HIGH_ANALYSIS) \
    #                  & (analysis_df['zero_col_cutoff'].isin([0,1])) \
    #                  & (analysis_df['cutoff'].isin([4,5])) \
       #                 & ((analysis_df['aft_all']>13) | (analysis_df['aft_1']>13) | (analysis_df['aft_2']>13)) \
        #                 & ((analysis_df['aft_all']<13) | (analysis_df['aft_1']<13) | (analysis_df['aft_2']<13)) \
        #                  & ((analysis_df['aft_all']+analysis_df['aft_1']+analysis_df['aft_2'])>=36) \

        if only_better_probs:
    #         mask1 = mask1 & (analysis_df.index.isin(analysis_list_hash[i].index))
            filtered=analysis_df.loc[analysis_list_hash[i].index][mask1]
        else:
            filtered = analysis_df[mask1]
            
        print(len(filtered))
        test_filt_all,filtered = save_test_ml_probs(filtered,test,
                                        xgb_models,
                                       np.mean(np.array(tr_last_cutoffs)),
                                         np.mean(np.array(val_cutoffs)), 
                                      PATTERN_99,HIGH_ANALYSIS,
                                      gen_probs=False,
                                      multi_models=True )
        analysis_list.append(filtered)
        records  = len(filtered)
    #     print('Records:',records)
#         cur_mean = filtered[cols].mean(axis=0)
        cur_mean = pd.Series()
        pr_count_dict = {'pr_1':'te_pr_1_count','pr_2':'te_pr_2_count','pr_3':'te_pr_3_count',
                           'pr_4':'te_pr_4_count','pr_5':'te_pr_5_count',
                          'pr_mean':'te_mean_count','pr_med':'te_med_count',
                         'pr_tr_cutoff':'te_tr_count','pr_val_cutoff':'te_val_count',
                         'te_ratio':'te_total'}

        for pr_col,pr_count_col in pr_count_dict.items():
            cur_mean.loc[pr_col]= (filtered[pr_col]*filtered[pr_count_col]).round(0).sum() / filtered[pr_count_col].sum()

        
        
        
        model_mean = cur_mean.mean()
        te_total_sum = filtered['te_total'].sum(axis=0)
        
        
    #     model_mean = filtered[cols].mean(axis=None)
    #     print('Overall mean:',model_mean)
    #     print(cur_mean)

        cur_mean.loc['te_total_sum']=te_total_sum
        cur_mean.loc['overall_mean']=model_mean
        cur_mean.loc['ratio_cutoff']=ratio_cutoff
        cur_mean.loc['total_cutoff']=count_cutoff
        cur_mean.loc['records']=records

        if mean_df is None:
            mean_df = pd.DataFrame({'temp':cur_mean})    
        else:
            mean_df=pd.concat([mean_df,cur_mean],axis=1)

    mean_df=mean_df.transpose().reset_index(drop=True)
    mean_df['nonce']=nonce
    mean_df['pattern_99']=PATTERN_99
    mean_df['target_high']=HIGH_ANALYSIS
    key_cols = ['nonce','pattern_99','target_high','ratio_cutoff','total_cutoff', 
                'records','te_total_sum','te_ratio','pr_tr_cutoff','pr_val_cutoff',
                'pr_mean','pr_med','overall_mean']
    cols_reorder = key_cols+ [col for col in mean_df.columns if col not in key_cols]

    mean_df=mean_df[cols_reorder]
    # mean_df
    # temp_df
    if analysis_mean_df is None:
        analysis_mean_df=mean_df 
    else:
        analysis_mean_df = pd.concat([analysis_mean_df,mean_df],axis=0,ignore_index=True)
    analysis_mean_df

In [ ]:
analysis_mean_df

In [ ]:
analysis_list[0]

In [ ]:
print(test_hash['probs'].describe())
cutoff = test_hash['probs'].quantile(q=0.97)
print(cutoff)
mask = test_hash['probs']>cutoff
total = len(test_hash[mask])
mask2 = test_hash['roll_actual']>=8944
success = len(test_hash[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
print(test['probs'].describe())
cutoff = test['probs'].quantile(q=0.97)
print(cutoff)
mask = test['probs']>cutoff
total = len(test[mask])
mask2 = test['roll_actual']>=8944
success = len(test[mask & mask2])
ratio = success / total
print(ratio,success,total)

In [ ]:
if not IS_PROD:
    analysis_mean_df.to_csv(f'data/analysis/analysis_mean_df_{nonce}.csv',index=False)

##### Production Code

In [ ]:
# digitlist=[9,8,7,5]
# zerocolcutoffs=list(range(0,6))

# digitlist=[1,2,3,4]
digitlist=[9,8,7,5]
zerocolcutoffs=[4,5]
ratio_cutoff=15
count_cutoff_min=20
count_cutoff_max=10000
analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
                     ratio_cutoff,count_cutoff_min,count_cutoff_max,
                     PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
anal_size=len(analysis_filt)
test_size=analysis_filt['te_total'].sum()
print(f'Analysis Selection Size: {anal_size} Total Test Size:{test_size}')

In [ ]:
analysis_filt

In [ ]:
#Fetch test selection based on te ratio 

def get_test_from_analysis(analysis_df,test,
              pattern,bHighCheck,
              is_prod=False,):
    
    start = time.time()
    features_test = [col for col in features if col not in ['target','probs']]
    if  is_prod:
        analysis_df_filt= analysis_df
    else:
        mask = (analysis_df['pattern_99']==pattern) & (analysis_df['target_high']==bHighCheck) 
        analysis_df_filt = analysis_df[mask]
    
    test_filt_all = None
    for row in analysis_df_filt.itertuples():
        digit = row.digit
        cutoff = row.cutoff
        after_val = row.afterval
        bHighCheck = row.target_high
        zero_col_cutoff = row.zero_col_cutoff

        test_filt_df = validate_pattern(test,bHighCheck,zero_col_cutoff,
                                        digit,cutoff,after_val)
        filt_size = len(test_filt_df)
        print(f'{filt_size=}')
        mask=create_target_mask(test_filt_df)
        success = len(test_filt_df[mask])
#         success=len(test_filt_df[test_filt_df['roll_actual']>HIGH_TARGET])
        print(f'{success=}')        
        
        if filt_size==0:
            print(f'No Record for zero_col_cutoff:{zero_col_cutoff} digit:{digit} cutoff:{cutoff} after_val:{after_val}')
            continue

        if is_prod:
            if test_filt_all is None:
                test_filt_all=test_filt_df
            else:
                test_filt_all=pd.concat([test_filt_all,test_filt_df],axis=0)
            
            
    end = time.time()
    print(f'Function time:{end-start}')   

    return test_filt_all

In [ ]:
# temp_sel = save_test_stats(analysis_filt, test_manual,PATTERN_99,HIGH_ANALYSIS,is_prod=False)
# print(len(temp_sel))

In [ ]:
# %%time
# test_sel = get_test_from_analysis(analysis_filt,test,PATTERN_99,HIGH_ANALYSIS,
#                                   is_prod=True)
# print(len(test_sel))

In [ ]:
%%time
test_sel,ret_analysis_df = save_test_ml_probs(analysis_filt,
                                              test,
                                    xgb_models,
                                   np.mean(np.array(tr_last_cutoffs)),
                                  np.mean(np.array(val_cutoffs)), 
                                  PATTERN_99,HIGH_ANALYSIS,
                                  is_prod=True,
                                 gen_probs=False,
                                  prob_cutoff=0.3,
                                  multi_models=True )
print(f'Selected Test Size:{len(test_sel)}')

In [ ]:
if not IS_PROD:
    success=len(test_sel[test_sel['roll_actual']>=9000])
    print(success)
    print(success / len(test_sel))

In [ ]:
test_sel.head()

In [373]:
# def get_random_client_seed(test):
#     size = len(test)
#     test = test.reset_index(drop=True).sample(n=size).reset_index(drop=True)
#     rand_pos = random.randint(0,size-1)
#     print(f'rand_pos:{rand_pos}')
#     return test.iloc[rand_pos]['client_seed'],test

def get_random_client_seed(test):
    size = len(test)
    final_test= test.sample(n=size)
    rand_pos=random.randint(0,size-1)
    sample=final_test.reset_index(drop=True).loc[rand_pos]
#     rand_pos = sample.index[0]
    print(f'rand_pos:{rand_pos}')
    return sample['client_seed'],final_test

In [374]:
client_seed,final_test = get_random_client_seed(test_sel)
print('Client Seed Selected')
print(client_seed)
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')
# final_test.head()

rand_pos:0
Client Seed Selected
woxpwoxpwoxpwoxpwoxpf598f694dca32659bd691ad98b4bd6954c92d880706c55c4a018722771531723


In [375]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_13,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll

In [ ]:
final_test.reset_index()[final_test.reset_index()['index']==6826]

After Actual SEED is known

In [376]:
actual_seed="e658f73f9831527c16614b72cf87f0a9718961a6099024b6301b10b54dc166d0"
# actual_seed=cur_hash_list[4]
# print(f'{actual_seed=}')

In [377]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       final_test['client_seed'],
                                       nonce)
final_test['roll_actual_dummy']=final_test['roll_actual'].copy()
final_test['roll_actual']=roll_array
# final_test.to_csv(f'data/analysis/final_test_{nonce}.csv')

In [378]:
final_test

,roll_actual,seed,hash,client_seed,roll_0,roll_1,roll_2,roll_3,roll_4,roll_5,roll_6,roll_7,roll_8,roll_9,roll_10,roll_11,roll_12,roll_13,roll_14,roll_15,roll_16,roll_17,roll_18,roll_19,roll_20,roll_21,roll_22,roll_23,roll_24,roll_25,roll_26,roll_27,roll_28,roll_29,roll_30,roll_31,roll_32,roll_33,roll_34,roll_35,roll_36,roll_37,roll_38,roll_39,roll_40,roll_41,roll_42,roll_43,roll_44,roll_45,roll_46,roll_47,roll_48,roll_49,roll_50,roll_51,roll_52,roll_53,roll_54,roll_55,roll_56,roll_57,roll_58,roll_59,roll_60,count_lt_1000,count_gt_9000,count_gt_8000,count_gt_7000,count_gt_6000,count_gt_5000,count_gt_4000,count_gt_3000,count_gt_2000,count_gt_1000,count_hi,count_lo,count_lo_hi,count_gt_9000_k,count_lt_1000_k,total_dig_9,total_dig_0,roll_mean,roll_std,count_lt_1000_25_50,count_gt_9000_25_50,count_gt_8000_25_50,count_gt_7000_25_50,count_gt_6000_25_50,count_gt_5000_25_50,count_gt_4000_25_50,count_gt_3000_25_50,count_gt_2000_25_50,count_gt_1000_25_50,count_hi_25_50,count_lo_25_50,count_lo_hi_25_50,count_gt_9000_k_25_50,count_lt_1000_k_25_50,total_dig_9_25_50,total_dig_0_25_50,roll_mean_25_50,roll_std_25_50,count_lt_1000_50_60,count_gt_9000_50_60,count_gt_8000_50_60,count_gt_7000_50_60,count_gt_6000_50_60,count_gt_5000_50_60,count_gt_4000_50_60,count_gt_3000_50_60,count_gt_2000_50_60,count_gt_1000_50_60,count_hi_50_60,count_lo_50_60,count_lo_hi_50_60,count_gt_9000_k_50_60,count_lt_1000_k_50_60,total_dig_9_50_60,total_dig_0_50_60,roll_mean_50_60,roll_std_50_60,count_lt_1000_1_60,count_gt_9000_1_60,count_gt_8000_1_60,count_gt_7000_1_60,count_gt_6000_1_60,count_gt_5000_1_60,count_gt_4000_1_60,count_gt_3000_1_60,count_gt_2000_1_60,count_gt_1000_1_60,count_hi_1_60,count_lo_1_60,count_lo_hi_1_60,count_gt_9000_k_1_60,count_lt_1000_k_1_60,total_dig_9_1_60,total_dig_0_1_60,roll_mean_1_60,roll_std_1_60,roll_0_nonce,roll_1_nonce,roll_2_nonce,roll_3_nonce,roll_4_nonce,roll_5_nonce,roll_6_nonce,roll_7_nonce,roll_8_nonce,roll_9_nonce,roll_10_nonce,roll_11_nonce,roll_12_nonce,roll_13_nonce,roll_14_nonce,roll_15_nonce,roll_16_nonce,roll_17_nonce,roll_18_nonce,roll_19_nonce,roll_20_nonce,roll_21_nonce,roll_22_nonce,roll_23_nonce,roll_24_nonce,roll_25_nonce,roll_26_nonce,roll_27_nonce,roll_28_nonce,roll_29_nonce,roll_30_nonce,roll_31_nonce,roll_32_nonce,roll_33_nonce,roll_34_nonce,roll_35_nonce,roll_36_nonce,roll_37_nonce,roll_38_nonce,roll_39_nonce,roll_40_nonce,roll_41_nonce,roll_42_nonce,roll_43_nonce,roll_44_nonce,roll_45_nonce,roll_46_nonce,roll_47_nonce,roll_48_nonce,roll_49_nonce,roll_50_nonce,roll_51_nonce,roll_52_nonce,roll_53_nonce,roll_54_nonce,roll_55_nonce,roll_56_nonce,roll_57_nonce,roll_58_nonce,roll_59_nonce,roll_60_nonce,count_lt_1000_nonce,count_gt_9000_nonce,count_gt_8000_nonce,count_gt_7000_nonce,count_gt_6000_nonce,count_gt_5000_nonce,count_gt_4000_nonce,count_gt_3000_nonce,count_gt_2000_nonce,count_gt_1000_nonce,count_hi_nonce,count_lo_nonce,count_lo_hi_nonce,count_gt_9000_k_nonce,count_lt_1000_k_nonce,total_dig_9_nonce,total_dig_0_nonce,roll_mean_nonce,roll_std_nonce,count_lt_1000_25_50_nonce,count_gt_9000_25_50_nonce,count_gt_8000_25_50_nonce,count_gt_7000_25_50_nonce,count_gt_6000_25_50_nonce,count_gt_5000_25_50_nonce,count_gt_4000_25_50_nonce,count_gt_3000_25_50_nonce,count_gt_2000_25_50_nonce,count_gt_1000_25_50_nonce,count_hi_25_50_nonce,count_lo_25_50_nonce,count_lo_hi_25_50_nonce,count_gt_9000_k_25_50_nonce,count_lt_1000_k_25_50_nonce,total_dig_9_25_50_nonce,total_dig_0_25_50_nonce,roll_mean_25_50_nonce,roll_std_25_50_nonce,count_lt_1000_50_60_nonce,count_gt_9000_50_60_nonce,count_gt_8000_50_60_nonce,count_gt_7000_50_60_nonce,count_gt_6000_50_60_nonce,count_gt_5000_50_60_nonce,count_gt_4000_50_60_nonce,count_gt_3000_50_60_nonce,count_gt_2000_50_60_nonce,count_gt_1000_50_60_nonce,...,roll_actual_14,roll_actual_15,roll_actual_16,roll_actual_17,roll_actual_18,roll_actual_19,roll_actual_20,roll_actual_21,roll_actual_22,roll_actual_23,roll_actual_24,roll_actual_25,roll_actual_26,roll_actual_27,roll_actual_28,roll_actual_29,roll

In [ ]:
final_test.to_csv(f'data/final_test_{nonce}_{file_pattern_str}_pattern.csv')

In [ ]:
LOW_TARGET

In [ ]:
mask = create_target_mask(final_test)
success_pos = final_test[mask].index

if HIGH_ANALYSIS:
    success_pos = final_test[final_test['roll_actual']>=HIGH_TARGET].index
else:
    success_pos = final_test[final_test['roll_actual']<LOW_TARGET].index
print(len(success_pos))
print(f'success_pos:{success_pos}')

# target = set([17, 19, 34, 35, 44, 47])
# matched  = set(list(success_pos)).intersection(target)
# print(matched)

# print(len(matched)/len(success_pos))

In [ ]:
test_sel.head()

In [ ]:
final_test

In [ ]:
# Vectorize the function
vectorized_calculate_roll = np.vectorize(calculate_roll)

# Compute the roll values for the input arrays
roll_array = vectorized_calculate_roll(actual_seed,
                                       results_df_test['client_seed'],
                                       nonce)

results_df_test['roll_actual']=roll_array
test_manual = generate_test_features(results_df_test)
test = generate_features_full(results_df_test,False,feature_chain_length)


analysis_df,test = gen_analysis(nonce,train_manual,test_manual,test,xgb_models,tr_last_cutoffs,val_cutoffs,
                multi_models=True,file_suffix="")
analysis_df.head()

In [ ]:
# ratio_cutoff=18
# count_cutoff_min=3
# # test_sel_options= [([1],[0,1],10000),
# #                     ([2],[0,1],10000),
# #                    ([3],[0,1],10000),
# #                    ([4],[0,1],10000),
# #                     ([5],[0,1],10000),
# # #                     ([6],[0,1],10000),
# #                    ([7],[0,1],10000),
# #                    ([8],[0,1],10000),
# #                    ([9],[0,1],10000),
# #                     ([1],[4,5],10000),
# #                     ([2],[4,5],10000),
# #                    ([3],[4,5],10000),
# #                    ([4],[4,5],10000),
# #                     ([5],[4,5],10000),
# # #                     ([6],[4,5],10000),
# #                    ([7],[4,5],10000),
# #                    ([8],[4,5],10000),
# #                    ([9],[4,5],10000),
# #                   ]

# test_sel_options= [([1,2,3,4],[0,1],10000),
#                     ([9,8,7,5],[0,1],10000),
#                    ([6],[0,1],10000),
#                     ([1,2,3,4],[4,5],10000),
#                     ([9,8,7,5],[4,5],10000),
#                    ([6],[4,5],10000),
#                   ]
# test_sel_options= [([1],[0],10000),]

#                    #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),

# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]


# # test_sel_options= [([9,8,7,6,5,1,2,3,4],[0,1,4,5],10000)]
# # test_sel_options= [([9,8,7,5],[4,5],10000)]
# # test_sel_options= [([6],[4,5],10000)]

# # test_sel_options= [([1,2,3,4],[0,1],10),
# #                     ([9,8,7,5,6],[0,1],10),
# #                     ([1,2,3,4],[4,5],10),
# #                     ([1,2,3,4],[4,5],15),
# #                     ([1,2,3,4],[4,5],20),
# #                     ([1,2,3,4],[4,5],25),
# #                     ([6],[0,1],10000),
# #                     ([9,8,7,5],[0,1],10000),
# #                     ([1,2,3,4],[0,1],10000),
# #                   ]

# anal_sizes =[]
# analysis_filts=[]
# test_sizes=[]
# filt_options=[]
# for option in test_sel_options:
#     digitlist=option[0]
#     zerocolcutoffs=option[1]
#     count_cutoff_max=option[2]
#     analysis_filt = report_exceptional_teratio(analysis_df,digitlist,
#                          ratio_cutoff,count_cutoff_min,count_cutoff_max,
#                          PATTERN_99,HIGH_ANALYSIS,zerocolcutoffs)
#     anal_size=len(analysis_filt)
#     if anal_size!=0:
#         test_size=analysis_filt['te_total'].sum()
#         print(f'option: {option} Analysis Selection Size: {anal_size} Total Test Size:{test_size}')
#         break
# #         anal_sizes.append(anal_size)
# #         analysis_filts.append(analysis_filt)
# #         filt_options.append(option)
# #         test_sizes.append(test_size)
#     else:
#         print(f'No records for option:{option}')

# # sel_option = np.argmin(np.array(test_sizes))
# # print(f'selected option: {filt_options[sel_option]} \
# #       Analysis Selection Size: {anal_sizes[sel_option]} Total Test Size:{test_sizes[sel_option]}')

# # analysis_filt = analysis_filts[sel_option]

#### END

Finalize Training

In [ ]:
tr_index

In [ ]:
train.index

In [ ]:
# tr_index = train[30 * train_client_size:(30 + 5) * train_client_size].index
tr_index = train[train_client_size:].index
train_ignored = train[0:30*train_client_size]
# tr_index = list(tr_index) + list(train_ignored[train_ignored['target']==1].index)
print(len(tr_index))
xgb_model_final = xgb.XGBClassifier(**params)

X_tr,y_tr = X.iloc[tr_index],y.iloc[tr_index]
oversample = RandomOverSampler(sampling_strategy='minority',random_state=123)
# fit and apply the transform
X_tr, y_tr = oversample.fit_resample(X_tr, y_tr)
print(pd.Series(y_tr).value_counts())
xgb_model_final.fit(X_tr,y_tr)

In [ ]:
len(X_tr)

In [ ]:
test_probs = xgb_model_final.predict_proba(test)[:,1]

test_labels = convert_probtolabels(test_probs)
subm=pd.DataFrame()
subm['client_seed']= results_df_test['client_seed']
subm['preds'] = test_labels
subm['prob'] = test_probs
score = accuracy_score(y_test,test_labels)
print(f'test score:{score}')

In [ ]:
mask = (subm['prob']<0.52) & (subm['preds']==1)
print(len(subm[mask]))
score = accuracy_score(y_test[mask],test_labels[mask])
print(f'test score:{score}')